Ниже представлен пайплайн с использованием BM25 вместо эмбеддингов для поиска релевантных документов, а также код для оценки его эффективности с использованием тех же метрик.

In [31]:
import requests
import markdown
from bs4 import BeautifulSoup
import re
import numpy as np
import pandas as pd
import time
from sklearn.metrics import ndcg_score
from rank_bm25 import BM25Okapi
import nltk
import string
import os
nltk.download('all')

# Явная установка директории для загрузки NLTK данных
nltk_data_dir = os.path.join(os.getcwd(), 'nltk_data')
os.makedirs(nltk_data_dir, exist_ok=True)
nltk.data.path.append(nltk_data_dir)

# Загрузка необходимых ресурсов NLTK с явным указанием пути
try:
    nltk.download('punkt', download_dir=nltk_data_dir, quiet=True)
    nltk.download('stopwords', download_dir=nltk_data_dir, quiet=True)
    from nltk.tokenize import word_tokenize
    from nltk.corpus import stopwords
    from nltk.stem import PorterStemmer
    stop_words = set(stopwords.words('english'))
except LookupError:
    print("Не удалось загрузить ресурсы NLTK. Используем упрощенную токенизацию.")
    
    # Упрощенная имплементация токенизации без зависимостей NLTK
    def word_tokenize(text):
        """Простая токенизация по пробелам и пунктуации"""
        # Заменяем пунктуацию на пробелы
        for punct in string.punctuation:
            text = text.replace(punct, ' ')
        # Разбиваем по пробелам и фильтруем пустые токены
        return [token for token in text.lower().split() if token]
    
    # Пустой набор стоп-слов
    stop_words = set()
    
    # Упрощенный стеммер
    class SimplePorterStemmer:
        """Очень упрощенная версия стеммера - убирает только окончания -ing, -ed, -s"""
        def stem(self, word):
            if word.endswith('ing'):
                return word[:-3]
            elif word.endswith('ed') and len(word) > 3:
                return word[:-2]
            elif word.endswith('s') and len(word) > 2:
                return word[:-1]
            return word
    
    PorterStemmer = SimplePorterStemmer

class DocumentationQA_BM25:
    def __init__(self):
        self.bm25 = None
        self.doc_paragraphs = []
        self.tokenized_corpus = []
        self.md_list = [
            'https://raw.githubusercontent.com/qdrant/landing_page/master/qdrant-landing/content/documentation/concepts/collections.md',
            'https://raw.githubusercontent.com/qdrant/landing_page/master/qdrant-landing/content/documentation/concepts/explore.md',
            'https://raw.githubusercontent.com/qdrant/landing_page/master/qdrant-landing/content/documentation/concepts/filtering.md',
            'https://raw.githubusercontent.com/qdrant/landing_page/master/qdrant-landing/content/documentation/concepts/hybrid-queries.md',
            'https://raw.githubusercontent.com/qdrant/landing_page/master/qdrant-landing/content/documentation/concepts/indexing.md',
            'https://raw.githubusercontent.com/qdrant/landing_page/master/qdrant-landing/content/documentation/concepts/optimizer.md',
            'https://raw.githubusercontent.com/qdrant/landing_page/master/qdrant-landing/content/documentation/concepts/payload.md',
            'https://raw.githubusercontent.com/qdrant/landing_page/master/qdrant-landing/content/documentation/concepts/search.md',
            'https://raw.githubusercontent.com/qdrant/landing_page/master/qdrant-landing/content/documentation/concepts/points.md',
            'https://raw.githubusercontent.com/qdrant/landing_page/master/qdrant-landing/content/documentation/concepts/snapshots.md',
            'https://raw.githubusercontent.com/qdrant/landing_page/master/qdrant-landing/content/documentation/concepts/storage.md',
            'https://raw.githubusercontent.com/qdrant/landing_page/master/qdrant-landing/content/documentation/concepts/vectors.md'
        ]
        # Использование переменных из глобального контекста
        self.stop_words = stop_words
        self.stemmer = PorterStemmer()

    def preprocess_text(self, text):
        """Предобработка текста: токенизация, удаление стоп-слов и стемминг"""
        # Токенизация и приведение к нижнему регистру
        tokens = word_tokenize(text.lower())
        # Удаление пунктуации и цифр
        tokens = [token for token in tokens if token not in string.punctuation and not token.isdigit()]
        # Удаление стоп-слов
        tokens = [token for token in tokens if token not in self.stop_words]
        # Стемминг
        try:
            tokens = [self.stemmer.stem(token) for token in tokens]
        except Exception as e:
            print(f"Ошибка стемминга: {e}. Пропускаем этап стемминга.")
        return tokens

    def extract_text_from_md(self, url, max_characters=1500, new_after_n_chars=1000, overlap=0):
        """Извлечение текста из Markdown файла и разбиение на параграфы"""
        try:
            response = requests.get(url)
            response.raise_for_status()
            html_content = markdown.markdown(response.text)
            soup = BeautifulSoup(html_content, features="html.parser")
            text = soup.get_text()
        except Exception as e:
            print(f"Ошибка при получении документа {url}: {e}")
            return []

        # Разделение на смысловые элементы
        raw_paragraphs = [p.strip() for p in re.split(r'\n\s*\n', text) if p.strip()]
        
        paragraphs = []
        current_chunk = ""
        
        for p in raw_paragraphs:
            # Нормализация пробелов
            cleaned_p = re.sub(r'\s+', ' ', p).strip()
            
            # Пропуск слишком коротких фрагментов
            if len(cleaned_p.split()) < 5:
                continue
                
            # Определение, является ли текущий параграф заголовком
            is_title = len(cleaned_p.split()) < 10 and not cleaned_p.endswith(('.', '?', '!'))
            
            # Если новый параграф - заголовок или текущий чанк станет слишком большим
            if is_title or len(current_chunk) + len(cleaned_p) > new_after_n_chars:
                # Сохранение предыдущего чанка, если он не пустой
                if current_chunk:
                    paragraphs.append(current_chunk)
                    current_chunk = ""
            
            # Если параграф слишком большой, разбиваем его на части
            if len(cleaned_p) > max_characters:
                # Разбиение на предложения
                sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?|\!)\s', cleaned_p)
                
                sentence_chunk = ""
                for sentence in sentences:
                    if len(sentence_chunk) + len(sentence) > max_characters:
                        paragraphs.append(sentence_chunk)
                        # Добавление перекрытия, если задано
                        if overlap > 0:
                            words = sentence_chunk.split()
                            overlap_text = ' '.join(words[-min(len(words), overlap//5):])
                            sentence_chunk = overlap_text + " " + sentence
                        else:
                            sentence_chunk = sentence
                    else:
                        sentence_chunk = (sentence_chunk + " " + sentence).strip() if sentence_chunk else sentence
                
                if sentence_chunk:
                    paragraphs.append(sentence_chunk)
            else:
                # Добавление параграфа к текущему чанку
                current_chunk = (current_chunk + "\n\n" + cleaned_p).strip() if current_chunk else cleaned_p
                
                # Если чанк превысил максимальный размер, сохраняем его
                if len(current_chunk) > max_characters:
                    paragraphs.append(current_chunk)
                    current_chunk = ""
        
        # Добавление последнего чанка, если он не пустой
        if current_chunk:
            paragraphs.append(current_chunk)
        
        return paragraphs

    def initialize_database(self):
        """Инициализация базы данных: загрузка и предобработка документов"""
        # Обработка всех документов
        self.doc_paragraphs = []
        for url in self.md_list:
            paragraphs = self.extract_text_from_md(url)
            name = url.split('concepts/')[1].split('.md')[0]

            if name == 'collections':
                paragraphs = [p for p in paragraphs if '/ Collections' not in p]
            else:
                paragraphs = [p for p in paragraphs if f'/{name}' not in p]

            for paragraph in paragraphs:
                self.doc_paragraphs.append({
                    'name': name,
                    'text': paragraph
                })
        
        print(f"Всего загружено {len(self.doc_paragraphs)} фрагментов.")
        
        if not self.doc_paragraphs:
            raise ValueError("Не удалось загрузить ни одного документа!")
        
        # Токенизация и предобработка текстовых фрагментов для BM25
        print("Начинаем токенизацию и предобработку текста...")
        self.tokenized_corpus = [self.preprocess_text(doc['text']) for doc in self.doc_paragraphs]
        
        # Проверка на пустые токенизированные документы
        non_empty_docs = [(i, doc) for i, doc in enumerate(self.tokenized_corpus) if doc]
        if len(non_empty_docs) < len(self.tokenized_corpus):
            print(f"Внимание: {len(self.tokenized_corpus) - len(non_empty_docs)} документов не содержат токенов после предобработки.")
            
            # Отфильтровываем пустые документы
            valid_indices = [i for i, _ in non_empty_docs]
            self.tokenized_corpus = [self.tokenized_corpus[i] for i in valid_indices]
            self.doc_paragraphs = [self.doc_paragraphs[i] for i in valid_indices]
        
        # Инициализация BM25
        print("Инициализация BM25...")
        try:
            self.bm25 = BM25Okapi(self.tokenized_corpus)
            print(f"BM25 успешно инициализирован. Всего документов: {len(self.tokenized_corpus)}")
        except Exception as e:
            print(f"Ошибка при инициализации BM25: {e}")
            raise

    def search_similar_paragraphs(self, user_query, top_k=3):
        """Поиск похожих параграфов с использованием BM25"""
        if not self.bm25:
            print("Ошибка: BM25 не инициализирован!")
            return []
            
        # Предобработка запроса
        tokenized_query = self.preprocess_text(user_query)
        
        if not tokenized_query:
            print("Предупреждение: запрос не содержит значимых токенов после предобработки!")
            return []
        
        # Получение BM25 scores для всех документов
        try:
            scores = self.bm25.get_scores(tokenized_query)
        except Exception as e:
            print(f"Ошибка при получении оценок BM25: {e}")
            return []
        
        # Индексы документов с наивысшими оценками
        top_n = np.argsort(scores)[::-1][:top_k]
        
        # Возвращение текста, имени документа и оценки BM25
        results = []
        for i in top_n:
            if scores[i] > 0:  # Добавление только если оценка больше 0
                results.append((self.doc_paragraphs[i]['text'], self.doc_paragraphs[i]['name'], float(scores[i])))
        
        return results

# Функция для оценки релевантности с использованием API
def evaluate_relevance_with_claude(question, fragment_text, api_key):
    """Оценивает релевантность фрагмента к вопросу через API Claude"""
    url = "https://ask.chadgpt.ru/api/public/gpt-4o-mini"
    
    # Ограничиваем длину фрагмента для запроса
    max_fragment_length = 4000
    if len(fragment_text) > max_fragment_length:
        fragment_text = fragment_text[:max_fragment_length] + "..."
    
    prompt = f"""
    Задача: оценить релевантность текстового фрагмента вопросу.
    
    Вопрос: {question}
    
    Фрагмент: {fragment_text}
    
    Оцени релевантность фрагмента к вопросу по шкале от 1 до 5, где:
    1 - совершенно не релевантен
    2 - слабо релевантен
    3 - умеренно релевантен
    4 - очень релевантен
    5 - идеально релевантен
    
    Ответь только числом от 1 до 5 без пояснений.
    """
    
    # Формируем запрос согласно примеру
    request_json = {
        "message": prompt,
        "api_key": api_key
    }
    
    try:
        # Отправляем запрос и дожидаемся ответа
        response = requests.post(url=url, json=request_json)
        
        # Проверяем, отправился ли запрос
        if response.status_code != 200:
            print(f'Ошибка! Код http-ответа: {response.status_code}')
            return 1  # Возвращаем минимальную оценку в случае ошибки
        else:
            # Получаем текст ответа и преобразовываем в dict
            resp_json = response.json()
            
            # Если успешен ответ,            # Если успешен ответ, то извлекаем результат
            if resp_json['is_success']:
                resp_msg = resp_json['response'].strip()
                # Ищем число от 1 до 5 в ответе
                import re
                score_match = re.search(r'[1-5]', resp_msg)
                if score_match:
                    relevance_score = int(score_match.group(0))
                    return relevance_score
                else:
                    print(f'Не удалось извлечь оценку из ответа: {resp_msg}')
                    return 3  # Средняя оценка по умолчанию в случае неоднозначного ответа
            else:
                error = resp_json['error_message']
                print(f'Ошибка: {error}')
                return 1  # Возвращаем минимальную оценку в случае ошибки
    except Exception as e:
        print(f'Исключение при обработке запроса: {str(e)}')
        return 1  # Возвращаем минимальную оценку в случае ошибки

def get_relevant_fragments_bm25(qa_system, question, top_k=6):
    """Получение релевантных фрагментов с использованием BM25"""
    fragments = qa_system.search_similar_paragraphs(question, top_k=top_k)
    return fragments

def calculate_metrics(retrieval_results):
    """Вычисление метрик эффективности ретривала"""
    metrics = {
        'recall@1': [],
        'recall@4': [],
        'recall@6': [],
        'precision@1': [],
        'precision@4': [],
        'precision@6': [],
        'mrr@4': [],
        'mrr@6': [],
        'ndcg@4': [],
        'ndcg@6': []
    }
    
    for result in retrieval_results:
        fragments = result['fragments']
        if not fragments:
            print(f"Предупреждение: для вопроса '{result['question']}' не найдено фрагментов")
            # Пропускаем вычисление метрик для этого запроса
            continue
            
        # Сортировка фрагментов по оценке релевантности от Claude (по убыванию)
        sorted_fragments = sorted(fragments, key=lambda x: x[3], reverse=True)
        
        # Сортировка фрагментов по скору из системы ретривала (по убыванию)
        retrieved_fragments = sorted(fragments, key=lambda x: x[2], reverse=True)
        
        # Вычисление Recall@k
        relevant_fragments = [f for f in sorted_fragments if f[3] >= 4]  # Считаем релевантными фрагменты с оценкой >= 4
        total_relevant = len(relevant_fragments)
        
        if total_relevant > 0:
            # Recall@1
            relevant_at_1 = sum(1 for f in retrieved_fragments[:1] if f[3] >= 4)
            metrics['recall@1'].append(relevant_at_1 / total_relevant)
            
            # Recall@4
            relevant_at_4 = sum(1 for f in retrieved_fragments[:4] if f[3] >= 4)
            metrics['recall@4'].append(relevant_at_4 / total_relevant)
            
            # Recall@6
            relevant_at_6 = sum(1 for f in retrieved_fragments[:min(6, len(retrieved_fragments))] if f[3] >= 4)
            metrics['recall@6'].append(relevant_at_6 / total_relevant)
            
            # Precision@1
            metrics['precision@1'].append(relevant_at_1 / 1 if len(retrieved_fragments) >= 1 else 0)
            
            # Precision@4
            metrics['precision@4'].append(relevant_at_4 / min(4, len(retrieved_fragments)))
            
            # Precision@6
            metrics['precision@6'].append(relevant_at_6 / min(6, len(retrieved_fragments)))
        else:
            # Если нет релевантных фрагментов, устанавливаем recall = 1.0 (все релевантные найдены)
            metrics['recall@1'].append(1.0)
            metrics['recall@4'].append(1.0)
            metrics['recall@6'].append(1.0)
            
            # Если нет релевантных фрагментов, устанавливаем precision = 0.0
            metrics['precision@1'].append(0.0)
            metrics['precision@4'].append(0.0)
            metrics['precision@6'].append(0.0)
        
        # MRR@4 (Mean Reciprocal Rank для первых 4)
        first_relevant_rank_at_4 = next((i + 1 for i, f in enumerate(retrieved_fragments[:min(4, len(retrieved_fragments))]) if f[3] >= 4), 0)
        if first_relevant_rank_at_4 > 0:
            metrics['mrr@4'].append(1.0 / first_relevant_rank_at_4)
        else:
            metrics['mrr@4'].append(0.0)
            
        # MRR@6 (Mean Reciprocal Rank для первых 6)
        first_relevant_rank_at_6 = next((i + 1 for i, f in enumerate(retrieved_fragments[:min(6, len(retrieved_fragments))]) if f[3] >= 4), 0)
        if first_relevant_rank_at_6 > 0:
            metrics['mrr@6'].append(1.0 / first_relevant_rank_at_6)
        else:
            metrics['mrr@6'].append(0.0)
        
        # nDCG@4
        if len(sorted_fragments) >= 1 and len(retrieved_fragments) >= 1:
            # Определяем количество документов для оценки
            k4 = min(4, len(sorted_fragments), len(retrieved_fragments))
            
            # Берем только первые k4 документа
            true_relevance_4 = np.array([f[3] for f in sorted_fragments[:k4]])
            predicted_order_relevance_4 = np.array([f[3] for f in retrieved_fragments[:k4]])
            
            try:
                ndcg_4 = ndcg_score([true_relevance_4], [predicted_order_relevance_4])
                metrics['ndcg@4'].append(ndcg_4)
            except Exception as e:
                print(f"Ошибка при вычислении nDCG@4: {e}")
                metrics['ndcg@4'].append(0.0)
        else:
            metrics['ndcg@4'].append(0.0)
            
        # nDCG@6
        if len(sorted_fragments) >= 1 and len(retrieved_fragments) >= 1:
            # Определяем количество документов для оценки
            k6 = min(6, len(sorted_fragments), len(retrieved_fragments))
            
            # Берем только первые k6 документов
            true_relevance_6 = np.array([f[3] for f in sorted_fragments[:k6]])
            predicted_order_relevance_6 = np.array([f[3] for f in retrieved_fragments[:k6]])
            
            try:
                ndcg_6 = ndcg_score([true_relevance_6], [predicted_order_relevance_6])
                metrics['ndcg@6'].append(ndcg_6)
            except Exception as e:
                print(f"Ошибка при вычислении nDCG@6: {e}")
                metrics['ndcg@6'].append(0.0)
        else:
            metrics['ndcg@6'].append(0.0)
    
    # Вычисляем средние значения метрик
    result_metrics = {}
    for key, values in metrics.items():
        result_metrics[key] = sum(values) / len(values) if values else 0.0
    
    return result_metrics

# Основной код для тестирования системы и вычисления метрик

def run_evaluation(api_key, test_questions):
    """Запуск оценки BM25 системы на наборе тестовых вопросов"""
    # Инициализация системы BM25
    qa_system = DocumentationQA_BM25()
    qa_system.initialize_database()
    
    # Результаты для последующей оценки
    retrieval_results = []
    
    # Обработка каждого вопроса
    for question in test_questions:
        
        # Получение фрагментов с помощью BM25
        fragments = get_relevant_fragments_bm25(qa_system, question, top_k=6)
        
        # Результаты для текущего вопроса
        result = {'question': question, 'fragments': []}
        
        # Оценка релевантности для каждого фрагмента
        for text, name, score in fragments:
            # Делаем задержку между запросами, чтобы не превысить лимиты API
            time.sleep(2)
            relevance_score = evaluate_relevance_with_claude(question, text, api_key)
            result['fragments'].append((text, name, score, relevance_score))
        
        retrieval_results.append(result)
    
    # Вычисление метрик
    metrics_results = calculate_metrics(retrieval_results)
    
    return metrics_results, retrieval_results

def main():
    try:
        # Загрузка API ключа
        api_key_file = 'api.txt'
        if os.path.exists(api_key_file):
            with open(api_key_file, 'r') as file:
                api_key = file.read().strip()
        else:
            print(f"Файл с API ключом {api_key_file} не найден!")
            api_key = input("Введите ваш API ключ: ")
        
        # Проверка API ключа
        if not api_key:
            raise ValueError("API ключ не может быть пустым")
        
        df = pd.read_csv('texts_with_answers.csv')
        test_questions = df.question.to_list()
        
        # Запуск оценки
        print("Начало оценки BM25 ретривала...")
        metrics, results = run_evaluation(api_key, test_questions)
        
        # Вывод результатов
        print("\nРезультаты оценки BM25-ретривала:")
        for metric, value in metrics.items():
            print(f"{metric}: {value:.4f}")
        
        # Детальный анализ результатов
        print("\nДетальный анализ результатов:")
        for result in results:
            question = result['question']
            print(f"\nВопрос: {question}")
            
            if not result['fragments']:
                print("Не найдено релевантных фрагментов для этого вопроса.")
                continue
                
            # Сортировка по оценке релевантности (от Claude)
            sorted_by_relevance = sorted(result['fragments'], key=lambda x: x[3], reverse=True)
            print("Топ-3 наиболее релевантных фрагмента по оценке Claude:")
            for i, (text, name, bm25_score, relevance) in enumerate(sorted_by_relevance[:min(3, len(sorted_by_relevance))]):
                print(f"{i+1}. {name} (BM25: {bm25_score:.4f}, Релевантность: {relevance})")
                print(f"   {text[:100]}...")
            
            # Сортировка по BM25
            sorted_by_bm25 = sorted(result['fragments'], key=lambda x: x[2], reverse=True)
            print("\nТоп-3 наиболее релевантных фрагмента по BM25:")
            for i, (text, name, bm25_score, relevance) in enumerate(sorted_by_bm25[:min(3, len(sorted_by_bm25))]):
                print(f"{i+1}. {name} (BM25: {bm25_score:.4f}, Релевантность: {relevance})")
                print(f"   {text[:100]}...")
        
        # Сохранение результатов в CSV для дальнейшего анализа
        save_results_to_csv(results, "bm25_retrieval_results.csv")
        
        return metrics, results
    except Exception as e:
        print(f"Ошибка в функции main: {e}")
        import traceback
        traceback.print_exc()
        return None, None

def save_results_to_csv(results, filename):
    """Сохранение результатов в CSV файл"""
    rows = []
    for result in results:
        question = result['question']
        for text, name, bm25_score, relevance in result['fragments']:
            rows.append({
                'question': question,
                'document': name,
                'bm25_score': bm25_score,
                'relevance_score': relevance,
                'text': text[:200]  # Ограничиваем длину текста для CSV
            })
    
    df = pd.DataFrame(rows)
    df.to_csv(filename, index=False)
    print(f"Результаты сохранены в {filename}")


[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\sekho\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\sekho\AppData\Roaming\nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\sekho\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     C:\Users\sekho\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_eng is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\Users\sekho\AppData\Roaming\nltk_data...
[

In [32]:
df = pd.read_csv('texts_with_answers.csv')
questions = df['question'].tolist()

api_key_file = 'api.txt'
if os.path.exists(api_key_file):
    with open(api_key_file, 'r') as file:
        api_key = file.read().strip()

In [33]:
metrics, res = run_evaluation(api_key = api_key, test_questions = questions)

Всего загружено 292 фрагментов.
Начинаем токенизацию и предобработку текста...
Внимание: 16 документов не содержат токенов после предобработки.
Инициализация BM25...
BM25 успешно инициализирован. Всего документов: 276


In [34]:
metrics

{'recall@1': 0.37041666666666656,
 'recall@4': 0.8120833333333334,
 'recall@6': 1.0,
 'precision@1': 0.6666666666666666,
 'precision@4': 0.4791666666666667,
 'precision@6': 0.4208333333333332,
 'mrr@4': 0.7659722222222222,
 'mrr@6': 0.7726388888888889,
 'ndcg@4': np.float64(0.9662130678581775),
 'ndcg@6': np.float64(0.9384500595773312)}

Ниже представлена реализация с базовой моделью SentenceTransformer('BAAI/bge-large-en-v1.5') и различными реранкерами

In [36]:
import requests
import markdown
from bs4 import BeautifulSoup
import re
import numpy as np
import pandas as pd
import time
from sklearn.metrics import ndcg_score
import nltk
import string
import os
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

nltk.download('all')

# Явная установка директории для загрузки NLTK данных
nltk_data_dir = os.path.join(os.getcwd(), 'nltk_data')
os.makedirs(nltk_data_dir, exist_ok=True)
nltk.data.path.append(nltk_data_dir)

# Загрузка необходимых ресурсов NLTK с явным указанием пути
try:
    nltk.download('punkt', download_dir=nltk_data_dir, quiet=True)
    nltk.download('stopwords', download_dir=nltk_data_dir, quiet=True)
    from nltk.tokenize import word_tokenize
    from nltk.corpus import stopwords
    from nltk.stem import PorterStemmer
    stop_words = set(stopwords.words('english'))
except LookupError:
    print("Не удалось загрузить ресурсы NLTK. Используем упрощенную токенизацию.")
    
    # Упрощенная имплементация токенизации без зависимостей NLTK
    def word_tokenize(text):
        """Простая токенизация по пробелам и пунктуации"""
        # Заменяем пунктуацию на пробелы
        for punct in string.punctuation:
            text = text.replace(punct, ' ')
        # Разбиваем по пробелам и фильтруем пустые токены
        return [token for token in text.lower().split() if token]
    
    # Пустой набор стоп-слов
    stop_words = set()
    
    # Упрощенный стеммер
    class SimplePorterStemmer:
        """Очень упрощенная версия стеммера - убирает только окончения -ing, -ed, -s"""
        def stem(self, word):
            if word.endswith('ing'):
                return word[:-3]
            elif word.endswith('ed') and len(word) > 3:
                return word[:-2]
            elif word.endswith('s') and len(word) > 2:
                return word[:-1]
            return word
    
    PorterStemmer = SimplePorterStemmer

class DocumentationQA_E5Embeddings:
    def __init__(self):
        self.model = None
        self.doc_paragraphs = []
        self.doc_embeddings = None
        self.md_list = [
            'https://raw.githubusercontent.com/qdrant/landing_page/master/qdrant-landing/content/documentation/concepts/collections.md',
            'https://raw.githubusercontent.com/qdrant/landing_page/master/qdrant-landing/content/documentation/concepts/explore.md',
            'https://raw.githubusercontent.com/qdrant/landing_page/master/qdrant-landing/content/documentation/concepts/filtering.md',
            'https://raw.githubusercontent.com/qdrant/landing_page/master/qdrant-landing/content/documentation/concepts/hybrid-queries.md',
            'https://raw.githubusercontent.com/qdrant/landing_page/master/qdrant-landing/content/documentation/concepts/indexing.md',
            'https://raw.githubusercontent.com/qdrant/landing_page/master/qdrant-landing/content/documentation/concepts/optimizer.md',
            'https://raw.githubusercontent.com/qdrant/landing_page/master/qdrant-landing/content/documentation/concepts/payload.md',
            'https://raw.githubusercontent.com/qdrant/landing_page/master/qdrant-landing/content/documentation/concepts/search.md',
            'https://raw.githubusercontent.com/qdrant/landing_page/master/qdrant-landing/content/documentation/concepts/points.md',
            'https://raw.githubusercontent.com/qdrant/landing_page/master/qdrant-landing/content/documentation/concepts/snapshots.md',
            'https://raw.githubusercontent.com/qdrant/landing_page/master/qdrant-landing/content/documentation/concepts/storage.md',
            'https://raw.githubusercontent.com/qdrant/landing_page/master/qdrant-landing/content/documentation/concepts/vectors.md'
        ]

    def extract_text_from_md(self, url, max_characters=1500, new_after_n_chars=1000, overlap=0):
        """Извлечение текста из Markdown файла и разбиение на параграфы"""
        try:
            response = requests.get(url)
            response.raise_for_status()
            html_content = markdown.markdown(response.text)
            soup = BeautifulSoup(html_content, features="html.parser")
            text = soup.get_text()
        except Exception as e:
            print(f"Ошибка при получении документа {url}: {e}")
            return []

        # Разделение на смысловые элементы
        raw_paragraphs = [p.strip() for p in re.split(r'\n\s*\n', text) if p.strip()]
        
        paragraphs = []
        current_chunk = ""
        
        for p in raw_paragraphs:
            # Нормализация пробелов
            cleaned_p = re.sub(r'\s+', ' ', p).strip()
            
            # Пропуск слишком коротких фрагментов
            if len(cleaned_p.split()) < 5:
                continue
                
            # Определение, является ли текущий параграф заголовком
            is_title = len(cleaned_p.split()) < 10 and not cleaned_p.endswith(('.', '?', '!'))
            
            # Если новый параграф - заголовок или текущий чанк станет слишком большим
            if is_title or len(current_chunk) + len(cleaned_p) > new_after_n_chars:
                # Сохранение предыдущего чанка, если он не пустой
                if current_chunk:
                    paragraphs.append(current_chunk)
                    current_chunk = ""
            
            # Если параграф слишком большой, разбиваем его на части
            if len(cleaned_p) > max_characters:
                # Разбиение на предложения
                sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?|\!)\s', cleaned_p)
                
                sentence_chunk = ""
                for sentence in sentences:
                    if len(sentence_chunk) + len(sentence) > max_characters:
                        paragraphs.append(sentence_chunk)
                        # Добавление перекрытия, если задано
                        if overlap > 0:
                            words = sentence_chunk.split()
                            overlap_text = ' '.join(words[-min(len(words), overlap//5):])
                            sentence_chunk = overlap_text + " " + sentence
                        else:
                            sentence_chunk = sentence
                    else:
                        sentence_chunk = (sentence_chunk + " " + sentence).strip() if sentence_chunk else sentence
                
                if sentence_chunk:
                    paragraphs.append(sentence_chunk)
            else:
                # Добавление параграфа к текущему чанку
                current_chunk = (current_chunk + "\n\n" + cleaned_p).strip() if current_chunk else cleaned_p
                
                # Если чанк превысил максимальный размер, сохраняем его
                if len(current_chunk) > max_characters:
                    paragraphs.append(current_chunk)
                    current_chunk = ""
        
        # Добавление последнего чанка, если он не пустой
        if current_chunk:
            paragraphs.append(current_chunk)
        
        return paragraphs

    def initialize_database(self):
        """Инициализация базы данных: загрузка и предобработка документов"""
        print("Загрузка модели...")
        try:
            self.model = SentenceTransformer('BAAI/bge-large-en-v1.5')
            print("Модель успешно загружена")
        except Exception as e:
            print(f"Ошибка при загрузке модели: {e}")
            raise
        
        # Обработка всех документов
        self.doc_paragraphs = []
        for url in self.md_list:
            paragraphs = self.extract_text_from_md(url)
            name = url.split('concepts/')[1].split('.md')[0]

            if name == 'collections':
                paragraphs = [p for p in paragraphs if '/ Collections' not in p]
            else:
                paragraphs = [p for p in paragraphs if f'/{name}' not in p]

            for paragraph in paragraphs:
                self.doc_paragraphs.append({
                    'name': name,
                    'text': paragraph
                })
        
        print(f"Всего загружено {len(self.doc_paragraphs)} фрагментов.")
        
        if not self.doc_paragraphs:
            raise ValueError("Не удалось загрузить ни одного документа!")
        
        # Создание эмбеддингов для всех документов
        print("Генерация эмбеддингов для документов...")
        try:
            # Формируем тексты документов с инструкцией для E5
            doc_texts = [f"passage: {doc['text']}" for doc in self.doc_paragraphs]
            
            # Преобразуем тексты в эмбеддинги
            self.doc_embeddings = self.model.encode(doc_texts, convert_to_numpy=True, show_progress_bar=True)
            print(f"Эмбеддинги успешно созданы. Размерность: {self.doc_embeddings.shape}")
        except Exception as e:
            print(f"Ошибка при создании эмбеддингов: {e}")
            raise

    def search_similar_paragraphs(self, user_query, top_k=3):
        """Поиск похожих параграфов с использованием эмбеддингов E5"""
        if self.doc_embeddings is None or not self.model:
            print("Ошибка: База данных не инициализирована!")
            return []
            
        try:
            # Формируем запрос с инструкцией для E5
            query_text = f"query: {user_query}"
            
            # Получаем эмбеддинг запроса
            query_embedding = self.model.encode([query_text], convert_to_numpy=True)[0]
            
            # Вычисляем косинусное сходство между запросом и всеми документами
            similarities = cosine_similarity([query_embedding], self.doc_embeddings)[0]
            
            # Находим индексы с наибольшим сходством
            top_indices = np.argsort(similarities)[::-1][:top_k]
            
            # Возвращаем результаты с указанием текста, имени и оценки сходства
            results = []
            for idx in top_indices:
                if similarities[idx] > 0:  # Добавляем только позитивные сходства
                    results.append((self.doc_paragraphs[idx]['text'], 
                                   self.doc_paragraphs[idx]['name'], 
                                   float(similarities[idx])))
            
            return results
        except Exception as e:
            print(f"Ошибка при поиске похожих параграфов: {e}")
            return []

# Функция для оценки релевантности с использованием API
def evaluate_relevance_with_claude(question, fragment_text, api_key):
    """Оценивает релевантность фрагмента к вопросу через API Claude"""
    url = "https://ask.chadgpt.ru/api/public/gpt-4o-mini"
    
    # Ограничиваем длину фрагмента для запроса
    max_fragment_length = 4000
    if len(fragment_text) > max_fragment_length:
        fragment_text = fragment_text[:max_fragment_length] + "..."
    
    prompt = f"""
    Задача: оценить релевантность текстового фрагмента вопросу.
    
    Вопрос: {question}
    
    Фрагмент: {fragment_text}
    
    Оцени релевантность фрагмента к вопросу по шкале от 1 до 5, где:
    1 - совершенно не релевантен
    2 - слабо релевантен
    3 - умеренно релевантен
    4 - очень релевантен
    5 - идеально релевантен
    
    Ответь только числом от 1 до 5 без пояснений.
    """
    
    # Формируем запрос согласно примеру
    request_json = {
        "message": prompt,
        "api_key": api_key
    }
    
    try:
        # Отправляем запрос и дожидаемся ответа
        response = requests.post(url=url, json=request_json)
        
        # Проверяем, отправился ли запрос
        if response.status_code != 200:
            print(f'Ошибка! Код http-ответа: {response.status_code}')
            return 1  # Возвращаем минимальную оценку в случае ошибки
        else:
            # Получаем текст ответа и преобразовываем в dict
            resp_json = response.json()
            
            # Если успешен ответ, то извлекаем результат
            if resp_json['is_success']:
                resp_msg = resp_json['response'].strip()
                # Ищем число от 1 до 5 в ответе
                import re
                score_match = re.search(r'[1-5]', resp_msg)
                if score_match:
                    relevance_score = int(score_match.group(0))
                    return relevance_score
                else:
                    print(f'Не удалось извлечь оценку из ответа: {resp_msg}')
                    return 3  # Средняя оценка по умолчанию в случае неоднозначного ответа
            else:
                error = resp_json['error_message']
                print(f'Ошибка: {error}')
                return 1  # Возвращаем минимальную оценку в случае ошибки
    except Exception as e:
        print(f'Исключение при обработке запроса: {str(e)}')
        return 1  # Возвращаем минимальную оценку в случае ошибки

def get_relevant_fragments_e5(qa_system, question, top_k=6):
    """Получение релевантных фрагментов с использованием E5 эмбеддингов"""
    fragments = qa_system.search_similar_paragraphs(question, top_k=top_k)
    return fragments

def calculate_metrics(retrieval_results):
    """Вычисление метрик эффективности ретривала"""
    metrics = {
        'recall@1': [],
        'recall@4': [],
        'recall@6': [],
        'precision@1': [],
        'precision@4': [],
        'precision@6': [],
        'mrr@4': [],
        'mrr@6': [],
        'ndcg@4': [],
        'ndcg@6': []
    }
    
    for result in retrieval_results:
        fragments = result['fragments']
        if not fragments:
            print(f"Предупреждение: для вопроса '{result['question']}' не найдено фрагментов")
            # Пропускаем вычисление метрик для этого запроса
            continue
            
        # Сортировка фрагментов по оценке релевантности от Claude (по убыванию)
        sorted_fragments = sorted(fragments, key=lambda x: x[3], reverse=True)
        
        # Сортировка фрагментов по скору из системы ретривала (по убыванию)
        retrieved_fragments = sorted(fragments, key=lambda x: x[2], reverse=True)
        
        # Вычисление Recall@k
        relevant_fragments = [f for f in sorted_fragments if f[3] >= 4]  # Считаем релевантными фрагменты с оценкой >= 4
        total_relevant = len(relevant_fragments)
        
        if total_relevant > 0:
            # Recall@1
            relevant_at_1 = sum(1 for f in retrieved_fragments[:1] if f[3] >= 4)
            metrics['recall@1'].append(relevant_at_1 / total_relevant)
            
            # Recall@4
            relevant_at_4 = sum(1 for f in retrieved_fragments[:4] if f[3] >= 4)
            metrics['recall@4'].append(relevant_at_4 / total_relevant)
            
            # Recall@6
            relevant_at_6 = sum(1 for f in retrieved_fragments[:min(6, len(retrieved_fragments))] if f[3] >= 4)
            metrics['recall@6'].append(relevant_at_6 / total_relevant)
            
            # Precision@1
            metrics['precision@1'].append(relevant_at_1 / 1 if len(retrieved_fragments) >= 1 else 0)
            
            # Precision@4
            metrics['precision@4'].append(relevant_at_4 / min(4, len(retrieved_fragments)))
            
            # Precision@6
            metrics['precision@6'].append(relevant_at_6 / min(6, len(retrieved_fragments)))
        else:
            # Если нет релевантных фрагментов, устанавливаем recall = 1.0 (все релевантные найдены)
            metrics['recall@1'].append(1.0)
            metrics['recall@4'].append(1.0)
            metrics['recall@6'].append(1.0)
            
            # Если нет релевантных фрагментов, устанавливаем precision = 0.0
            metrics['precision@1'].append(0.0)
            metrics['precision@4'].append(0.0)
            metrics['precision@6'].append(0.0)
        
        # MRR@4 (Mean Reciprocal Rank для первых 4)
        first_relevant_rank_at_4 = next((i + 1 for i, f in enumerate(retrieved_fragments[:min(4, len(retrieved_fragments))]) if f[3] >= 4), 0)
        if first_relevant_rank_at_4 > 0:
            metrics['mrr@4'].append(1.0 / first_relevant_rank_at_4)
        else:
            metrics['mrr@4'].append(0.0)
            
        # MRR@6 (Mean Reciprocal Rank для первых 6)
        first_relevant_rank_at_6 = next((i + 1 for i, f in enumerate(retrieved_fragments[:min(6, len(retrieved_fragments))]) if f[3] >= 4), 0)
        if first_relevant_rank_at_6 > 0:
            metrics['mrr@6'].append(1.0 / first_relevant_rank_at_6)
        else:
            metrics['mrr@6'].append(0.0)
        
        # nDCG@4
        if len(sorted_fragments) >= 1 and len(retrieved_fragments) >= 1:
            # Определяем количество документов для оценки
            k4 = min(4, len(sorted_fragments), len(retrieved_fragments))
            
            # Берем только первые k4 документа
            true_relevance_4 = np.array([f[3] for f in sorted_fragments[:k4]])
            predicted_order_relevance_4 = np.array([f[3] for f in retrieved_fragments[:k4]])
            
            try:
                ndcg_4 = ndcg_score([true_relevance_4], [predicted_order_relevance_4])
                metrics['ndcg@4'].append(ndcg_4)
            except Exception as e:
                print(f"Ошибка при вычислении nDCG@4: {e}")
                metrics['ndcg@4'].append(0.0)
        else:
            metrics['ndcg@4'].append(0.0)
            
        # nDCG@6
        if len(sorted_fragments) >= 1 and len(retrieved_fragments) >= 1:
            # Определяем количество документов для оценки
            k6 = min(6, len(sorted_fragments), len(retrieved_fragments))
            
            # Берем только первые k6 документов
            true_relevance_6 = np.array([f[3] for f in sorted_fragments[:k6]])
            predicted_order_relevance_6 = np.array([f[3] for f in retrieved_fragments[:k6]])
            
            try:
                ndcg_6 = ndcg_score([true_relevance_6], [predicted_order_relevance_6])
                metrics['ndcg@6'].append(ndcg_6)
            except Exception as e:
                print(f"Ошибка при вычислении nDCG@6: {e}")
                metrics['ndcg@6'].append(0.0)
        else:
            metrics['ndcg@6'].append(0.0)
    
    # Вычисляем средние значения метрик
    result_metrics = {}
    for key, values in metrics.items():
        result_metrics[key] = sum(values) / len(values) if values else 0.0
    
    return result_metrics

# Основной код для тестирования системы и вычисления метрик

def run_evaluation(api_key, test_questions):
    """Запуск оценки системы ретривала на основе E5 на наборе тестовых вопросов"""
    # Инициализация системы на основе E5
    qa_system = DocumentationQA_E5Embeddings()
    qa_system.initialize_database()
    
    # Результаты для последующей оценки
    retrieval_results = []
    
    # Обработка каждого вопроса
    for question in test_questions:
        
        # Получение фрагментов с помощью E5
        fragments = get_relevant_fragments_e5(qa_system, question, top_k=6)
        
        # Результаты для текущего вопроса
        result = {'question': question, 'fragments': []}
        
        # Оценка релевантности для каждого фрагмента
        for text, name, score in fragments:
            # Делаем задержку между запросами, чтобы не превысить лимиты API
            time.sleep(2)
            relevance_score = evaluate_relevance_with_claude(question, text, api_key)
            result['fragments'].append((text, name, score, relevance_score))
        
        retrieval_results.append(result)
    
    # Вычисление метрик
    metrics_results = calculate_metrics(retrieval_results)
    
    return metrics_results, retrieval_results

def save_results_to_csv(results, filename):
    """Сохранение результатов в CSV файл"""
    rows = []
    for result in results:
        question = result['question']
        for text, name, e5_score, relevance in result['fragments']:
            rows.append({
                'question': question,
                'document': name,
                'e5_score': e5_score,
                'relevance_score': relevance,
                'text': text[:200]  # Ограничиваем длину текста для CSV
            })
    
    df = pd.DataFrame(rows)
    df.to_csv(filename, index=False)
    print(f"Результаты сохранены в {filename}")

def main():
    try:
        # Загрузка API ключа
        api_key_file = 'api.txt'
        if os.path.exists(api_key_file):
            with open(api_key_file, 'r') as file:
                api_key = file.read().strip()
        else:
            print(f"Файл с API ключом {api_key_file} не найден!")
            api_key = input("Введите ваш API ключ: ")
        
        # Проверка API ключа
        if not api_key:
            raise ValueError("API ключ не может быть пустым")
        
        df = pd.read_csv('texts_with_answers.csv')
        test_questions = df.question.to_list()
        
        # Запуск оценки
        print("Начало оценки E5 ретривала...")
        metrics, results = run_evaluation(api_key, test_questions)
        
        # Вывод результатов
        print("\nРезультаты оценки E5-ретривала:")
        for metric, value in metrics.items():
            print(f"{metric}: {value:.4f}")
        
        # Детальный анализ результатов
        print("\nДетальный анализ результатов:")
        for result in results:
            question = result['question']
            print(f"\nВопрос: {question}")
            
            if not result['fragments']:
                print("Не найдено релевантных фрагментов для этого вопроса.")
                continue
                
            # Сортировка по оценке релевантности (от Claude)
            sorted_by_relevance = sorted(result['fragments'], key=lambda x: x[3], reverse=True)
            print("Топ-3 наиболее релевантных фрагмента по оценке Claude:")
            for i, (text, name, e5_score, relevance) in enumerate(sorted_by_relevance[:min(3, len(sorted_by_relevance))]):
                print(f"{i+1}. {name} (E5: {e5_score:.4f}, Релевантность: {relevance})")
                print(f"   {text[:100]}...")
            
            # Сортировка по E5 сходству
            sorted_by_e5 = sorted(result['fragments'], key=lambda x: x[2], reverse=True)
            print("\nТоп-3 наиболее релевантных фрагмента по E5:")
            for i, (text, name, e5_score, relevance) in enumerate(sorted_by_e5[:min(3, len(sorted_by_e5))]):
                print(f"{i+1}. {name} (E5: {e5_score:.4f}, Релевантность: {relevance})")
                print(f"   {text[:100]}...")
        
        # Сохранение результатов в CSV для дальнейшего анализа
        save_results_to_csv(results, "e5_retrieval_results.csv")
        
        return metrics, results
    except Exception as e:
        print(f"Ошибка в функции main: {e}")
        import traceback
        traceback.print_exc()
        return None, None

df = pd.read_csv('texts_with_answers.csv')
questions = df['question'].tolist()

api_key_file = 'api.txt'
if os.path.exists(api_key_file):
    with open(api_key_file, 'r') as file:
        api_key = file.read().strip()
metrics, res = run_evaluation(api_key=api_key, test_questions=questions)

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\sekho\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\sekho\AppData\Roaming\nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\sekho\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     C:\Users\sekho\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_eng is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\Users\sekho\AppData\Roaming\nltk_data...
[

Загрузка модели E5...


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

Модель E5 успешно загружена
Всего загружено 292 фрагментов.
Генерация эмбеддингов для документов...


Batches:   0%|          | 0/10 [00:00<?, ?it/s]

Эмбеддинги успешно созданы. Размерность: (292, 1024)


In [37]:
metrics

{'recall@1': 0.33069444444444446,
 'recall@4': 0.7888888888888885,
 'recall@6': 1.0,
 'precision@1': 0.7833333333333333,
 'precision@4': 0.5916666666666667,
 'precision@6': 0.5305555555555554,
 'mrr@4': 0.8506944444444445,
 'mrr@6': 0.8554166666666667,
 'ndcg@4': np.float64(0.9713621177570062),
 'ndcg@6': np.float64(0.948999405854192)}

Модель BAAI/bge-large-en-v1.5 с реранкером cross-encoder/ms-marco-MiniLM-L-12-v2

In [11]:
import requests
import markdown
from bs4 import BeautifulSoup
import re
import numpy as np
import pandas as pd
import time
from sklearn.metrics import ndcg_score
import nltk
import string
import os
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

nltk.download('all')

# Явная установка директории для загрузки NLTK данных
nltk_data_dir = os.path.join(os.getcwd(), 'nltk_data')
os.makedirs(nltk_data_dir, exist_ok=True)
nltk.data.path.append(nltk_data_dir)

# Загрузка необходимых ресурсов NLTK с явным указанием пути
try:
    nltk.download('punkt', download_dir=nltk_data_dir, quiet=True)
    nltk.download('stopwords', download_dir=nltk_data_dir, quiet=True)
    from nltk.tokenize import word_tokenize
    from nltk.corpus import stopwords
    from nltk.stem import PorterStemmer
    stop_words = set(stopwords.words('english'))
except LookupError:
    print("Не удалось загрузить ресурсы NLTK. Используем упрощенную токенизацию.")
    
    # Упрощенная имплементация токенизации без зависимостей NLTK
    def word_tokenize(text):
        """Простая токенизация по пробелам и пунктуации"""
        # Заменяем пунктуацию на пробелы
        for punct in string.punctuation:
            text = text.replace(punct, ' ')
        # Разбиваем по пробелам и фильтруем пустые токены
        return [token for token in text.lower().split() if token]
    
    # Пустой набор стоп-слов
    stop_words = set()
    
    # Упрощенный стеммер
    class SimplePorterStemmer:
        """Очень упрощенная версия стеммера - убирает только окончения -ing, -ed, -s"""
        def stem(self, word):
            if word.endswith('ing'):
                return word[:-3]
            elif word.endswith('ed') and len(word) > 3:
                return word[:-2]
            elif word.endswith('s') and len(word) > 2:
                return word[:-1]
            return word
    
    PorterStemmer = SimplePorterStemmer

class DocumentationQA_E5Embeddings:
    def __init__(self):
        self.model = None
        self.reranker = None
        self.doc_paragraphs = []
        self.doc_embeddings = None
        self.md_list = [
            'https://raw.githubusercontent.com/qdrant/landing_page/master/qdrant-landing/content/documentation/concepts/collections.md',
            'https://raw.githubusercontent.com/qdrant/landing_page/master/qdrant-landing/content/documentation/concepts/explore.md',
            'https://raw.githubusercontent.com/qdrant/landing_page/master/qdrant-landing/content/documentation/concepts/filtering.md',
            'https://raw.githubusercontent.com/qdrant/landing_page/master/qdrant-landing/content/documentation/concepts/hybrid-queries.md',
            'https://raw.githubusercontent.com/qdrant/landing_page/master/qdrant-landing/content/documentation/concepts/indexing.md',
            'https://raw.githubusercontent.com/qdrant/landing_page/master/qdrant-landing/content/documentation/concepts/optimizer.md',
            'https://raw.githubusercontent.com/qdrant/landing_page/master/qdrant-landing/content/documentation/concepts/payload.md',
            'https://raw.githubusercontent.com/qdrant/landing_page/master/qdrant-landing/content/documentation/concepts/search.md',
            'https://raw.githubusercontent.com/qdrant/landing_page/master/qdrant-landing/content/documentation/concepts/points.md',
            'https://raw.githubusercontent.com/qdrant/landing_page/master/qdrant-landing/content/documentation/concepts/snapshots.md',
            'https://raw.githubusercontent.com/qdrant/landing_page/master/qdrant-landing/content/documentation/concepts/storage.md',
            'https://raw.githubusercontent.com/qdrant/landing_page/master/qdrant-landing/content/documentation/concepts/vectors.md'
        ]

    def extract_text_from_md(self, url, max_characters=1500, new_after_n_chars=1000, overlap=0):
        """Извлечение текста из Markdown файла и разбиение на параграфы"""
        try:
            response = requests.get(url)
            response.raise_for_status()
            html_content = markdown.markdown(response.text)
            soup = BeautifulSoup(html_content, features="html.parser")
            text = soup.get_text()
        except Exception as e:
            print(f"Ошибка при получении документа {url}: {e}")
            return []

        # Разделение на смысловые элементы
        raw_paragraphs = [p.strip() for p in re.split(r'\n\s*\n', text) if p.strip()]
        
        paragraphs = []
        current_chunk = ""
        
        for p in raw_paragraphs:
            # Нормализация пробелов
            cleaned_p = re.sub(r'\s+', ' ', p).strip()
            
            # Пропуск слишком коротких фрагментов
            if len(cleaned_p.split()) < 5:
                continue
                
            # Определение, является ли текущий параграф заголовком
            is_title = len(cleaned_p.split()) < 10 and not cleaned_p.endswith(('.', '?', '!'))
            
            # Если новый параграф - заголовок или текущий чанк станет слишком большим
            if is_title or len(current_chunk) + len(cleaned_p) > new_after_n_chars:
                # Сохранение предыдущего чанка, если он не пустой
                if current_chunk:
                    paragraphs.append(current_chunk)
                    current_chunk = ""
            
            # Если параграф слишком большой, разбиваем его на части
            if len(cleaned_p) > max_characters:
                # Разбиение на предложения
                sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?|\!)\s', cleaned_p)
                
                sentence_chunk = ""
                for sentence in sentences:
                    if len(sentence_chunk) + len(sentence) > max_characters:
                        paragraphs.append(sentence_chunk)
                        # Добавление перекрытия, если задано
                        if overlap > 0:
                            words = sentence_chunk.split()
                            overlap_text = ' '.join(words[-min(len(words), overlap//5):])
                            sentence_chunk = overlap_text + " " + sentence
                        else:
                            sentence_chunk = sentence
                    else:
                        sentence_chunk = (sentence_chunk + " " + sentence).strip() if sentence_chunk else sentence
                
                if sentence_chunk:
                    paragraphs.append(sentence_chunk)
            else:
                # Добавление параграфа к текущему чанку
                current_chunk = (current_chunk + "\n\n" + cleaned_p).strip() if current_chunk else cleaned_p
                
                # Если чанк превысил максимальный размер, сохраняем его
                if len(current_chunk) > max_characters:
                    paragraphs.append(current_chunk)
                    current_chunk = ""
        
        # Добавление последнего чанка, если он не пустой
        if current_chunk:
            paragraphs.append(current_chunk)
        
        return paragraphs

    def initialize_database(self):
        """Инициализация базы данных: загрузка и предобработка документов"""
        print("Загрузка модели...")
        try:
            self.model = SentenceTransformer('BAAI/bge-large-en-v1.5')
            self.reranker = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-12-v2')
            print("Модели успешно загружены")
        except Exception as e:
            print(f"Ошибка при загрузке модели: {e}")
            raise
        
        # Обработка всех документов
        self.doc_paragraphs = []
        for url in self.md_list:
            paragraphs = self.extract_text_from_md(url)
            name = url.split('concepts/')[1].split('.md')[0]
            if name == 'collections':
                paragraphs = [p for p in paragraphs if '/ Collections' not in p]
            else:
                paragraphs = [p for p in paragraphs if f'/{name}' not in p]
            for paragraph in paragraphs:
                self.doc_paragraphs.append({
                    'name': name,
                    'text': paragraph
                })
        print(f"Всего загружено {len(self.doc_paragraphs)} фрагментов.")
        if not self.doc_paragraphs:
            raise ValueError("Не удалось загрузить ни одного документа!")

        # Создание эмбеддингов для всех документов
        print("Генерация эмбеддингов для документов...")
        try:
            doc_texts = [f"passage: {doc['text']}" for doc in self.doc_paragraphs]
            self.doc_embeddings = self.model.encode(doc_texts, convert_to_numpy=True, show_progress_bar=True)
            print(f"Эмбеддинги успешно созданы. Размерность: {self.doc_embeddings.shape}")
        except Exception as e:
            print(f"Ошибка при создании эмбеддингов: {e}")
            raise

    def search_similar_paragraphs_with_reranking(self, user_query, top_k_initial=20, top_k_final=6):
        """Поиск похожих параграфов с использованием эмбеддингов E5 и реранкинга"""
        if self.doc_embeddings is None or not self.model:
            print("Ошибка: База данных не инициализирована!")
            return []

        try:
            # Формируем запрос с инструкцией для E5
            query_text = f"query: {user_query}"
            query_embedding = self.model.encode([query_text], convert_to_numpy=True)[0]

            # Вычисляем косинусное сходство между запросом и всеми документами
            similarities = cosine_similarity([query_embedding], self.doc_embeddings)[0]
            top_indices = np.argsort(similarities)[::-1][:top_k_initial]

            # Получаем топ-20 фрагментов
            initial_results = [(self.doc_paragraphs[idx]['text'], self.doc_paragraphs[idx]['name'], float(similarities[idx]))
                               for idx in top_indices]

            # Подготовка данных для реранкера
            rerank_input = [[user_query, text] for text, _, _ in initial_results]
            rerank_scores = self.reranker.predict(rerank_input)

            # Комбинируем результаты и сортируем по оценкам реранкера
            combined_results = [(text, name, score) for (text, name, _), score in zip(initial_results, rerank_scores)]
            sorted_results = sorted(combined_results, key=lambda x: x[2], reverse=True)

            # Возвращаем топ-k фрагментов после реранкинга
            return sorted_results[:top_k_final]
        except Exception as e:
            print(f"Ошибка при поиске похожих параграфов: {e}")
            return []

# Функция для оценки релевантности с использованием API
def evaluate_relevance_with_claude(question, fragment_text, api_key):
    """Оценивает релевантность фрагмента к вопросу через API Claude"""
    url = "https://ask.chadgpt.ru/api/public/gpt-4o-mini"
    
    # Ограничиваем длину фрагмента для запроса
    max_fragment_length = 4000
    if len(fragment_text) > max_fragment_length:
        fragment_text = fragment_text[:max_fragment_length] + "..."
    
    prompt = f"""
    Задача: оценить релевантность текстового фрагмента вопросу.
    
    Вопрос: {question}
    
    Фрагмент: {fragment_text}
    
    Оцени релевантность фрагмента к вопросу по шкале от 1 до 5, где:
    1 - совершенно не релевантен
    2 - слабо релевантен
    3 - умеренно релевантен
    4 - очень релевантен
    5 - идеально релевантен
    
    Ответь только числом от 1 до 5 без пояснений.
    """
    
    # Формируем запрос согласно примеру
    request_json = {
        "message": prompt,
        "api_key": api_key
    }
    
    try:
        # Отправляем запрос и дожидаемся ответа
        response = requests.post(url=url, json=request_json)
        
        # Проверяем, отправился ли запрос
        if response.status_code != 200:
            print(f'Ошибка! Код http-ответа: {response.status_code}')
            return 1  # Возвращаем минимальную оценку в случае ошибки
        else:
            # Получаем текст ответа и преобразовываем в dict
            resp_json = response.json()
            
            # Если успешен ответ, то извлекаем результат
            if resp_json['is_success']:
                resp_msg = resp_json['response'].strip()
                # Ищем число от 1 до 5 в ответе
                import re
                score_match = re.search(r'[1-5]', resp_msg)
                if score_match:
                    relevance_score = int(score_match.group(0))
                    return relevance_score
                else:
                    print(f'Не удалось извлечь оценку из ответа: {resp_msg}')
                    return 3  # Средняя оценка по умолчанию в случае неоднозначного ответа
            else:
                error = resp_json['error_message']
                print(f'Ошибка: {error}')
                return 1  # Возвращаем минимальную оценку в случае ошибки
    except Exception as e:
        print(f'Исключение при обработке запроса: {str(e)}')
        return 1  # Возвращаем минимальную оценку в случае ошибки

def get_relevant_fragments_e5_with_reranking(qa_system, question, top_k_initial=20, top_k_final=6):
    """Получение релевантных фрагментов с использованием E5 эмбеддингов и реранкера"""
    fragments = qa_system.search_similar_paragraphs_with_reranking(question, top_k_initial=top_k_initial, top_k_final=top_k_final)
    return fragments

def calculate_metrics(retrieval_results):
    """Вычисление метрик эффективности ретривала"""
    metrics = {
        'recall@1': [],
        'recall@4': [],
        'recall@6': [],
        'precision@1': [],
        'precision@4': [],
        'precision@6': [],
        'mrr@4': [],
        'mrr@6': [],
        'ndcg@4': [],
        'ndcg@6': []
    }
    
    for result in retrieval_results:
        fragments = result['fragments']
        if not fragments:
            print(f"Предупреждение: для вопроса '{result['question']}' не найдено фрагментов")
            # Пропускаем вычисление метрик для этого запроса
            continue
            
        # Сортировка фрагментов по оценке релевантности от Claude (по убыванию)
        sorted_fragments = sorted(fragments, key=lambda x: x[3], reverse=True)
        
        # Сортировка фрагментов по скору из системы ретривала (по убыванию)
        retrieved_fragments = sorted(fragments, key=lambda x: x[2], reverse=True)
        
        # Вычисление Recall@k
        relevant_fragments = [f for f in sorted_fragments if f[3] >= 4]  # Считаем релевантными фрагменты с оценкой >= 4
        total_relevant = len(relevant_fragments)
        
        if total_relevant > 0:
            # Recall@1
            relevant_at_1 = sum(1 for f in retrieved_fragments[:1] if f[3] >= 4)
            metrics['recall@1'].append(relevant_at_1 / total_relevant)
            
            # Recall@4
            relevant_at_4 = sum(1 for f in retrieved_fragments[:4] if f[3] >= 4)
            metrics['recall@4'].append(relevant_at_4 / total_relevant)
            
            # Recall@6
            relevant_at_6 = sum(1 for f in retrieved_fragments[:min(6, len(retrieved_fragments))] if f[3] >= 4)
            metrics['recall@6'].append(relevant_at_6 / total_relevant)
            
            # Precision@1
            metrics['precision@1'].append(relevant_at_1 / 1 if len(retrieved_fragments) >= 1 else 0)
            
            # Precision@4
            metrics['precision@4'].append(relevant_at_4 / min(4, len(retrieved_fragments)))
            
            # Precision@6
            metrics['precision@6'].append(relevant_at_6 / min(6, len(retrieved_fragments)))
        else:
            # Если нет релевантных фрагментов, устанавливаем recall = 1.0 (все релевантные найдены)
            metrics['recall@1'].append(1.0)
            metrics['recall@4'].append(1.0)
            metrics['recall@6'].append(1.0)
            
            # Если нет релевантных фрагментов, устанавливаем precision = 0.0
            metrics['precision@1'].append(0.0)
            metrics['precision@4'].append(0.0)
            metrics['precision@6'].append(0.0)
        
        # MRR@4 (Mean Reciprocal Rank для первых 4)
        first_relevant_rank_at_4 = next((i + 1 for i, f in enumerate(retrieved_fragments[:min(4, len(retrieved_fragments))]) if f[3] >= 4), 0)
        if first_relevant_rank_at_4 > 0:
            metrics['mrr@4'].append(1.0 / first_relevant_rank_at_4)
        else:
            metrics['mrr@4'].append(0.0)
            
        # MRR@6 (Mean Reciprocal Rank для первых 6)
        first_relevant_rank_at_6 = next((i + 1 for i, f in enumerate(retrieved_fragments[:min(6, len(retrieved_fragments))]) if f[3] >= 4), 0)
        if first_relevant_rank_at_6 > 0:
            metrics['mrr@6'].append(1.0 / first_relevant_rank_at_6)
        else:
            metrics['mrr@6'].append(0.0)
        
        # nDCG@4
        if len(sorted_fragments) >= 1 and len(retrieved_fragments) >= 1:
            # Определяем количество документов для оценки
            k4 = min(4, len(sorted_fragments), len(retrieved_fragments))
            
            # Берем только первые k4 документа
            true_relevance_4 = np.array([f[3] for f in sorted_fragments[:k4]])
            predicted_order_relevance_4 = np.array([f[3] for f in retrieved_fragments[:k4]])
            
            try:
                ndcg_4 = ndcg_score([true_relevance_4], [predicted_order_relevance_4])
                metrics['ndcg@4'].append(ndcg_4)
            except Exception as e:
                print(f"Ошибка при вычислении nDCG@4: {e}")
                metrics['ndcg@4'].append(0.0)
        else:
            metrics['ndcg@4'].append(0.0)
            
        # nDCG@6
        if len(sorted_fragments) >= 1 and len(retrieved_fragments) >= 1:
            # Определяем количество документов для оценки
            k6 = min(6, len(sorted_fragments), len(retrieved_fragments))
            
            # Берем только первые k6 документов
            true_relevance_6 = np.array([f[3] for f in sorted_fragments[:k6]])
            predicted_order_relevance_6 = np.array([f[3] for f in retrieved_fragments[:k6]])
            
            try:
                ndcg_6 = ndcg_score([true_relevance_6], [predicted_order_relevance_6])
                metrics['ndcg@6'].append(ndcg_6)
            except Exception as e:
                print(f"Ошибка при вычислении nDCG@6: {e}")
                metrics['ndcg@6'].append(0.0)
        else:
            metrics['ndcg@6'].append(0.0)
    
    # Вычисляем средние значения метрик
    result_metrics = {}
    for key, values in metrics.items():
        result_metrics[key] = sum(values) / len(values) if values else 0.0
    
    return result_metrics

# Основной код для тестирования системы и вычисления метрик

def run_evaluation_with_reranking(api_key, test_questions):
    """Запуск оценки системы ретривала на основе E5 и реранкера на наборе тестовых вопросов"""
    # Инициализация системы на основе E5
    qa_system = DocumentationQA_E5Embeddings()
    qa_system.initialize_database()

    # Результаты для последующей оценки
    retrieval_results = []

    # Обработка каждого вопроса
    for question in test_questions:
        # Получение фрагментов с помощью E5 и реранкера
        fragments = get_relevant_fragments_e5_with_reranking(qa_system, question, top_k_initial=20, top_k_final=6)
        result = {'question': question, 'fragments': []}

        # Оценка релевантности для каждого фрагмента
        for text, name, score in fragments:
            time.sleep(2)  # Задержка между запросами
            relevance_score = evaluate_relevance_with_claude(question, text, api_key)
            result['fragments'].append((text, name, score, relevance_score))
        retrieval_results.append(result)

    # Вычисление метрик
    metrics_results = calculate_metrics(retrieval_results)
    return metrics_results, retrieval_results

def save_results_to_csv(results, filename):
    """Сохранение результатов в CSV файл"""
    rows = []
    for result in results:
        question = result['question']
        for text, name, e5_score, relevance in result['fragments']:
            rows.append({
                'question': question,
                'document': name,
                'e5_score': e5_score,
                'relevance_score': relevance,
                'text': text[:200]  # Ограничиваем длину текста для CSV
            })
    
    df = pd.DataFrame(rows)
    df.to_csv(filename, index=False)
    print(f"Результаты сохранены в {filename}")

def main():
    try:
        # Загрузка API ключа
        api_key_file = 'api.txt'
        if os.path.exists(api_key_file):
            with open(api_key_file, 'r') as file:
                api_key = file.read().strip()
        else:
            print(f"Файл с API ключом {api_key_file} не найден!")
            api_key = input("Введите ваш API ключ: ")

        # Проверка API ключа
        if not api_key:
            raise ValueError("API ключ не может быть пустым")

        df = pd.read_csv('texts_with_answers.csv')
        test_questions = df.question.to_list()

        # Запуск оценки
        print("Начало оценки E5 ретривала с реранкером...")
        metrics, results = run_evaluation_with_reranking(api_key, test_questions)

        # Вывод результатов
        print("\nРезультаты оценки E5-ретривала с реранкером:")
        for metric, value in metrics.items():
            print(f"{metric}: {value:.4f}")

        # Детальный анализ результатов
        print("\nДетальный анализ результатов:")
        for result in results:
            question = result['question']
            print(f"\nВопрос: {question}")
            if not result['fragments']:
                print("Не найдено релевантных фрагментов для этого вопроса.")
                continue

            # Сортировка по оценке релевантности (от Claude)
            sorted_by_relevance = sorted(result['fragments'], key=lambda x: x[3], reverse=True)
            print("Топ-3 наиболее релевантных фрагмента по оценке Claude:")
            for i, (text, name, e5_score, relevance) in enumerate(sorted_by_relevance[:min(3, len(sorted_by_relevance))]):
                print(f"{i+1}. {name} (E5: {e5_score:.4f}, Релевантность: {relevance})")
                print(f"   {text[:100]}...")

            # Сортировка по E5 сходству
            sorted_by_e5 = sorted(result['fragments'], key=lambda x: x[2], reverse=True)
            print("\nТоп-3 наиболее релевантных фрагмента по E5:")
            for i, (text, name, e5_score, relevance) in enumerate(sorted_by_e5[:min(3, len(sorted_by_e5))]):
                print(f"{i+1}. {name} (E5: {e5_score:.4f}, Релевантность: {relevance})")
                print(f"   {text[:100]}...")

        # Сохранение результатов в CSV для дальнейшего анализа
        save_results_to_csv(results, "e5_reranker_retrieval_results.csv")
        return metrics, results
    except Exception as e:
        print(f"Ошибка в функции main: {e}")
        import traceback
        traceback.print_exc()
        return None, None

df = pd.read_csv('texts_with_answers.csv')
questions = df['question'].tolist()

api_key_file = 'api.txt'
if os.path.exists(api_key_file):
    with open(api_key_file, 'r') as file:
        api_key = file.read().strip()
metrics, res = run_evaluation_with_reranking(api_key=api_key, test_questions=questions)

metrics

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\sekho\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\sekho\AppData\Roaming\nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\sekho\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     C:\Users\sekho\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_eng is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\Users\sekho\AppData\Roaming\nltk_data...
[

Загрузка модели...
Модели успешно загружены
Всего загружено 130 фрагментов.
Генерация эмбеддингов для документов...


Batches:   0%|          | 0/5 [00:00<?, ?it/s]

Эмбеддинги успешно созданы. Размерность: (130, 1024)


{'recall@1': 0.5195833333333334,
 'recall@4': 0.8645833333333334,
 'recall@6': 1.0,
 'precision@1': 0.7166666666666667,
 'precision@4': 0.4395833333333333,
 'precision@6': 0.37361111111111095,
 'mrr@4': 0.7784722222222221,
 'mrr@6': 0.7801388888888889,
 'ndcg@4': np.float64(0.9745421620607291),
 'ndcg@6': np.float64(0.9590797106850054)}

Реранкер BAAI/bge-reranker-base

In [12]:
import requests
import markdown
from bs4 import BeautifulSoup
import re
import numpy as np
import pandas as pd
import time
from sklearn.metrics import ndcg_score
import nltk
import string
import os
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

nltk.download('all')

# Явная установка директории для загрузки NLTK данных
nltk_data_dir = os.path.join(os.getcwd(), 'nltk_data')
os.makedirs(nltk_data_dir, exist_ok=True)
nltk.data.path.append(nltk_data_dir)

# Загрузка необходимых ресурсов NLTK с явным указанием пути
try:
    nltk.download('punkt', download_dir=nltk_data_dir, quiet=True)
    nltk.download('stopwords', download_dir=nltk_data_dir, quiet=True)
    from nltk.tokenize import word_tokenize
    from nltk.corpus import stopwords
    from nltk.stem import PorterStemmer
    stop_words = set(stopwords.words('english'))
except LookupError:
    print("Не удалось загрузить ресурсы NLTK. Используем упрощенную токенизацию.")
    
    # Упрощенная имплементация токенизации без зависимостей NLTK
    def word_tokenize(text):
        """Простая токенизация по пробелам и пунктуации"""
        # Заменяем пунктуацию на пробелы
        for punct in string.punctuation:
            text = text.replace(punct, ' ')
        # Разбиваем по пробелам и фильтруем пустые токены
        return [token for token in text.lower().split() if token]
    
    # Пустой набор стоп-слов
    stop_words = set()
    
    # Упрощенный стеммер
    class SimplePorterStemmer:
        """Очень упрощенная версия стеммера - убирает только окончения -ing, -ed, -s"""
        def stem(self, word):
            if word.endswith('ing'):
                return word[:-3]
            elif word.endswith('ed') and len(word) > 3:
                return word[:-2]
            elif word.endswith('s') and len(word) > 2:
                return word[:-1]
            return word
    
    PorterStemmer = SimplePorterStemmer

class DocumentationQA_E5Embeddings:
    def __init__(self):
        self.model = None
        self.reranker = None
        self.doc_paragraphs = []
        self.doc_embeddings = None
        self.md_list = [
            'https://raw.githubusercontent.com/qdrant/landing_page/master/qdrant-landing/content/documentation/concepts/collections.md',
            'https://raw.githubusercontent.com/qdrant/landing_page/master/qdrant-landing/content/documentation/concepts/explore.md',
            'https://raw.githubusercontent.com/qdrant/landing_page/master/qdrant-landing/content/documentation/concepts/filtering.md',
            'https://raw.githubusercontent.com/qdrant/landing_page/master/qdrant-landing/content/documentation/concepts/hybrid-queries.md',
            'https://raw.githubusercontent.com/qdrant/landing_page/master/qdrant-landing/content/documentation/concepts/indexing.md',
            'https://raw.githubusercontent.com/qdrant/landing_page/master/qdrant-landing/content/documentation/concepts/optimizer.md',
            'https://raw.githubusercontent.com/qdrant/landing_page/master/qdrant-landing/content/documentation/concepts/payload.md',
            'https://raw.githubusercontent.com/qdrant/landing_page/master/qdrant-landing/content/documentation/concepts/search.md',
            'https://raw.githubusercontent.com/qdrant/landing_page/master/qdrant-landing/content/documentation/concepts/points.md',
            'https://raw.githubusercontent.com/qdrant/landing_page/master/qdrant-landing/content/documentation/concepts/snapshots.md',
            'https://raw.githubusercontent.com/qdrant/landing_page/master/qdrant-landing/content/documentation/concepts/storage.md',
            'https://raw.githubusercontent.com/qdrant/landing_page/master/qdrant-landing/content/documentation/concepts/vectors.md'
        ]

    def extract_text_from_md(self, url, max_characters=1500, new_after_n_chars=1000, overlap=0):
        """Извлечение текста из Markdown файла и разбиение на параграфы"""
        try:
            response = requests.get(url)
            response.raise_for_status()
            html_content = markdown.markdown(response.text)
            soup = BeautifulSoup(html_content, features="html.parser")
            text = soup.get_text()
        except Exception as e:
            print(f"Ошибка при получении документа {url}: {e}")
            return []

        # Разделение на смысловые элементы
        raw_paragraphs = [p.strip() for p in re.split(r'\n\s*\n', text) if p.strip()]
        
        paragraphs = []
        current_chunk = ""
        
        for p in raw_paragraphs:
            # Нормализация пробелов
            cleaned_p = re.sub(r'\s+', ' ', p).strip()
            
            # Пропуск слишком коротких фрагментов
            if len(cleaned_p.split()) < 5:
                continue
                
            # Определение, является ли текущий параграф заголовком
            is_title = len(cleaned_p.split()) < 10 and not cleaned_p.endswith(('.', '?', '!'))
            
            # Если новый параграф - заголовок или текущий чанк станет слишком большим
            if is_title or len(current_chunk) + len(cleaned_p) > new_after_n_chars:
                # Сохранение предыдущего чанка, если он не пустой
                if current_chunk:
                    paragraphs.append(current_chunk)
                    current_chunk = ""
            
            # Если параграф слишком большой, разбиваем его на части
            if len(cleaned_p) > max_characters:
                # Разбиение на предложения
                sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?|\!)\s', cleaned_p)
                
                sentence_chunk = ""
                for sentence in sentences:
                    if len(sentence_chunk) + len(sentence) > max_characters:
                        paragraphs.append(sentence_chunk)
                        # Добавление перекрытия, если задано
                        if overlap > 0:
                            words = sentence_chunk.split()
                            overlap_text = ' '.join(words[-min(len(words), overlap//5):])
                            sentence_chunk = overlap_text + " " + sentence
                        else:
                            sentence_chunk = sentence
                    else:
                        sentence_chunk = (sentence_chunk + " " + sentence).strip() if sentence_chunk else sentence
                
                if sentence_chunk:
                    paragraphs.append(sentence_chunk)
            else:
                # Добавление параграфа к текущему чанку
                current_chunk = (current_chunk + "\n\n" + cleaned_p).strip() if current_chunk else cleaned_p
                
                # Если чанк превысил максимальный размер, сохраняем его
                if len(current_chunk) > max_characters:
                    paragraphs.append(current_chunk)
                    current_chunk = ""
        
        # Добавление последнего чанка, если он не пустой
        if current_chunk:
            paragraphs.append(current_chunk)
        
        return paragraphs

    def initialize_database(self):
        """Инициализация базы данных: загрузка и предобработка документов"""
        print("Загрузка моделей...")
        try:
            self.model = SentenceTransformer('BAAI/bge-large-en-v1.5')
            self.reranker = CrossEncoder('BAAI/bge-reranker-base')
            print("Модели успешно загружены")
        except Exception as e:
            print(f"Ошибка при загрузке моделей: {e}")
            raise

        # Обработка всех документов
        self.doc_paragraphs = []
        for url in self.md_list:
            paragraphs = self.extract_text_from_md(url)
            name = url.split('concepts/')[1].split('.md')[0]
            if name == 'collections':
                paragraphs = [p for p in paragraphs if '/ Collections' not in p]
            else:
                paragraphs = [p for p in paragraphs if f'/{name}' not in p]
            for paragraph in paragraphs:
                self.doc_paragraphs.append({
                    'name': name,
                    'text': paragraph
                })
        print(f"Всего загружено {len(self.doc_paragraphs)} фрагментов.")
        if not self.doc_paragraphs:
            raise ValueError("Не удалось загрузить ни одного документа!")

        # Создание эмбеддингов для всех документов
        print("Генерация эмбеддингов для документов...")
        try:
            doc_texts = [f"passage: {doc['text']}" for doc in self.doc_paragraphs]
            self.doc_embeddings = self.model.encode(doc_texts, convert_to_numpy=True, show_progress_bar=True)
            print(f"Эмбеддинги успешно созданы. Размерность: {self.doc_embeddings.shape}")
        except Exception as e:
            print(f"Ошибка при создании эмбеддингов: {e}")
            raise

    def search_similar_paragraphs_with_reranking(self, user_query, top_k_initial=20, top_k_final=6):
        """Поиск похожих параграфов с использованием эмбеддингов E5 и реранкера BAAI/bge-reranker-base"""
        if self.doc_embeddings is None or not self.model:
            print("Ошибка: База данных не инициализирована!")
            return []

        try:
            # Формируем запрос с инструкцией для E5
            query_text = f"query: {user_query}"
            query_embedding = self.model.encode([query_text], convert_to_numpy=True)[0]

            # Вычисляем косинусное сходство между запросом и всеми документами
            similarities = cosine_similarity([query_embedding], self.doc_embeddings)[0]
            top_indices = np.argsort(similarities)[::-1][:top_k_initial]

            # Получаем топ-20 фрагментов
            initial_results = [(self.doc_paragraphs[idx]['text'], self.doc_paragraphs[idx]['name'], float(similarities[idx]))
                               for idx in top_indices]

            # Подготовка данных для реранкера
            rerank_input = [[user_query, text] for text, _, _ in initial_results]
            rerank_scores = self.reranker.predict(rerank_input)

            # Комбинируем результаты и сортируем по оценкам реранкера
            combined_results = [(text, name, score) for (text, name, _), score in zip(initial_results, rerank_scores)]
            sorted_results = sorted(combined_results, key=lambda x: x[2], reverse=True)

            # Возвращаем топ-k фрагментов после реранкинга
            return sorted_results[:top_k_final]
        except Exception as e:
            print(f"Ошибка при поиске похожих параграфов: {e}")
            return []

# Функция для оценки релевантности с использованием API
def evaluate_relevance_with_claude(question, fragment_text, api_key):
    """Оценивает релевантность фрагмента к вопросу через API Claude"""
    url = "https://ask.chadgpt.ru/api/public/gpt-4o-mini"
    
    # Ограничиваем длину фрагмента для запроса
    max_fragment_length = 4000
    if len(fragment_text) > max_fragment_length:
        fragment_text = fragment_text[:max_fragment_length] + "..."
    
    prompt = f"""
    Задача: оценить релевантность текстового фрагмента вопросу.
    
    Вопрос: {question}
    
    Фрагмент: {fragment_text}
    
    Оцени релевантность фрагмента к вопросу по шкале от 1 до 5, где:
    1 - совершенно не релевантен
    2 - слабо релевантен
    3 - умеренно релевантен
    4 - очень релевантен
    5 - идеально релевантен
    
    Ответь только числом от 1 до 5 без пояснений.
    """
    
    # Формируем запрос согласно примеру
    request_json = {
        "message": prompt,
        "api_key": api_key
    }
    
    try:
        # Отправляем запрос и дожидаемся ответа
        response = requests.post(url=url, json=request_json)
        
        # Проверяем, отправился ли запрос
        if response.status_code != 200:
            print(f'Ошибка! Код http-ответа: {response.status_code}')
            return 1  # Возвращаем минимальную оценку в случае ошибки
        else:
            # Получаем текст ответа и преобразовываем в dict
            resp_json = response.json()
            
            # Если успешен ответ, то извлекаем результат
            if resp_json['is_success']:
                resp_msg = resp_json['response'].strip()
                # Ищем число от 1 до 5 в ответе
                import re
                score_match = re.search(r'[1-5]', resp_msg)
                if score_match:
                    relevance_score = int(score_match.group(0))
                    return relevance_score
                else:
                    print(f'Не удалось извлечь оценку из ответа: {resp_msg}')
                    return 3  # Средняя оценка по умолчанию в случае неоднозначного ответа
            else:
                error = resp_json['error_message']
                print(f'Ошибка: {error}')
                return 1  # Возвращаем минимальную оценку в случае ошибки
    except Exception as e:
        print(f'Исключение при обработке запроса: {str(e)}')
        return 1  # Возвращаем минимальную оценку в случае ошибки

def get_relevant_fragments_e5_with_reranking(qa_system, question, top_k_initial=20, top_k_final=6):
    """Получение релевантных фрагментов с использованием E5 эмбеддингов и реранкера"""
    fragments = qa_system.search_similar_paragraphs_with_reranking(question, top_k_initial=top_k_initial, top_k_final=top_k_final)
    return fragments

def calculate_metrics(retrieval_results):
    """Вычисление метрик эффективности ретривала"""
    metrics = {
        'recall@1': [],
        'recall@4': [],
        'recall@6': [],
        'precision@1': [],
        'precision@4': [],
        'precision@6': [],
        'mrr@4': [],
        'mrr@6': [],
        'ndcg@4': [],
        'ndcg@6': []
    }
    
    for result in retrieval_results:
        fragments = result['fragments']
        if not fragments:
            print(f"Предупреждение: для вопроса '{result['question']}' не найдено фрагментов")
            # Пропускаем вычисление метрик для этого запроса
            continue
            
        # Сортировка фрагментов по оценке релевантности от Claude (по убыванию)
        sorted_fragments = sorted(fragments, key=lambda x: x[3], reverse=True)
        
        # Сортировка фрагментов по скору из системы ретривала (по убыванию)
        retrieved_fragments = sorted(fragments, key=lambda x: x[2], reverse=True)
        
        # Вычисление Recall@k
        relevant_fragments = [f for f in sorted_fragments if f[3] >= 4]  # Считаем релевантными фрагменты с оценкой >= 4
        total_relevant = len(relevant_fragments)
        
        if total_relevant > 0:
            # Recall@1
            relevant_at_1 = sum(1 for f in retrieved_fragments[:1] if f[3] >= 4)
            metrics['recall@1'].append(relevant_at_1 / total_relevant)
            
            # Recall@4
            relevant_at_4 = sum(1 for f in retrieved_fragments[:4] if f[3] >= 4)
            metrics['recall@4'].append(relevant_at_4 / total_relevant)
            
            # Recall@6
            relevant_at_6 = sum(1 for f in retrieved_fragments[:min(6, len(retrieved_fragments))] if f[3] >= 4)
            metrics['recall@6'].append(relevant_at_6 / total_relevant)
            
            # Precision@1
            metrics['precision@1'].append(relevant_at_1 / 1 if len(retrieved_fragments) >= 1 else 0)
            
            # Precision@4
            metrics['precision@4'].append(relevant_at_4 / min(4, len(retrieved_fragments)))
            
            # Precision@6
            metrics['precision@6'].append(relevant_at_6 / min(6, len(retrieved_fragments)))
        else:
            # Если нет релевантных фрагментов, устанавливаем recall = 1.0 (все релевантные найдены)
            metrics['recall@1'].append(1.0)
            metrics['recall@4'].append(1.0)
            metrics['recall@6'].append(1.0)
            
            # Если нет релевантных фрагментов, устанавливаем precision = 0.0
            metrics['precision@1'].append(0.0)
            metrics['precision@4'].append(0.0)
            metrics['precision@6'].append(0.0)
        
        # MRR@4 (Mean Reciprocal Rank для первых 4)
        first_relevant_rank_at_4 = next((i + 1 for i, f in enumerate(retrieved_fragments[:min(4, len(retrieved_fragments))]) if f[3] >= 4), 0)
        if first_relevant_rank_at_4 > 0:
            metrics['mrr@4'].append(1.0 / first_relevant_rank_at_4)
        else:
            metrics['mrr@4'].append(0.0)
            
        # MRR@6 (Mean Reciprocal Rank для первых 6)
        first_relevant_rank_at_6 = next((i + 1 for i, f in enumerate(retrieved_fragments[:min(6, len(retrieved_fragments))]) if f[3] >= 4), 0)
        if first_relevant_rank_at_6 > 0:
            metrics['mrr@6'].append(1.0 / first_relevant_rank_at_6)
        else:
            metrics['mrr@6'].append(0.0)
        
        # nDCG@4
        if len(sorted_fragments) >= 1 and len(retrieved_fragments) >= 1:
            # Определяем количество документов для оценки
            k4 = min(4, len(sorted_fragments), len(retrieved_fragments))
            
            # Берем только первые k4 документа
            true_relevance_4 = np.array([f[3] for f in sorted_fragments[:k4]])
            predicted_order_relevance_4 = np.array([f[3] for f in retrieved_fragments[:k4]])
            
            try:
                ndcg_4 = ndcg_score([true_relevance_4], [predicted_order_relevance_4])
                metrics['ndcg@4'].append(ndcg_4)
            except Exception as e:
                print(f"Ошибка при вычислении nDCG@4: {e}")
                metrics['ndcg@4'].append(0.0)
        else:
            metrics['ndcg@4'].append(0.0)
            
        # nDCG@6
        if len(sorted_fragments) >= 1 and len(retrieved_fragments) >= 1:
            # Определяем количество документов для оценки
            k6 = min(6, len(sorted_fragments), len(retrieved_fragments))
            
            # Берем только первые k6 документов
            true_relevance_6 = np.array([f[3] for f in sorted_fragments[:k6]])
            predicted_order_relevance_6 = np.array([f[3] for f in retrieved_fragments[:k6]])
            
            try:
                ndcg_6 = ndcg_score([true_relevance_6], [predicted_order_relevance_6])
                metrics['ndcg@6'].append(ndcg_6)
            except Exception as e:
                print(f"Ошибка при вычислении nDCG@6: {e}")
                metrics['ndcg@6'].append(0.0)
        else:
            metrics['ndcg@6'].append(0.0)
    
    # Вычисляем средние значения метрик
    result_metrics = {}
    for key, values in metrics.items():
        result_metrics[key] = sum(values) / len(values) if values else 0.0
    
    return result_metrics

# Основной код для тестирования системы и вычисления метрик

def run_evaluation_with_reranking(api_key, test_questions):
    """Запуск оценки системы ретривала на основе E5 и реранкера на наборе тестовых вопросов"""
    # Инициализация системы на основе E5
    qa_system = DocumentationQA_E5Embeddings()
    qa_system.initialize_database()

    # Результаты для последующей оценки
    retrieval_results = []

    # Обработка каждого вопроса
    for question in test_questions:
        # Получение фрагментов с помощью E5 и реранкера
        fragments = get_relevant_fragments_e5_with_reranking(qa_system, question, top_k_initial=20, top_k_final=6)
        result = {'question': question, 'fragments': []}

        # Оценка релевантности для каждого фрагмента
        for text, name, score in fragments:
            time.sleep(2)  # Задержка между запросами
            relevance_score = evaluate_relevance_with_claude(question, text, api_key)
            result['fragments'].append((text, name, score, relevance_score))
        retrieval_results.append(result)

    # Вычисление метрик
    metrics_results = calculate_metrics(retrieval_results)
    return metrics_results, retrieval_results

def save_results_to_csv(results, filename):
    """Сохранение результатов в CSV файл"""
    rows = []
    for result in results:
        question = result['question']
        for text, name, e5_score, relevance in result['fragments']:
            rows.append({
                'question': question,
                'document': name,
                'e5_score': e5_score,
                'relevance_score': relevance,
                'text': text[:200]  # Ограничиваем длину текста для CSV
            })
    
    df = pd.DataFrame(rows)
    df.to_csv(filename, index=False)
    print(f"Результаты сохранены в {filename}")

def main():
    try:
        # Загрузка API ключа
        api_key_file = 'api.txt'
        if os.path.exists(api_key_file):
            with open(api_key_file, 'r') as file:
                api_key = file.read().strip()
        else:
            print(f"Файл с API ключом {api_key_file} не найден!")
            api_key = input("Введите ваш API ключ: ")

        # Проверка API ключа
        if not api_key:
            raise ValueError("API ключ не может быть пустым")

        df = pd.read_csv('texts_with_answers.csv')
        test_questions = df.question.to_list()

        # Запуск оценки
        print("Начало оценки E5 ретривала с реранкером BAAI/bge-reranker-base...")
        metrics, results = run_evaluation_with_reranking(api_key, test_questions)

        # Вывод результатов
        print("\nРезультаты оценки E5-ретривала с реранкером:")
        for metric, value in metrics.items():
            print(f"{metric}: {value:.4f}")

        # Детальный анализ результатов
        print("\nДетальный анализ результатов:")
        for result in results:
            question = result['question']
            print(f"\nВопрос: {question}")
            if not result['fragments']:
                print("Не найдено релевантных фрагментов для этого вопроса.")
                continue

            # Сортировка по оценке релевантности (от Claude)
            sorted_by_relevance = sorted(result['fragments'], key=lambda x: x[3], reverse=True)
            print("Топ-3 наиболее релевантных фрагмента по оценке Claude:")
            for i, (text, name, e5_score, relevance) in enumerate(sorted_by_relevance[:min(3, len(sorted_by_relevance))]):
                print(f"{i+1}. {name} (E5: {e5_score:.4f}, Релевантность: {relevance})")
                print(f"   {text[:100]}...")

            # Сортировка по E5 сходству
            sorted_by_e5 = sorted(result['fragments'], key=lambda x: x[2], reverse=True)
            print("\nТоп-3 наиболее релевантных фрагмента по E5:")
            for i, (text, name, e5_score, relevance) in enumerate(sorted_by_e5[:min(3, len(sorted_by_e5))]):
                print(f"{i+1}. {name} (E5: {e5_score:.4f}, Релевантность: {relevance})")
                print(f"   {text[:100]}...")

        # Сохранение результатов в CSV для дальнейшего анализа
        save_results_to_csv(results, "e5_bge_reranker_retrieval_results.csv")
        return metrics, results
    except Exception as e:
        print(f"Ошибка в функции main: {e}")
        import traceback
        traceback.print_exc()
        return None, None

df = pd.read_csv('texts_with_answers.csv')
questions = df['question'].tolist()

api_key_file = 'api.txt'
if os.path.exists(api_key_file):
    with open(api_key_file, 'r') as file:
        api_key = file.read().strip()
metrics, res = run_evaluation_with_reranking(api_key=api_key, test_questions=questions)

metrics

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\sekho\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\sekho\AppData\Roaming\nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\sekho\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     C:\Users\sekho\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_eng is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\Users\sekho\AppData\Roaming\nltk_data...
[

Загрузка моделей...
Модели успешно загружены
Всего загружено 130 фрагментов.
Генерация эмбеддингов для документов...


Batches:   0%|          | 0/5 [00:00<?, ?it/s]

Эмбеддинги успешно созданы. Размерность: (130, 1024)


{'recall@1': 0.4730555555555557,
 'recall@4': 0.8708333333333333,
 'recall@6': 1.0,
 'precision@1': 0.7333333333333333,
 'precision@4': 0.4625,
 'precision@6': 0.38333333333333336,
 'mrr@4': 0.8,
 'mrr@6': 0.8016666666666666,
 'ndcg@4': np.float64(0.9712515695237147),
 'ndcg@6': np.float64(0.9554671891932263)}

Реарнкер nboost/pt-tinybert-msmarco, модель та же

In [13]:
import requests
import markdown
from bs4 import BeautifulSoup
import re
import numpy as np
import pandas as pd
import time
from sklearn.metrics import ndcg_score
import nltk
import string
import os
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

nltk.download('all')

# Явная установка директории для загрузки NLTK данных
nltk_data_dir = os.path.join(os.getcwd(), 'nltk_data')
os.makedirs(nltk_data_dir, exist_ok=True)
nltk.data.path.append(nltk_data_dir)

# Загрузка необходимых ресурсов NLTK с явным указанием пути
try:
    nltk.download('punkt', download_dir=nltk_data_dir, quiet=True)
    nltk.download('stopwords', download_dir=nltk_data_dir, quiet=True)
    from nltk.tokenize import word_tokenize
    from nltk.corpus import stopwords
    from nltk.stem import PorterStemmer
    stop_words = set(stopwords.words('english'))
except LookupError:
    print("Не удалось загрузить ресурсы NLTK. Используем упрощенную токенизацию.")
    
    # Упрощенная имплементация токенизации без зависимостей NLTK
    def word_tokenize(text):
        """Простая токенизация по пробелам и пунктуации"""
        # Заменяем пунктуацию на пробелы
        for punct in string.punctuation:
            text = text.replace(punct, ' ')
        # Разбиваем по пробелам и фильтруем пустые токены
        return [token for token in text.lower().split() if token]
    
    # Пустой набор стоп-слов
    stop_words = set()
    
    # Упрощенный стеммер
    class SimplePorterStemmer:
        """Очень упрощенная версия стеммера - убирает только окончения -ing, -ed, -s"""
        def stem(self, word):
            if word.endswith('ing'):
                return word[:-3]
            elif word.endswith('ed') and len(word) > 3:
                return word[:-2]
            elif word.endswith('s') and len(word) > 2:
                return word[:-1]
            return word
    
    PorterStemmer = SimplePorterStemmer

class DocumentationQA_E5Embeddings:
    def __init__(self):
        self.model = None
        self.reranker_tokenizer = None
        self.reranker_model = None
        self.doc_paragraphs = []
        self.doc_embeddings = None
        self.md_list = [
            'https://raw.githubusercontent.com/qdrant/landing_page/master/qdrant-landing/content/documentation/concepts/collections.md',
            'https://raw.githubusercontent.com/qdrant/landing_page/master/qdrant-landing/content/documentation/concepts/explore.md',
            'https://raw.githubusercontent.com/qdrant/landing_page/master/qdrant-landing/content/documentation/concepts/filtering.md',
            'https://raw.githubusercontent.com/qdrant/landing_page/master/qdrant-landing/content/documentation/concepts/hybrid-queries.md',
            'https://raw.githubusercontent.com/qdrant/landing_page/master/qdrant-landing/content/documentation/concepts/indexing.md',
            'https://raw.githubusercontent.com/qdrant/landing_page/master/qdrant-landing/content/documentation/concepts/optimizer.md',
            'https://raw.githubusercontent.com/qdrant/landing_page/master/qdrant-landing/content/documentation/concepts/payload.md',
            'https://raw.githubusercontent.com/qdrant/landing_page/master/qdrant-landing/content/documentation/concepts/search.md',
            'https://raw.githubusercontent.com/qdrant/landing_page/master/qdrant-landing/content/documentation/concepts/points.md',
            'https://raw.githubusercontent.com/qdrant/landing_page/master/qdrant-landing/content/documentation/concepts/snapshots.md',
            'https://raw.githubusercontent.com/qdrant/landing_page/master/qdrant-landing/content/documentation/concepts/storage.md',
            'https://raw.githubusercontent.com/qdrant/landing_page/master/qdrant-landing/content/documentation/concepts/vectors.md'
        ]

    def extract_text_from_md(self, url, max_characters=1500, new_after_n_chars=1000, overlap=0):
        """Извлечение текста из Markdown файла и разбиение на параграфы"""
        try:
            response = requests.get(url)
            response.raise_for_status()
            html_content = markdown.markdown(response.text)
            soup = BeautifulSoup(html_content, features="html.parser")
            text = soup.get_text()
        except Exception as e:
            print(f"Ошибка при получении документа {url}: {e}")
            return []

        # Разделение на смысловые элементы
        raw_paragraphs = [p.strip() for p in re.split(r'\n\s*\n', text) if p.strip()]
        
        paragraphs = []
        current_chunk = ""
        
        for p in raw_paragraphs:
            # Нормализация пробелов
            cleaned_p = re.sub(r'\s+', ' ', p).strip()
            
            # Пропуск слишком коротких фрагментов
            if len(cleaned_p.split()) < 5:
                continue
                
            # Определение, является ли текущий параграф заголовком
            is_title = len(cleaned_p.split()) < 10 and not cleaned_p.endswith(('.', '?', '!'))
            
            # Если новый параграф - заголовок или текущий чанк станет слишком большим
            if is_title or len(current_chunk) + len(cleaned_p) > new_after_n_chars:
                # Сохранение предыдущего чанка, если он не пустой
                if current_chunk:
                    paragraphs.append(current_chunk)
                    current_chunk = ""
            
            # Если параграф слишком большой, разбиваем его на части
            if len(cleaned_p) > max_characters:
                # Разбиение на предложения
                sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?|\!)\s', cleaned_p)
                
                sentence_chunk = ""
                for sentence in sentences:
                    if len(sentence_chunk) + len(sentence) > max_characters:
                        paragraphs.append(sentence_chunk)
                        # Добавление перекрытия, если задано
                        if overlap > 0:
                            words = sentence_chunk.split()
                            overlap_text = ' '.join(words[-min(len(words), overlap//5):])
                            sentence_chunk = overlap_text + " " + sentence
                        else:
                            sentence_chunk = sentence
                    else:
                        sentence_chunk = (sentence_chunk + " " + sentence).strip() if sentence_chunk else sentence
                
                if sentence_chunk:
                    paragraphs.append(sentence_chunk)
            else:
                # Добавление параграфа к текущему чанку
                current_chunk = (current_chunk + "\n\n" + cleaned_p).strip() if current_chunk else cleaned_p
                
                # Если чанк превысил максимальный размер, сохраняем его
                if len(current_chunk) > max_characters:
                    paragraphs.append(current_chunk)
                    current_chunk = ""
        
        # Добавление последнего чанка, если он не пустой
        if current_chunk:
            paragraphs.append(current_chunk)
        
        return paragraphs

    def initialize_database(self):
        """Инициализация базы данных: загрузка и предобработка документов"""
        print("Загрузка моделей...")
        try:
            # Загрузка модели SentenceTransformer для начального поиска
            self.model = SentenceTransformer('BAAI/bge-large-en-v1.5')
            
            # Загрузка модели nboost/pt-tinybert-msmarco для реранкинга
            self.reranker_tokenizer = AutoTokenizer.from_pretrained('nboost/pt-tinybert-msmarco')
            self.reranker_model = AutoModelForSequenceClassification.from_pretrained('nboost/pt-tinybert-msmarco')
            self.reranker_model.eval()  # Переводим модель в режим оценки
            
            print("Модели успешно загружены")
        except Exception as e:
            print(f"Ошибка при загрузке моделей: {e}")
            raise

        # Обработка всех документов
        self.doc_paragraphs = []
        for url in self.md_list:
            paragraphs = self.extract_text_from_md(url)
            name = url.split('concepts/')[1].split('.md')[0]
            if name == 'collections':
                paragraphs = [p for p in paragraphs if '/ Collections' not in p]
            else:
                paragraphs = [p for p in paragraphs if f'/{name}' not in p]
            for paragraph in paragraphs:
                self.doc_paragraphs.append({
                    'name': name,
                    'text': paragraph
                })
        print(f"Всего загружено {len(self.doc_paragraphs)} фрагментов.")
        if not self.doc_paragraphs:
            raise ValueError("Не удалось загрузить ни одного документа!")

        # Создание эмбеддингов для всех документов
        print("Генерация эмбеддингов для документов...")
        try:
            doc_texts = [f"passage: {doc['text']}" for doc in self.doc_paragraphs]
            self.doc_embeddings = self.model.encode(doc_texts, convert_to_numpy=True, show_progress_bar=True)
            print(f"Эмбеддинги успешно созданы. Размерность: {self.doc_embeddings.shape}")
        except Exception as e:
            print(f"Ошибка при создании эмбеддингов: {e}")
            raise

    def rerank_with_nboost(self, query, passages):
        """Реранкинг с использованием модели nboost/pt-tinybert-msmarco"""
        inputs = self.reranker_tokenizer(
            [query] * len(passages),
            passages,
            padding=True,
            truncation=True,
            return_tensors="pt",
            max_length=512
        )
        with torch.no_grad():
            outputs = self.reranker_model(**inputs)
        scores = torch.softmax(outputs.logits, dim=1)[:, 1].cpu().numpy()  # Вероятности релевантности
        return scores

    def search_similar_paragraphs_with_reranking(self, user_query, top_k_initial=20, top_k_final=6):
        """Поиск похожих параграфов с использованием эмбеддингов E5 и реранкера nboost/pt-tinybert-msmarco"""
        if self.doc_embeddings is None or not self.model:
            print("Ошибка: База данных не инициализирована!")
            return []

        try:
            # Формируем запрос с инструкцией для E5
            query_text = f"query: {user_query}"
            query_embedding = self.model.encode([query_text], convert_to_numpy=True)[0]

            # Вычисляем косинусное сходство между запросом и всеми документами
            similarities = cosine_similarity([query_embedding], self.doc_embeddings)[0]
            top_indices = np.argsort(similarities)[::-1][:top_k_initial]

            # Получаем топ-20 фрагментов
            initial_results = [(self.doc_paragraphs[idx]['text'], self.doc_paragraphs[idx]['name'], float(similarities[idx]))
                               for idx in top_indices]

            # Подготовка данных для реранкера
            passages = [text for text, _, _ in initial_results]
            rerank_scores = self.rerank_with_nboost(user_query, passages)

            # Комбинируем результаты и сортируем по оценкам реранкера
            combined_results = [(text, name, score) for (text, name, _), score in zip(initial_results, rerank_scores)]
            sorted_results = sorted(combined_results, key=lambda x: x[2], reverse=True)

            # Возвращаем топ-k фрагментов после реранкинга
            return sorted_results[:top_k_final]
        except Exception as e:
            print(f"Ошибка при поиске похожих параграфов: {e}")
            return []

# Функция для оценки релевантности с использованием API
def evaluate_relevance_with_claude(question, fragment_text, api_key):
    """Оценивает релевантность фрагмента к вопросу через API Claude"""
    url = "https://ask.chadgpt.ru/api/public/gpt-4o-mini"
    
    # Ограничиваем длину фрагмента для запроса
    max_fragment_length = 4000
    if len(fragment_text) > max_fragment_length:
        fragment_text = fragment_text[:max_fragment_length] + "..."
    
    prompt = f"""
    Задача: оценить релевантность текстового фрагмента вопросу.
    
    Вопрос: {question}
    
    Фрагмент: {fragment_text}
    
    Оцени релевантность фрагмента к вопросу по шкале от 1 до 5, где:
    1 - совершенно не релевантен
    2 - слабо релевантен
    3 - умеренно релевантен
    4 - очень релевантен
    5 - идеально релевантен
    
    Ответь только числом от 1 до 5 без пояснений.
    """
    
    # Формируем запрос согласно примеру
    request_json = {
        "message": prompt,
        "api_key": api_key
    }
    
    try:
        # Отправляем запрос и дожидаемся ответа
        response = requests.post(url=url, json=request_json)
        
        # Проверяем, отправился ли запрос
        if response.status_code != 200:
            print(f'Ошибка! Код http-ответа: {response.status_code}')
            return 1  # Возвращаем минимальную оценку в случае ошибки
        else:
            # Получаем текст ответа и преобразовываем в dict
            resp_json = response.json()
            
            # Если успешен ответ, то извлекаем результат
            if resp_json['is_success']:
                resp_msg = resp_json['response'].strip()
                # Ищем число от 1 до 5 в ответе
                import re
                score_match = re.search(r'[1-5]', resp_msg)
                if score_match:
                    relevance_score = int(score_match.group(0))
                    return relevance_score
                else:
                    print(f'Не удалось извлечь оценку из ответа: {resp_msg}')
                    return 3  # Средняя оценка по умолчанию в случае неоднозначного ответа
            else:
                error = resp_json['error_message']
                print(f'Ошибка: {error}')
                return 1  # Возвращаем минимальную оценку в случае ошибки
    except Exception as e:
        print(f'Исключение при обработке запроса: {str(e)}')
        return 1  # Возвращаем минимальную оценку в случае ошибки

def get_relevant_fragments_e5(qa_system, question, top_k=6):
    """Получение релевантных фрагментов с использованием E5 эмбеддингов"""
    fragments = qa_system.search_similar_paragraphs(question, top_k=top_k)
    return fragments

def calculate_metrics(retrieval_results):
    """Вычисление метрик эффективности ретривала"""
    metrics = {
        'recall@1': [],
        'recall@4': [],
        'recall@6': [],
        'precision@1': [],
        'precision@4': [],
        'precision@6': [],
        'mrr@4': [],
        'mrr@6': [],
        'ndcg@4': [],
        'ndcg@6': []
    }
    
    for result in retrieval_results:
        fragments = result['fragments']
        if not fragments:
            print(f"Предупреждение: для вопроса '{result['question']}' не найдено фрагментов")
            # Пропускаем вычисление метрик для этого запроса
            continue
            
        # Сортировка фрагментов по оценке релевантности от Claude (по убыванию)
        sorted_fragments = sorted(fragments, key=lambda x: x[3], reverse=True)
        
        # Сортировка фрагментов по скору из системы ретривала (по убыванию)
        retrieved_fragments = sorted(fragments, key=lambda x: x[2], reverse=True)
        
        # Вычисление Recall@k
        relevant_fragments = [f for f in sorted_fragments if f[3] >= 4]  # Считаем релевантными фрагменты с оценкой >= 4
        total_relevant = len(relevant_fragments)
        
        if total_relevant > 0:
            # Recall@1
            relevant_at_1 = sum(1 for f in retrieved_fragments[:1] if f[3] >= 4)
            metrics['recall@1'].append(relevant_at_1 / total_relevant)
            
            # Recall@4
            relevant_at_4 = sum(1 for f in retrieved_fragments[:4] if f[3] >= 4)
            metrics['recall@4'].append(relevant_at_4 / total_relevant)
            
            # Recall@6
            relevant_at_6 = sum(1 for f in retrieved_fragments[:min(6, len(retrieved_fragments))] if f[3] >= 4)
            metrics['recall@6'].append(relevant_at_6 / total_relevant)
            
            # Precision@1
            metrics['precision@1'].append(relevant_at_1 / 1 if len(retrieved_fragments) >= 1 else 0)
            
            # Precision@4
            metrics['precision@4'].append(relevant_at_4 / min(4, len(retrieved_fragments)))
            
            # Precision@6
            metrics['precision@6'].append(relevant_at_6 / min(6, len(retrieved_fragments)))
        else:
            # Если нет релевантных фрагментов, устанавливаем recall = 1.0 (все релевантные найдены)
            metrics['recall@1'].append(1.0)
            metrics['recall@4'].append(1.0)
            metrics['recall@6'].append(1.0)
            
            # Если нет релевантных фрагментов, устанавливаем precision = 0.0
            metrics['precision@1'].append(0.0)
            metrics['precision@4'].append(0.0)
            metrics['precision@6'].append(0.0)
        
        # MRR@4 (Mean Reciprocal Rank для первых 4)
        first_relevant_rank_at_4 = next((i + 1 for i, f in enumerate(retrieved_fragments[:min(4, len(retrieved_fragments))]) if f[3] >= 4), 0)
        if first_relevant_rank_at_4 > 0:
            metrics['mrr@4'].append(1.0 / first_relevant_rank_at_4)
        else:
            metrics['mrr@4'].append(0.0)
            
        # MRR@6 (Mean Reciprocal Rank для первых 6)
        first_relevant_rank_at_6 = next((i + 1 for i, f in enumerate(retrieved_fragments[:min(6, len(retrieved_fragments))]) if f[3] >= 4), 0)
        if first_relevant_rank_at_6 > 0:
            metrics['mrr@6'].append(1.0 / first_relevant_rank_at_6)
        else:
            metrics['mrr@6'].append(0.0)
        
        # nDCG@4
        if len(sorted_fragments) >= 1 and len(retrieved_fragments) >= 1:
            # Определяем количество документов для оценки
            k4 = min(4, len(sorted_fragments), len(retrieved_fragments))
            
            # Берем только первые k4 документа
            true_relevance_4 = np.array([f[3] for f in sorted_fragments[:k4]])
            predicted_order_relevance_4 = np.array([f[3] for f in retrieved_fragments[:k4]])
            
            try:
                ndcg_4 = ndcg_score([true_relevance_4], [predicted_order_relevance_4])
                metrics['ndcg@4'].append(ndcg_4)
            except Exception as e:
                print(f"Ошибка при вычислении nDCG@4: {e}")
                metrics['ndcg@4'].append(0.0)
        else:
            metrics['ndcg@4'].append(0.0)
            
        # nDCG@6
        if len(sorted_fragments) >= 1 and len(retrieved_fragments) >= 1:
            # Определяем количество документов для оценки
            k6 = min(6, len(sorted_fragments), len(retrieved_fragments))
            
            # Берем только первые k6 документов
            true_relevance_6 = np.array([f[3] for f in sorted_fragments[:k6]])
            predicted_order_relevance_6 = np.array([f[3] for f in retrieved_fragments[:k6]])
            
            try:
                ndcg_6 = ndcg_score([true_relevance_6], [predicted_order_relevance_6])
                metrics['ndcg@6'].append(ndcg_6)
            except Exception as e:
                print(f"Ошибка при вычислении nDCG@6: {e}")
                metrics['ndcg@6'].append(0.0)
        else:
            metrics['ndcg@6'].append(0.0)
    
    # Вычисляем средние значения метрик
    result_metrics = {}
    for key, values in metrics.items():
        result_metrics[key] = sum(values) / len(values) if values else 0.0
    
    return result_metrics

# Основной код для тестирования системы и вычисления метрик

def run_evaluation_with_reranking(api_key, test_questions):
    """Запуск оценки системы ретривала на основе E5 и реранкера на наборе тестовых вопросов"""
    # Инициализация системы на основе E5
    qa_system = DocumentationQA_E5Embeddings()
    qa_system.initialize_database()

    # Результаты для последующей оценки
    retrieval_results = []

    # Обработка каждого вопроса
    for question in test_questions:
        # Получение фрагментов с помощью E5 и реранкера
        fragments = get_relevant_fragments_e5_with_reranking(qa_system, question, top_k_initial=20, top_k_final=6)
        result = {'question': question, 'fragments': []}

        # Оценка релевантности для каждого фрагмента
        for text, name, score in fragments:
            time.sleep(2)  # Задержка между запросами
            relevance_score = evaluate_relevance_with_claude(question, text, api_key)
            result['fragments'].append((text, name, score, relevance_score))
        retrieval_results.append(result)

    # Вычисление метрик
    metrics_results = calculate_metrics(retrieval_results)
    return metrics_results, retrieval_results

def save_results_to_csv(results, filename):
    """Сохранение результатов в CSV файл"""
    rows = []
    for result in results:
        question = result['question']
        for text, name, e5_score, relevance in result['fragments']:
            rows.append({
                'question': question,
                'document': name,
                'e5_score': e5_score,
                'relevance_score': relevance,
                'text': text[:200]  # Ограничиваем длину текста для CSV
            })
    
    df = pd.DataFrame(rows)
    df.to_csv(filename, index=False)
    print(f"Результаты сохранены в {filename}")

def main():
    try:
        # Загрузка API ключа
        api_key_file = 'api.txt'
        if os.path.exists(api_key_file):
            with open(api_key_file, 'r') as file:
                api_key = file.read().strip()
        else:
            print(f"Файл с API ключом {api_key_file} не найден!")
            api_key = input("Введите ваш API ключ: ")

        # Проверка API ключа
        if not api_key:
            raise ValueError("API ключ не может быть пустым")

        df = pd.read_csv('texts_with_answers.csv')
        test_questions = df.question.to_list()

        # Запуск оценки
        print("Начало оценки E5 ретривала с реранкером BAAI/bge-reranker-base...")
        metrics, results = run_evaluation_with_reranking(api_key, test_questions)

        # Вывод результатов
        print("\nРезультаты оценки E5-ретривала с реранкером:")
        for metric, value in metrics.items():
            print(f"{metric}: {value:.4f}")

        # Детальный анализ результатов
        print("\nДетальный анализ результатов:")
        for result in results:
            question = result['question']
            print(f"\nВопрос: {question}")
            if not result['fragments']:
                print("Не найдено релевантных фрагментов для этого вопроса.")
                continue

            # Сортировка по оценке релевантности (от Claude)
            sorted_by_relevance = sorted(result['fragments'], key=lambda x: x[3], reverse=True)
            print("Топ-3 наиболее релевантных фрагмента по оценке Claude:")
            for i, (text, name, e5_score, relevance) in enumerate(sorted_by_relevance[:min(3, len(sorted_by_relevance))]):
                print(f"{i+1}. {name} (E5: {e5_score:.4f}, Релевантность: {relevance})")
                print(f"   {text[:100]}...")

            # Сортировка по E5 сходству
            sorted_by_e5 = sorted(result['fragments'], key=lambda x: x[2], reverse=True)
            print("\nТоп-3 наиболее релевантных фрагмента по E5:")
            for i, (text, name, e5_score, relevance) in enumerate(sorted_by_e5[:min(3, len(sorted_by_e5))]):
                print(f"{i+1}. {name} (E5: {e5_score:.4f}, Релевантность: {relevance})")
                print(f"   {text[:100]}...")

        # Сохранение результатов в CSV для дальнейшего анализа
        save_results_to_csv(results, "e5_bge_reranker_retrieval_results.csv")
        return metrics, results
    except Exception as e:
        print(f"Ошибка в функции main: {e}")
        import traceback
        traceback.print_exc()
        return None, None

df = pd.read_csv('texts_with_answers.csv')
questions = df['question'].tolist()

api_key_file = 'api.txt'
if os.path.exists(api_key_file):
    with open(api_key_file, 'r') as file:
        api_key = file.read().strip()
metrics, res = run_evaluation_with_reranking(api_key=api_key, test_questions=questions)

metrics

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\sekho\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\sekho\AppData\Roaming\nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\sekho\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     C:\Users\sekho\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_eng is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\Users\sekho\AppData\Roaming\nltk_data...
[

Загрузка моделей...
Модели успешно загружены
Всего загружено 130 фрагментов.
Генерация эмбеддингов для документов...


Batches:   0%|          | 0/5 [00:00<?, ?it/s]

Эмбеддинги успешно созданы. Размерность: (130, 1024)


{'recall@1': 0.44041666666666673,
 'recall@4': 0.8283333333333333,
 'recall@6': 1.0,
 'precision@1': 0.5083333333333333,
 'precision@4': 0.33541666666666664,
 'precision@6': 0.30277777777777787,
 'mrr@4': 0.6291666666666667,
 'mrr@6': 0.6369444444444444,
 'ndcg@4': np.float64(0.9480241434393737),
 'ndcg@6': np.float64(0.9257728989466522)}

Реранкер sentence-transformers/msmarco-distilbert-base-tas-b

In [17]:
import requests
import markdown
from bs4 import BeautifulSoup
import re
import numpy as np
import pandas as pd
import time
from sklearn.metrics import ndcg_score
import nltk
import string
import os
from sentence_transformers import CrossEncoder
from sklearn.metrics.pairwise import cosine_similarity

nltk.download('all')

# Явная установка директории для загрузки NLTK данных
nltk_data_dir = os.path.join(os.getcwd(), 'nltk_data')
os.makedirs(nltk_data_dir, exist_ok=True)
nltk.data.path.append(nltk_data_dir)

# Загрузка необходимых ресурсов NLTK с явным указанием пути
try:
    nltk.download('punkt', download_dir=nltk_data_dir, quiet=True)
    nltk.download('stopwords', download_dir=nltk_data_dir, quiet=True)
    from nltk.tokenize import word_tokenize
    from nltk.corpus import stopwords
    from nltk.stem import PorterStemmer
    stop_words = set(stopwords.words('english'))
except LookupError:
    print("Не удалось загрузить ресурсы NLTK. Используем упрощенную токенизацию.")
    
    # Упрощенная имплементация токенизации без зависимостей NLTK
    def word_tokenize(text):
        """Простая токенизация по пробелам и пунктуации"""
        # Заменяем пунктуацию на пробелы
        for punct in string.punctuation:
            text = text.replace(punct, ' ')
        # Разбиваем по пробелам и фильтруем пустые токены
        return [token for token in text.lower().split() if token]
    
    # Пустой набор стоп-слов
    stop_words = set()
    
    # Упрощенный стеммер
    class SimplePorterStemmer:
        """Очень упрощенная версия стеммера - убирает только окончения -ing, -ed, -s"""
        def stem(self, word):
            if word.endswith('ing'):
                return word[:-3]
            elif word.endswith('ed') and len(word) > 3:
                return word[:-2]
            elif word.endswith('s') and len(word) > 2:
                return word[:-1]
            return word
    
    PorterStemmer = SimplePorterStemmer

class DocumentationQA_E5Embeddings:
    def __init__(self):
        self.model = None
        self.reranker = None
        self.doc_paragraphs = []
        self.doc_embeddings = None
        self.md_list = [
            'https://raw.githubusercontent.com/qdrant/landing_page/master/qdrant-landing/content/documentation/concepts/collections.md',
            'https://raw.githubusercontent.com/qdrant/landing_page/master/qdrant-landing/content/documentation/concepts/explore.md',
            'https://raw.githubusercontent.com/qdrant/landing_page/master/qdrant-landing/content/documentation/concepts/filtering.md',
            'https://raw.githubusercontent.com/qdrant/landing_page/master/qdrant-landing/content/documentation/concepts/hybrid-queries.md',
            'https://raw.githubusercontent.com/qdrant/landing_page/master/qdrant-landing/content/documentation/concepts/indexing.md',
            'https://raw.githubusercontent.com/qdrant/landing_page/master/qdrant-landing/content/documentation/concepts/optimizer.md',
            'https://raw.githubusercontent.com/qdrant/landing_page/master/qdrant-landing/content/documentation/concepts/payload.md',
            'https://raw.githubusercontent.com/qdrant/landing_page/master/qdrant-landing/content/documentation/concepts/search.md',
            'https://raw.githubusercontent.com/qdrant/landing_page/master/qdrant-landing/content/documentation/concepts/points.md',
            'https://raw.githubusercontent.com/qdrant/landing_page/master/qdrant-landing/content/documentation/concepts/snapshots.md',
            'https://raw.githubusercontent.com/qdrant/landing_page/master/qdrant-landing/content/documentation/concepts/storage.md',
            'https://raw.githubusercontent.com/qdrant/landing_page/master/qdrant-landing/content/documentation/concepts/vectors.md'
        ]

    def extract_text_from_md(self, url, max_characters=1500, new_after_n_chars=1000, overlap=0):
        """Извлечение текста из Markdown файла и разбиение на параграфы"""
        try:
            response = requests.get(url)
            response.raise_for_status()
            html_content = markdown.markdown(response.text)
            soup = BeautifulSoup(html_content, features="html.parser")
            text = soup.get_text()
        except Exception as e:
            print(f"Ошибка при получении документа {url}: {e}")
            return []

        # Разделение на смысловые элементы
        raw_paragraphs = [p.strip() for p in re.split(r'\n\s*\n', text) if p.strip()]
        
        paragraphs = []
        current_chunk = ""
        
        for p in raw_paragraphs:
            # Нормализация пробелов
            cleaned_p = re.sub(r'\s+', ' ', p).strip()
            
            # Пропуск слишком коротких фрагментов
            if len(cleaned_p.split()) < 5:
                continue
                
            # Определение, является ли текущий параграф заголовком
            is_title = len(cleaned_p.split()) < 10 and not cleaned_p.endswith(('.', '?', '!'))
            
            # Если новый параграф - заголовок или текущий чанк станет слишком большим
            if is_title or len(current_chunk) + len(cleaned_p) > new_after_n_chars:
                # Сохранение предыдущего чанка, если он не пустой
                if current_chunk:
                    paragraphs.append(current_chunk)
                    current_chunk = ""
            
            # Если параграф слишком большой, разбиваем его на части
            if len(cleaned_p) > max_characters:
                # Разбиение на предложения
                sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?|\!)\s', cleaned_p)
                
                sentence_chunk = ""
                for sentence in sentences:
                    if len(sentence_chunk) + len(sentence) > max_characters:
                        paragraphs.append(sentence_chunk)
                        # Добавление перекрытия, если задано
                        if overlap > 0:
                            words = sentence_chunk.split()
                            overlap_text = ' '.join(words[-min(len(words), overlap//5):])
                            sentence_chunk = overlap_text + " " + sentence
                        else:
                            sentence_chunk = sentence
                    else:
                        sentence_chunk = (sentence_chunk + " " + sentence).strip() if sentence_chunk else sentence
                
                if sentence_chunk:
                    paragraphs.append(sentence_chunk)
            else:
                # Добавление параграфа к текущему чанку
                current_chunk = (current_chunk + "\n\n" + cleaned_p).strip() if current_chunk else cleaned_p
                
                # Если чанк превысил максимальный размер, сохраняем его
                if len(current_chunk) > max_characters:
                    paragraphs.append(current_chunk)
                    current_chunk = ""
        
        # Добавление последнего чанка, если он не пустой
        if current_chunk:
            paragraphs.append(current_chunk)
        
        return paragraphs

    def initialize_database(self):
        """Инициализация базы данных: загрузка и предобработка документов"""
        print("Загрузка моделей...")
        try:
            # Загрузка модели SentenceTransformer для начального поиска
            self.model = SentenceTransformer('BAAI/bge-large-en-v1.5')
            
            # Загрузка модели msmarco-distilbert-base-tas-b для реранкинга
            self.reranker = CrossEncoder('sentence-transformers/msmarco-distilbert-base-tas-b')
            print("Модели успешно загружены")
        except Exception as e:
            print(f"Ошибка при загрузке моделей: {e}")
            raise

        # Обработка всех документов
        self.doc_paragraphs = []
        for url in self.md_list:
            paragraphs = self.extract_text_from_md(url)
            name = url.split('concepts/')[1].split('.md')[0]
            if name == 'collections':
                paragraphs = [p for p in paragraphs if '/ Collections' not in p]
            else:
                paragraphs = [p for p in paragraphs if f'/{name}' not in p]
            for paragraph in paragraphs:
                self.doc_paragraphs.append({
                    'name': name,
                    'text': paragraph
                })
        print(f"Всего загружено {len(self.doc_paragraphs)} фрагментов.")
        if not self.doc_paragraphs:
            raise ValueError("Не удалось загрузить ни одного документа!")

        # Создание эмбеддингов для всех документов
        print("Генерация эмбеддингов для документов...")
        try:
            doc_texts = [f"passage: {doc['text']}" for doc in self.doc_paragraphs]
            self.doc_embeddings = self.model.encode(doc_texts, convert_to_numpy=True, show_progress_bar=True)
            print(f"Эмбеддинги успешно созданы. Размерность: {self.doc_embeddings.shape}")
        except Exception as e:
            print(f"Ошибка при создании эмбеддингов: {e}")
            raise

    def search_similar_paragraphs_with_reranking(self, user_query, top_k_initial=20, top_k_final=6):
        """Поиск похожих параграфов с использованием эмбеддингов E5 и реранкера msmarco-distilbert-base-tas-b"""
        if self.doc_embeddings is None or not self.model:
            print("Ошибка: База данных не инициализирована!")
            return []

        try:
            # Формируем запрос с инструкцией для E5
            query_text = f"query: {user_query}"
            query_embedding = self.model.encode([query_text], convert_to_numpy=True)[0]

            # Вычисляем косинусное сходство между запросом и всеми документами
            similarities = cosine_similarity([query_embedding], self.doc_embeddings)[0]
            top_indices = np.argsort(similarities)[::-1][:top_k_initial]

            # Получаем топ-20 фрагментов
            initial_results = [(self.doc_paragraphs[idx]['text'], self.doc_paragraphs[idx]['name'], float(similarities[idx]))
                               for idx in top_indices]

            # Подготовка данных для реранкера
            rerank_input = [[user_query, text] for text, _, _ in initial_results]
            rerank_scores = self.reranker.predict(rerank_input)

            # Комбинируем результаты и сортируем по оценкам реранкера
            combined_results = [(text, name, score) for (text, name, _), score in zip(initial_results, rerank_scores)]
            sorted_results = sorted(combined_results, key=lambda x: x[2], reverse=True)

            # Возвращаем топ-k фрагментов после реранкинга
            return sorted_results[:top_k_final]
        except Exception as e:
            print(f"Ошибка при поиске похожих параграфов: {e}")
            return []

# Функция для оценки релевантности с использованием API
def evaluate_relevance_with_claude(question, fragment_text, api_key):
    """Оценивает релевантность фрагмента к вопросу через API Claude"""
    url = "https://ask.chadgpt.ru/api/public/gpt-4o-mini"
    
    # Ограничиваем длину фрагмента для запроса
    max_fragment_length = 4000
    if len(fragment_text) > max_fragment_length:
        fragment_text = fragment_text[:max_fragment_length] + "..."
    
    prompt = f"""
    Задача: оценить релевантность текстового фрагмента вопросу.
    
    Вопрос: {question}
    
    Фрагмент: {fragment_text}
    
    Оцени релевантность фрагмента к вопросу по шкале от 1 до 5, где:
    1 - совершенно не релевантен
    2 - слабо релевантен
    3 - умеренно релевантен
    4 - очень релевантен
    5 - идеально релевантен
    
    Ответь только числом от 1 до 5 без пояснений.
    """
    
    # Формируем запрос согласно примеру
    request_json = {
        "message": prompt,
        "api_key": api_key
    }
    
    try:
        # Отправляем запрос и дожидаемся ответа
        response = requests.post(url=url, json=request_json)
        
        # Проверяем, отправился ли запрос
        if response.status_code != 200:
            print(f'Ошибка! Код http-ответа: {response.status_code}')
            return 1  # Возвращаем минимальную оценку в случае ошибки
        else:
            # Получаем текст ответа и преобразовываем в dict
            resp_json = response.json()
            
            # Если успешен ответ, то извлекаем результат
            if resp_json['is_success']:
                resp_msg = resp_json['response'].strip()
                # Ищем число от 1 до 5 в ответе
                import re
                score_match = re.search(r'[1-5]', resp_msg)
                if score_match:
                    relevance_score = int(score_match.group(0))
                    return relevance_score
                else:
                    print(f'Не удалось извлечь оценку из ответа: {resp_msg}')
                    return 3  # Средняя оценка по умолчанию в случае неоднозначного ответа
            else:
                error = resp_json['error_message']
                print(f'Ошибка: {error}')
                return 1  # Возвращаем минимальную оценку в случае ошибки
    except Exception as e:
        print(f'Исключение при обработке запроса: {str(e)}')
        return 1  # Возвращаем минимальную оценку в случае ошибки

def get_relevant_fragments_e5_with_reranking(qa_system, question, top_k_initial=20, top_k_final=6):
    """Получение релевантных фрагментов с использованием E5 эмбеддингов и реранкера"""
    fragments = qa_system.search_similar_paragraphs_with_reranking(question, top_k_initial=top_k_initial, top_k_final=top_k_final)
    return fragments

def calculate_metrics(retrieval_results):
    """Вычисление метрик эффективности ретривала"""
    metrics = {
        'recall@1': [],
        'recall@4': [],
        'recall@6': [],
        'precision@1': [],
        'precision@4': [],
        'precision@6': [],
        'mrr@4': [],
        'mrr@6': [],
        'ndcg@4': [],
        'ndcg@6': []
    }
    
    for result in retrieval_results:
        fragments = result['fragments']
        if not fragments:
            print(f"Предупреждение: для вопроса '{result['question']}' не найдено фрагментов")
            # Пропускаем вычисление метрик для этого запроса
            continue
            
        # Сортировка фрагментов по оценке релевантности от Claude (по убыванию)
        sorted_fragments = sorted(fragments, key=lambda x: x[3], reverse=True)
        
        # Сортировка фрагментов по скору из системы ретривала (по убыванию)
        retrieved_fragments = sorted(fragments, key=lambda x: x[2], reverse=True)
        
        # Вычисление Recall@k
        relevant_fragments = [f for f in sorted_fragments if f[3] >= 4]  # Считаем релевантными фрагменты с оценкой >= 4
        total_relevant = len(relevant_fragments)
        
        if total_relevant > 0:
            # Recall@1
            relevant_at_1 = sum(1 for f in retrieved_fragments[:1] if f[3] >= 4)
            metrics['recall@1'].append(relevant_at_1 / total_relevant)
            
            # Recall@4
            relevant_at_4 = sum(1 for f in retrieved_fragments[:4] if f[3] >= 4)
            metrics['recall@4'].append(relevant_at_4 / total_relevant)
            
            # Recall@6
            relevant_at_6 = sum(1 for f in retrieved_fragments[:min(6, len(retrieved_fragments))] if f[3] >= 4)
            metrics['recall@6'].append(relevant_at_6 / total_relevant)
            
            # Precision@1
            metrics['precision@1'].append(relevant_at_1 / 1 if len(retrieved_fragments) >= 1 else 0)
            
            # Precision@4
            metrics['precision@4'].append(relevant_at_4 / min(4, len(retrieved_fragments)))
            
            # Precision@6
            metrics['precision@6'].append(relevant_at_6 / min(6, len(retrieved_fragments)))
        else:
            # Если нет релевантных фрагментов, устанавливаем recall = 1.0 (все релевантные найдены)
            metrics['recall@1'].append(1.0)
            metrics['recall@4'].append(1.0)
            metrics['recall@6'].append(1.0)
            
            # Если нет релевантных фрагментов, устанавливаем precision = 0.0
            metrics['precision@1'].append(0.0)
            metrics['precision@4'].append(0.0)
            metrics['precision@6'].append(0.0)
        
        # MRR@4 (Mean Reciprocal Rank для первых 4)
        first_relevant_rank_at_4 = next((i + 1 for i, f in enumerate(retrieved_fragments[:min(4, len(retrieved_fragments))]) if f[3] >= 4), 0)
        if first_relevant_rank_at_4 > 0:
            metrics['mrr@4'].append(1.0 / first_relevant_rank_at_4)
        else:
            metrics['mrr@4'].append(0.0)
            
        # MRR@6 (Mean Reciprocal Rank для первых 6)
        first_relevant_rank_at_6 = next((i + 1 for i, f in enumerate(retrieved_fragments[:min(6, len(retrieved_fragments))]) if f[3] >= 4), 0)
        if first_relevant_rank_at_6 > 0:
            metrics['mrr@6'].append(1.0 / first_relevant_rank_at_6)
        else:
            metrics['mrr@6'].append(0.0)
        
        # nDCG@4
        if len(sorted_fragments) >= 1 and len(retrieved_fragments) >= 1:
            # Определяем количество документов для оценки
            k4 = min(4, len(sorted_fragments), len(retrieved_fragments))
            
            # Берем только первые k4 документа
            true_relevance_4 = np.array([f[3] for f in sorted_fragments[:k4]])
            predicted_order_relevance_4 = np.array([f[3] for f in retrieved_fragments[:k4]])
            
            try:
                ndcg_4 = ndcg_score([true_relevance_4], [predicted_order_relevance_4])
                metrics['ndcg@4'].append(ndcg_4)
            except Exception as e:
                print(f"Ошибка при вычислении nDCG@4: {e}")
                metrics['ndcg@4'].append(0.0)
        else:
            metrics['ndcg@4'].append(0.0)
            
        # nDCG@6
        if len(sorted_fragments) >= 1 and len(retrieved_fragments) >= 1:
            # Определяем количество документов для оценки
            k6 = min(6, len(sorted_fragments), len(retrieved_fragments))
            
            # Берем только первые k6 документов
            true_relevance_6 = np.array([f[3] for f in sorted_fragments[:k6]])
            predicted_order_relevance_6 = np.array([f[3] for f in retrieved_fragments[:k6]])
            
            try:
                ndcg_6 = ndcg_score([true_relevance_6], [predicted_order_relevance_6])
                metrics['ndcg@6'].append(ndcg_6)
            except Exception as e:
                print(f"Ошибка при вычислении nDCG@6: {e}")
                metrics['ndcg@6'].append(0.0)
        else:
            metrics['ndcg@6'].append(0.0)
    
    # Вычисляем средние значения метрик
    result_metrics = {}
    for key, values in metrics.items():
        result_metrics[key] = sum(values) / len(values) if values else 0.0
    
    return result_metrics

# Основной код для тестирования системы и вычисления метрик

def run_evaluation_with_reranking(api_key, test_questions):
    """Запуск оценки системы ретривала на основе E5 и реранкера на наборе тестовых вопросов"""
    # Инициализация системы на основе E5
    qa_system = DocumentationQA_E5Embeddings()
    qa_system.initialize_database()

    # Результаты для последующей оценки
    retrieval_results = []

    # Обработка каждого вопроса
    for question in test_questions:
        # Получение фрагментов с помощью E5 и реранкера
        fragments = get_relevant_fragments_e5_with_reranking(qa_system, question, top_k_initial=20, top_k_final=6)
        result = {'question': question, 'fragments': []}

        # Оценка релевантности для каждого фрагмента
        for text, name, score in fragments:
            time.sleep(2)  # Задержка между запросами
            relevance_score = evaluate_relevance_with_claude(question, text, api_key)
            result['fragments'].append((text, name, score, relevance_score))
        retrieval_results.append(result)

    # Вычисление метрик
    metrics_results = calculate_metrics(retrieval_results)
    return metrics_results, retrieval_results

def save_results_to_csv(results, filename):
    """Сохранение результатов в CSV файл"""
    rows = []
    for result in results:
        question = result['question']
        for text, name, e5_score, relevance in result['fragments']:
            rows.append({
                'question': question,
                'document': name,
                'e5_score': e5_score,
                'relevance_score': relevance,
                'text': text[:200]  # Ограничиваем длину текста для CSV
            })
    
    df = pd.DataFrame(rows)
    df.to_csv(filename, index=False)
    print(f"Результаты сохранены в {filename}")

def main():
    try:
        # Загрузка API ключа
        api_key_file = 'api.txt'
        if os.path.exists(api_key_file):
            with open(api_key_file, 'r') as file:
                api_key = file.read().strip()
        else:
            print(f"Файл с API ключом {api_key_file} не найден!")
            api_key = input("Введите ваш API ключ: ")

        # Проверка API ключа
        if not api_key:
            raise ValueError("API ключ не может быть пустым")

        df = pd.read_csv('texts_with_answers.csv')
        test_questions = df.question.to_list()

        # Запуск оценки
        print("Начало оценки E5 ретривала с реранкером msmarco-distilbert-base-tas-b...")
        metrics, results = run_evaluation_with_reranking(api_key, test_questions)

        # Вывод результатов
        print("\nРезультаты оценки E5-ретривала с реранкером:")
        for metric, value in metrics.items():
            print(f"{metric}: {value:.4f}")

        # Детальный анализ результатов
        print("\nДетальный анализ результатов:")
        for result in results:
            question = result['question']
            print(f"\nВопрос: {question}")
            if not result['fragments']:
                print("Не найдено релевантных фрагментов для этого вопроса.")
                continue

            # Сортировка по оценке релевантности (от Claude)
            sorted_by_relevance = sorted(result['fragments'], key=lambda x: x[3], reverse=True)
            print("Топ-3 наиболее релевантных фрагмента по оценке Claude:")
            for i, (text, name, e5_score, relevance) in enumerate(sorted_by_relevance[:min(3, len(sorted_by_relevance))]):
                print(f"{i+1}. {name} (E5: {e5_score:.4f}, Релевантность: {relevance})")
                print(f"   {text[:100]}...")

            # Сортировка по E5 сходству
            sorted_by_e5 = sorted(result['fragments'], key=lambda x: x[2], reverse=True)
            print("\nТоп-3 наиболее релевантных фрагмента по E5:")
            for i, (text, name, e5_score, relevance) in enumerate(sorted_by_e5[:min(3, len(sorted_by_e5))]):
                print(f"{i+1}. {name} (E5: {e5_score:.4f}, Релевантность: {relevance})")
                print(f"   {text[:100]}...")

        # Сохранение результатов в CSV для дальнейшего анализа
        save_results_to_csv(results, "e5_msmarco_distilbert_retrieval_results.csv")
        return metrics, results
    except Exception as e:
        print(f"Ошибка в функции main: {e}")
        import traceback
        traceback.print_exc()
        return None, None

df = pd.read_csv('texts_with_answers.csv')
questions = df['question'].tolist()

api_key_file = 'api.txt'
if os.path.exists(api_key_file):
    with open(api_key_file, 'r') as file:
        api_key = file.read().strip()
metrics, res = run_evaluation_with_reranking(api_key=api_key, test_questions=questions)

metrics

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\sekho\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\sekho\AppData\Roaming\nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\sekho\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     C:\Users\sekho\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_eng is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\Users\sekho\AppData\Roaming\nltk_data...
[

Загрузка моделей...


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at sentence-transformers/msmarco-distilbert-base-tas-b and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Модели успешно загружены
Всего загружено 130 фрагментов.
Генерация эмбеддингов для документов...


Batches:   0%|          | 0/5 [00:00<?, ?it/s]

Эмбеддинги успешно созданы. Размерность: (130, 1024)


{'recall@1': 0.46,
 'recall@4': 0.7545833333333332,
 'recall@6': 1.0,
 'precision@1': 0.21666666666666667,
 'precision@4': 0.18125,
 'precision@6': 0.19166666666666668,
 'mrr@4': 0.32152777777777775,
 'mrr@6': 0.34958333333333336,
 'ndcg@4': np.float64(0.9351913913241777),
 'ndcg@6': np.float64(0.8959125339131639)}

Теперь те же реранкеры, однако в качестве базовой модели возьмем BM25

In [14]:
import requests
import markdown
from bs4 import BeautifulSoup
import re
import numpy as np
import pandas as pd
import time
from sklearn.metrics import ndcg_score
from rank_bm25 import BM25Okapi
import nltk
import string
import os
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

nltk.download('all')

# Явная установка директории для загрузки NLTK данных
nltk_data_dir = os.path.join(os.getcwd(), 'nltk_data')
os.makedirs(nltk_data_dir, exist_ok=True)
nltk.data.path.append(nltk_data_dir)

# Загрузка необходимых ресурсов NLTK с явным указанием пути
try:
    nltk.download('punkt', download_dir=nltk_data_dir, quiet=True)
    nltk.download('stopwords', download_dir=nltk_data_dir, quiet=True)
    from nltk.tokenize import word_tokenize
    from nltk.corpus import stopwords
    from nltk.stem import PorterStemmer
    stop_words = set(stopwords.words('english'))
except LookupError:
    print("Не удалось загрузить ресурсы NLTK. Используем упрощенную токенизацию.")
    
    # Упрощенная имплементация токенизации без зависимостей NLTK
    def word_tokenize(text):
        """Простая токенизация по пробелам и пунктуации"""
        # Заменяем пунктуацию на пробелы
        for punct in string.punctuation:
            text = text.replace(punct, ' ')
        # Разбиваем по пробелам и фильтруем пустые токены
        return [token for token in text.lower().split() if token]
    
    # Пустой набор стоп-слов
    stop_words = set()
    
    # Упрощенный стеммер
    class SimplePorterStemmer:
        """Очень упрощенная версия стеммера - убирает только окончания -ing, -ed, -s"""
        def stem(self, word):
            if word.endswith('ing'):
                return word[:-3]
            elif word.endswith('ed') and len(word) > 3:
                return word[:-2]
            elif word.endswith('s') and len(word) > 2:
                return word[:-1]
            return word
    
    PorterStemmer = SimplePorterStemmer

# Класс для реранкинга с использованием модели TinyBERT
class TinyBertReranker:
    def __init__(self):
        try:
            print("Инициализация TinyBert реранкера...")
            self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            self.tokenizer = AutoTokenizer.from_pretrained("nboost/pt-tinybert-msmarco")
            self.model = AutoModelForSequenceClassification.from_pretrained("nboost/pt-tinybert-msmarco").to(self.device)
            self.model.eval()
            print(f"TinyBert реранкер успешно инициализирован (устройство: {self.device})")
        except Exception as e:
            print(f"Ошибка при инициализации TinyBert реранкера: {e}")
            raise

    def rerank(self, query, documents, fragment_scores, max_seq_length=512):
        """
        Переранжирует документы с использованием TinyBERT модели.
        
        Args:
            query: Текст запроса
            documents: Список текстов документов
            fragment_scores: Исходные оценки документов (из BM25)
            max_seq_length: Максимальная длина последовательности для токенизатора
            
        Returns:
            Список кортежей (документ, исходная_оценка, новая_оценка)
        """
        if not documents:
            return []

        # Подготовка входных данных
        inputs = []
        for doc in documents:
            # Обрезаем документы, чтобы избежать слишком длинных последовательностей
            tokenized = self.tokenizer.encode_plus(
                query, 
                doc, 
                add_special_tokens=True,
                max_length=max_seq_length,
                truncation=True,
                return_tensors="pt"
            )
            inputs.append(tokenized)

        # Получение предсказаний
        scores = []
        with torch.no_grad():
            for input_dict in inputs:
                input_dict = {k: v.to(self.device) for k, v in input_dict.items()}
                output = self.model(**input_dict)
                # Для моделей ранжирования обычно берем последний скор (релевантность)
                score = output.logits[0][1].item()  # Используем score релевантности
                scores.append(score)

        # Комбинирование результатов с исходными документами и оценками
        ranked_results = list(zip(documents, fragment_scores, scores))
        
        # Сортировка по убыванию нового скора
        ranked_results = sorted(ranked_results, key=lambda x: x[2], reverse=True)
        
        return ranked_results

class DocumentationQA_BM25:
    def __init__(self):
        self.bm25 = None
        self.doc_paragraphs = []
        self.tokenized_corpus = []
        self.md_list = [
            'https://raw.githubusercontent.com/qdrant/landing_page/master/qdrant-landing/content/documentation/concepts/collections.md',
            'https://raw.githubusercontent.com/qdrant/landing_page/master/qdrant-landing/content/documentation/concepts/explore.md',
            'https://raw.githubusercontent.com/qdrant/landing_page/master/qdrant-landing/content/documentation/concepts/filtering.md',
            'https://raw.githubusercontent.com/qdrant/landing_page/master/qdrant-landing/content/documentation/concepts/hybrid-queries.md',
            'https://raw.githubusercontent.com/qdrant/landing_page/master/qdrant-landing/content/documentation/concepts/indexing.md',
            'https://raw.githubusercontent.com/qdrant/landing_page/master/qdrant-landing/content/documentation/concepts/optimizer.md',
            'https://raw.githubusercontent.com/qdrant/landing_page/master/qdrant-landing/content/documentation/concepts/payload.md',
            'https://raw.githubusercontent.com/qdrant/landing_page/master/qdrant-landing/content/documentation/concepts/search.md',
            'https://raw.githubusercontent.com/qdrant/landing_page/master/qdrant-landing/content/documentation/concepts/points.md',
            'https://raw.githubusercontent.com/qdrant/landing_page/master/qdrant-landing/content/documentation/concepts/snapshots.md',
            'https://raw.githubusercontent.com/qdrant/landing_page/master/qdrant-landing/content/documentation/concepts/storage.md',
            'https://raw.githubusercontent.com/qdrant/landing_page/master/qdrant-landing/content/documentation/concepts/vectors.md'
        ]
        # Использование переменных из глобального контекста
        self.stop_words = stop_words
        self.stemmer = PorterStemmer()

    def preprocess_text(self, text):
        """Предобработка текста: токенизация, удаление стоп-слов и стемминг"""
        # Токенизация и приведение к нижнему регистру
        tokens = word_tokenize(text.lower())
        # Удаление пунктуации и цифр
        tokens = [token for token in tokens if token not in string.punctuation and not token.isdigit()]
        # Удаление стоп-слов
        tokens = [token for token in tokens if token not in self.stop_words]
        # Стемминг
        try:
            tokens = [self.stemmer.stem(token) for token in tokens]
        except Exception as e:
            print(f"Ошибка стемминга: {e}. Пропускаем этап стемминга.")
        return tokens

    def extract_text_from_md(self, url, max_characters=1500, new_after_n_chars=1000, overlap=0):
        """Извлечение текста из Markdown файла и разбиение на параграфы"""
        try:
            response = requests.get(url)
            response.raise_for_status()
            html_content = markdown.markdown(response.text)
            soup = BeautifulSoup(html_content, features="html.parser")
            text = soup.get_text()
        except Exception as e:
            print(f"Ошибка при получении документа {url}: {e}")
            return []

        # Разделение на смысловые элементы
        raw_paragraphs = [p.strip() for p in re.split(r'\n\s*\n', text) if p.strip()]
        
        paragraphs = []
        current_chunk = ""
        
        for p in raw_paragraphs:
            # Нормализация пробелов
            cleaned_p = re.sub(r'\s+', ' ', p).strip()
            
            # Пропуск слишком коротких фрагментов
            if len(cleaned_p.split()) < 5:
                continue
                
            # Определение, является ли текущий параграф заголовком
            is_title = len(cleaned_p.split()) < 10 and not cleaned_p.endswith(('.', '?', '!'))
            
            # Если новый параграф - заголовок или текущий чанк станет слишком большим
            if is_title or len(current_chunk) + len(cleaned_p) > new_after_n_chars:
                # Сохранение предыдущего чанка, если он не пустой
                if current_chunk:
                    paragraphs.append(current_chunk)
                    current_chunk = ""
            
            # Если параграф слишком большой, разбиваем его на части
            if len(cleaned_p) > max_characters:
                # Разбиение на предложения
                sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?|\!)\s', cleaned_p)
                
                sentence_chunk = ""
                for sentence in sentences:
                    if len(sentence_chunk) + len(sentence) > max_characters:
                        paragraphs.append(sentence_chunk)
                        # Добавление перекрытия, если задано
                        if overlap > 0:
                            words = sentence_chunk.split()
                            overlap_text = ' '.join(words[-min(len(words), overlap//5):])
                            sentence_chunk = overlap_text + " " + sentence
                        else:
                            sentence_chunk = sentence
                    else:
                        sentence_chunk = (sentence_chunk + " " + sentence).strip() if sentence_chunk else sentence
                
                if sentence_chunk:
                    paragraphs.append(sentence_chunk)
            else:
                # Добавление параграфа к текущему чанку
                current_chunk = (current_chunk + "\n\n" + cleaned_p).strip() if current_chunk else cleaned_p
                
                # Если чанк превысил максимальный размер, сохраняем его
                if len(current_chunk) > max_characters:
                    paragraphs.append(current_chunk)
                    current_chunk = ""
        
        # Добавление последнего чанка, если он не пустой
        if current_chunk:
            paragraphs.append(current_chunk)
        
        return paragraphs

    def initialize_database(self):
        """Инициализация базы данных: загрузка и предобработка документов"""
        # Обработка всех документов
        self.doc_paragraphs = []
        for url in self.md_list:
            paragraphs = self.extract_text_from_md(url)
            name = url.split('concepts/')[1].split('.md')[0]

            if name == 'collections':
                paragraphs = [p for p in paragraphs if '/ Collections' not in p]
            else:
                paragraphs = [p for p in paragraphs if f'/{name}' not in p]

            for paragraph in paragraphs:
                self.doc_paragraphs.append({
                    'name': name,
                    'text': paragraph
                })
        
        print(f"Всего загружено {len(self.doc_paragraphs)} фрагментов.")
        
        if not self.doc_paragraphs:
            raise ValueError("Не удалось загрузить ни одного документа!")
        
        # Токенизация и предобработка текстовых фрагментов для BM25
        print("Начинаем токенизацию и предобработку текста...")
        self.tokenized_corpus = [self.preprocess_text(doc['text']) for doc in self.doc_paragraphs]
        
        # Проверка на пустые токенизированные документы
        non_empty_docs = [(i, doc) for i, doc in enumerate(self.tokenized_corpus) if doc]
        if len(non_empty_docs) < len(self.tokenized_corpus):
            print(f"Внимание: {len(self.tokenized_corpus) - len(non_empty_docs)} документов не содержат токенов после предобработки.")
            
            # Отфильтровываем пустые документы
            valid_indices = [i for i, _ in non_empty_docs]
            self.tokenized_corpus = [self.tokenized_corpus[i] for i in valid_indices]
            self.doc_paragraphs = [self.doc_paragraphs[i] for i in valid_indices]
        
        # Инициализация BM25
        print("Инициализация BM25...")
        try:
            self.bm25 = BM25Okapi(self.tokenized_corpus)
            print(f"BM25 успешно инициализирован. Всего документов: {len(self.tokenized_corpus)}")
        except Exception as e:
            print(f"Ошибка при инициализации BM25: {e}")
            raise

    def search_similar_paragraphs(self, user_query, top_k=20):
        """Поиск похожих параграфов с использованием BM25"""
        if not self.bm25:
            print("Ошибка: BM25 не инициализирован!")
            return []
            
        # Предобработка запроса
        tokenized_query = self.preprocess_text(user_query)
        
        if not tokenized_query:
            print("Предупреждение: запрос не содержит значимых токенов после предобработки!")
            return []
                # Получение BM25 scores для всех документов
        try:
            scores = self.bm25.get_scores(tokenized_query)
        except Exception as e:
            print(f"Ошибка при получении оценок BM25: {e}")
            return []
        
        # Индексы документов с наивысшими оценками
        top_n = np.argsort(scores)[::-1][:top_k]
        
        # Возвращение текста, имени документа и оценки BM25
        results = []
        for i in top_n:
            if scores[i] > 0:  # Добавление только если оценка больше 0
                results.append((self.doc_paragraphs[i]['text'], self.doc_paragraphs[i]['name'], float(scores[i])))
        
        return results

# Функция для оценки релевантности с использованием API
def evaluate_relevance_with_claude(question, fragment_text, api_key):
    """Оценивает релевантность фрагмента к вопросу через API Claude"""
    url = "https://ask.chadgpt.ru/api/public/gpt-4o-mini"
    
    # Ограничиваем длину фрагмента для запроса
    max_fragment_length = 4000
    if len(fragment_text) > max_fragment_length:
        fragment_text = fragment_text[:max_fragment_length] + "..."
    
    prompt = f"""
    Задача: оценить релевантность текстового фрагмента вопросу.
    
    Вопрос: {question}
    
    Фрагмент: {fragment_text}
    
    Оцени релевантность фрагмента к вопросу по шкале от 1 до 5, где:
    1 - совершенно не релевантен
    2 - слабо релевантен
    3 - умеренно релевантен
    4 - очень релевантен
    5 - идеально релевантен
    
    Ответь только числом от 1 до 5 без пояснений.
    """
    
    # Формируем запрос согласно примеру
    request_json = {
        "message": prompt,
        "api_key": api_key
    }
    
    try:
        # Отправляем запрос и дожидаемся ответа
        response = requests.post(url=url, json=request_json)
        
        # Проверяем, отправился ли запрос
        if response.status_code != 200:
            print(f'Ошибка! Код http-ответа: {response.status_code}')
            return 1  # Возвращаем минимальную оценку в случае ошибки
        else:
            # Получаем текст ответа и преобразовываем в dict
            resp_json = response.json()
            
            # Если успешен ответ, то извлекаем результат
            if resp_json['is_success']:
                resp_msg = resp_json['response'].strip()
                # Ищем число от 1 до 5 в ответе
                import re
                score_match = re.search(r'[1-5]', resp_msg)
                if score_match:
                    relevance_score = int(score_match.group(0))
                    return relevance_score
                else:
                    print(f'Не удалось извлечь оценку из ответа: {resp_msg}')
                    return 3  # Средняя оценка по умолчанию в случае неоднозначного ответа
            else:
                error = resp_json['error_message']
                print(f'Ошибка: {error}')
                return 1  # Возвращаем минимальную оценку в случае ошибки
    except Exception as e:
        print(f'Исключение при обработке запроса: {str(e)}')
        return 1  # Возвращаем минимальную оценку в случае ошибки

def get_relevant_fragments_with_reranker(qa_system, reranker, question, initial_top_k=20, final_top_k=6):
    """
    Получение релевантных фрагментов с использованием BM25 и последующего реранкинга
    
    Args:
        qa_system: Экземпляр системы BM25
        reranker: Экземпляр реранкера
        question: Текст вопроса
        initial_top_k: Количество фрагментов, получаемых из BM25
        final_top_k: Количество фрагментов после реранкинга
    
    Returns:
        Список отсортированных по релевантности фрагментов
    """
    # Получаем исходные фрагменты с помощью BM25
    initial_fragments = qa_system.search_similar_paragraphs(question, top_k=initial_top_k)
    
    if not initial_fragments:
        print(f"Предупреждение: BM25 не нашел фрагментов для запроса '{question}'")
        return []
    
    # Разделяем фрагменты на составляющие для реранкера
    texts = [fragment[0] for fragment in initial_fragments]
    names = [fragment[1] for fragment in initial_fragments]
    scores = [fragment[2] for fragment in initial_fragments]
    
    # Применяем реранкер
    try:
        reranked_fragments = reranker.rerank(question, texts, scores)
        
        # Ограничиваем количество возвращаемых фрагментов
        reranked_fragments = reranked_fragments[:final_top_k]
        
        # Восстанавливаем формат результатов с именами документов
        result_fragments = [(text, names[texts.index(text)], orig_score, rerank_score) 
                           for text, orig_score, rerank_score in reranked_fragments]
        
        return result_fragments
    except Exception as e:
        print(f"Ошибка при реранкинге: {e}")
        # В случае ошибки возвращаем исходные результаты BM25
        return [(text, name, score, 0.0) for text, name, score in initial_fragments[:final_top_k]]

def calculate_metrics(retrieval_results, k_values=[4, 6]):
    """
    Вычисление метрик эффективности ретривала для разных значений k
    
    Args:
        retrieval_results: Результаты ретривала
        k_values: Список значений k для вычисления метрик
    
    Returns:
        Словарь метрик
    """
    metrics = {}
    
    # Инициализация метрик для всех значений k
    for k in k_values:
        metrics.update({
            f'recall@{k}': [],
            f'precision@{k}': [],
            f'mrr@{k}': [],
            f'ndcg@{k}': []
        })
    
    # Добавляем recall@1 и precision@1
    metrics['recall@1'] = []
    metrics['precision@1'] = []
    
    for result in retrieval_results:
        fragments = result['fragments']
        if not fragments:
            print(f"Предупреждение: для вопроса '{result['question']}' не найдено фрагментов")
            # Пропускаем вычисление метрик для этого запроса
            continue
            
        # Сортировка фрагментов по оценке релевантности от Claude (по убыванию)
        sorted_fragments = sorted(fragments, key=lambda x: x[3], reverse=True)
        
        # Сортировка фрагментов по скору из системы ретривала (по убыванию)
        retrieved_fragments = sorted(fragments, key=lambda x: x[2], reverse=True)
        
        # Вычисление Recall@k
        relevant_fragments = [f for f in sorted_fragments if f[3] >= 4]  # Считаем релевантными фрагменты с оценкой >= 4
        total_relevant = len(relevant_fragments)
        
        if total_relevant > 0:
            # Recall@1
            relevant_at_1 = sum(1 for f in retrieved_fragments[:1] if f[3] >= 4)
            metrics['recall@1'].append(relevant_at_1 / total_relevant)
            
            # Precision@1
            metrics['precision@1'].append(relevant_at_1 / 1 if len(retrieved_fragments) >= 1 else 0)
            
            # Для каждого значения k вычисляем метрики
            for k in k_values:
                # Recall@k
                relevant_at_k = sum(1 for f in retrieved_fragments[:min(k, len(retrieved_fragments))] if f[3] >= 4)
                metrics[f'recall@{k}'].append(relevant_at_k / total_relevant)
                
                # Precision@k
                metrics[f'precision@{k}'].append(relevant_at_k / min(k, len(retrieved_fragments)))
                
                # MRR@k (Mean Reciprocal Rank)
                first_relevant_rank = next((i + 1 for i, f in enumerate(retrieved_fragments[:min(k, len(retrieved_fragments))]) if f[3] >= 4), 0)
                if first_relevant_rank > 0:
                    metrics[f'mrr@{k}'].append(1.0 / first_relevant_rank)
                else:
                    metrics[f'mrr@{k}'].append(0.0)
                
                # nDCG@k
                if len(sorted_fragments) >= 1 and len(retrieved_fragments) >= 1:
                    # Определяем количество документов для оценки
                    k_actual = min(k, len(sorted_fragments), len(retrieved_fragments))
                    
                    # Берем только первые k_actual документов
                    true_relevance = np.array([f[3] for f in sorted_fragments[:k_actual]])
                    predicted_order_relevance = np.array([f[3] for f in retrieved_fragments[:k_actual]])
                    
                    try:
                        ndcg = ndcg_score([true_relevance], [predicted_order_relevance])
                        metrics[f'ndcg@{k}'].append(ndcg)
                    except Exception as e:
                        print(f"Ошибка при вычислении nDCG@{k}: {e}")
                        metrics[f'ndcg@{k}'].append(0.0)
                else:
                    metrics[f'ndcg@{k}'].append(0.0)
        else:
            # Если нет релевантных фрагментов
            metrics['recall@1'].append(1.0)
            metrics['precision@1'].append(0.0)
            
            for k in k_values:
                metrics[f'recall@{k}'].append(1.0)
                metrics[f'precision@{k}'].append(0.0)
                metrics[f'mrr@{k}'].append(0.0)
                metrics[f'ndcg@{k}'].append(0.0)
    
    # Вычисляем средние значения метрик
    result_metrics = {}
    for key, values in metrics.items():
        result_metrics[key] = sum(values) / len(values) if values else 0.0
    
    return result_metrics

def run_evaluation_with_reranker(api_key, test_questions, k_values=[4, 6]):
    """
    Запуск оценки системы с BM25 и реранкером на наборе тестовых вопросов
    
    Args:
        api_key: API ключ для оценки релевантности
        test_questions: Список тестовых вопросов
        k_values: Список значений k для оценки
    
    Returns:
        Метрики и результаты оценки
    """
    # Инициализация системы BM25
    qa_system = DocumentationQA_BM25()
    qa_system.initialize_database()
    
    # Инициализация реранкера
    reranker = TinyBertReranker()
    
    # Результаты для последующей оценки
    retrieval_results = []
    
    # Обработка каждого вопроса
    for question in test_questions:
        # Получение фрагментов с помощью BM25 и реранкера
        # Базовая модель отбирает 20 фрагментов, затем реранкер выбирает лучшие
        max_k = max(k_values)  # Максимальное k из запрошенных
        reranked_fragments = get_relevant_fragments_with_reranker(
            qa_system, reranker, question, initial_top_k=20, final_top_k=max_k
        )
        
        # Результаты для текущего вопроса
        result = {'question': question, 'fragments': []}
        
        # Оценка релевантности для каждого фрагмента
        for text, name, bm25_score, rerank_score in reranked_fragments:
            # Делаем задержку между запросами, чтобы не превысить лимиты API
            time.sleep(2)
            relevance_score = evaluate_relevance_with_claude(question, text, api_key)
            result['fragments'].append((text, name, bm25_score, relevance_score))
        
        retrieval_results.append(result)
    
    # Отдельные метрики для каждого значения k
    metrics_results = {}
    for k in k_values:
        # Для каждого k создаем копию результатов, но ограничиваем количество фрагментов до k
        k_results = []
        for result in retrieval_results:
            k_result = {
                'question': result['question'],
                'fragments': result['fragments'][:k] if result['fragments'] else []
            }
            k_results.append(k_result)
        
        # Вычисление метрик для текущего k
        k_metrics = calculate_metrics(k_results, [k])
        metrics_results[f'top_{k}'] = k_metrics
    
    # Объединение всех метрик
    combined_metrics = {}
    for k, metrics in metrics_results.items():
        for metric_name, value in metrics.items():
            combined_metrics[f"{k}_{metric_name}"] = value
    
    return combined_metrics, retrieval_results

def save_results_to_csv(results, filename):
    """Сохранение результатов в CSV файл"""
    rows = []
    for result in results:
        question = result['question']
        for text, name, bm25_score, relevance in result['fragments']:
            rows.append({
                'question': question,
                'document': name,
                'bm25_score': bm25_score,
                'relevance_score': relevance,
                'text': text[:200]  # Ограничиваем длину текста для CSV
            })
    
        df = pd.DataFrame(rows)
    df.to_csv(filename, index=False)
    print(f"Результаты сохранены в {filename}")

def main():
    try:
        # Загрузка API ключа
        api_key_file = 'api.txt'
        if os.path.exists(api_key_file):
            with open(api_key_file, 'r') as file:
                api_key = file.read().strip()
        else:
            print(f"Файл с API ключом {api_key_file} не найден!")
            api_key = input("Введите ваш API ключ: ")
        
        # Проверка API ключа
        if not api_key:
            raise ValueError("API ключ не может быть пустым")
        
        df = pd.read_csv('texts_with_answers.csv')
        test_questions = df.question.to_list()
        
        # Определяем значения k для оценки
        k_values = [4, 6]
        
        # Запуск оценки
        print("Начало оценки системы с BM25 и реранкером TinyBERT...")
        metrics, results = run_evaluation_with_reranker(api_key, test_questions, k_values)
        
        # Вывод результатов
        print("\nРезультаты оценки системы:")
        for metric, value in metrics.items():
            print(f"{metric}: {value:.4f}")
        
        # Детальный анализ результатов
        print("\nДетальный анализ результатов:")
        for result in results:
            question = result['question']
            print(f"\nВопрос: {question}")
            
            if not result['fragments']:
                print("Не найдено релевантных фрагментов для этого вопроса.")
                continue
                
            # Сортировка по оценке релевантности (от Claude)
            sorted_by_relevance = sorted(result['fragments'], key=lambda x: x[3], reverse=True)
            print("Топ-3 наиболее релевантных фрагмента по оценке Claude:")
            for i, (text, name, bm25_score, relevance) in enumerate(sorted_by_relevance[:min(3, len(sorted_by_relevance))]):
                print(f"{i+1}. {name} (BM25: {bm25_score:.4f}, Релевантность: {relevance})")
                print(f"   {text[:100]}...")
            
            # Сортировка по BM25
            sorted_by_bm25 = sorted(result['fragments'], key=lambda x: x[2], reverse=True)
            print("\nТоп-3 наиболее релевантных фрагмента по BM25:")
            for i, (text, name, bm25_score, relevance) in enumerate(sorted_by_bm25[:min(3, len(sorted_by_bm25))]):
                print(f"{i+1}. {name} (BM25: {bm25_score:.4f}, Релевантность: {relevance})")
                print(f"   {text[:100]}...")
        
        # Сохранение результатов в CSV для дальнейшего анализа
        save_results_to_csv(results, "reranker_retrieval_results.csv")
        
        # Проведем сравнение с исходной версией без реранкера
        print("\nНачало оценки только BM25 (без реранкера) для сравнения...")
        
        # Функция для оценки BM25 без реранкера
        def run_evaluation_bm25_only(api_key, test_questions, top_k):
            """Запуск оценки только BM25 системы на наборе тестовых вопросов"""
            # Инициализация системы BM25
            qa_system = DocumentationQA_BM25()
            qa_system.initialize_database()
            
            # Результаты для последующей оценки
            retrieval_results = []
            
            # Обработка каждого вопроса
            for question in test_questions:
                # Получение фрагментов с помощью BM25
                fragments = qa_system.search_similar_paragraphs(question, top_k=top_k)
                
                # Результаты для текущего вопроса
                result = {'question': question, 'fragments': []}
                
                # Оценка релевантности для каждого фрагмента
                for text, name, score in fragments:
                    # Используем сохраненные оценки релевантности, если есть
                    found = False
                    for r in results:
                        if r['question'] == question:
                            for t, n, _, rel_score in r['fragments']:
                                if t == text and n == name:
                                    result['fragments'].append((text, name, score, rel_score))
                                    found = True
                                    break
                        if found:
                            break
                    
                    # Если не нашли сохраненную оценку, запрашиваем новую
                    if not found:
                        time.sleep(2)
                        relevance_score = evaluate_relevance_with_claude(question, text, api_key)
                        result['fragments'].append((text, name, score, relevance_score))
                
                retrieval_results.append(result)
            
            # Вычисление метрик
            metrics_results = calculate_metrics(retrieval_results, k_values)
            
            return metrics_results, retrieval_results
        
        # Запускаем оценку BM25 отдельно для каждого значения k
        bm25_metrics = {}
        for k in k_values:
            print(f"Оценка BM25 для top_{k}...")
            k_metrics, k_results = run_evaluation_bm25_only(api_key, test_questions, k)
            
            # Добавляем префикс к метрикам
            for metric, value in k_metrics.items():
                if f"@{k}" in metric:  # Добавляем только метрики для текущего k
                    bm25_metrics[f"top_{k}_{metric}"] = value
            
            # Сохраняем результаты BM25
            save_results_to_csv(k_results, f"bm25_only_top{k}_results.csv")
        
        # Сравнение метрик
        print("\nСравнение метрик BM25 и BM25+TinyBERT:")
        print("{:<20} {:<15} {:<15}".format("Метрика", "BM25", "BM25+TinyBERT"))
        print("-" * 50)
        
        for k in k_values:
            for metric_name in [f"recall@{k}", f"precision@{k}", f"mrr@{k}", f"ndcg@{k}"]:
                bm25_key = f"top_{k}_{metric_name}"
                reranker_key = f"top_{k}_{metric_name}"
                
                if bm25_key in bm25_metrics and reranker_key in metrics:
                    bm25_value = bm25_metrics[bm25_key]
                    reranker_value = metrics[reranker_key]
                    
                    print("{:<20} {:<15.4f} {:<15.4f}".format(
                        bm25_key, bm25_value, reranker_value
                    ))
        
        # Анализ улучшений
        total_improvements = 0
        total_metrics = 0
        
        for k in k_values:
            for metric_name in [f"recall@{k}", f"precision@{k}", f"mrr@{k}", f"ndcg@{k}"]:
                bm25_key = f"top_{k}_{metric_name}"
                reranker_key = f"top_{k}_{metric_name}"
                
                if bm25_key in bm25_metrics and reranker_key in metrics:
                    bm25_value = bm25_metrics[bm25_key]
                    reranker_value = metrics[reranker_key]
                    
                    if reranker_value > bm25_value:
                        total_improvements += 1
                    
                    total_metrics += 1
        
        if total_metrics > 0:
            improvement_percentage = (total_improvements / total_metrics) * 100
            print(f"\nРеранкер улучшил {total_improvements} из {total_metrics} метрик ({improvement_percentage:.2f}%)")
        
        return metrics, results
    except Exception as e:
        print(f"Ошибка в функции main: {e}")
        import traceback
        traceback.print_exc()
        return None, None

if __name__ == "__main__":
    main()

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\sekho\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\sekho\AppData\Roaming\nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\sekho\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     C:\Users\sekho\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_eng is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\Users\sekho\AppData\Roaming\nltk_data...
[

Начало оценки системы с BM25 и реранкером TinyBERT...
Всего загружено 292 фрагментов.
Начинаем токенизацию и предобработку текста...
Внимание: 16 документов не содержат токенов после предобработки.
Инициализация BM25...
BM25 успешно инициализирован. Всего документов: 276
Инициализация TinyBert реранкера...
TinyBert реранкер успешно инициализирован (устройство: cpu)

Результаты оценки системы:
top_4_recall@4: 1.0000
top_4_precision@4: 0.4771
top_4_mrr@4: 0.7715
top_4_ndcg@4: 0.8638
top_4_recall@1: 0.4764
top_4_precision@1: 0.6917
top_6_recall@6: 1.0000
top_6_precision@6: 0.4222
top_6_mrr@6: 0.7739
top_6_ndcg@6: 0.8684
top_6_recall@1: 0.3728
top_6_precision@1: 0.6833

Детальный анализ результатов:

Вопрос: What is a collection in the context of Qdrant?
Топ-3 наиболее релевантных фрагмента по оценке Claude:
1. collections (BM25: 2.3204, Релевантность: 5)
   title: Collections weight: 30 aliases: - ../collections - /concepts/collections/ - /documentation/fr...
2. storage (BM25: 2.1274, Рел

In [1]:
import requests
import markdown
from bs4 import BeautifulSoup
import re
import numpy as np
import pandas as pd
import time
from sklearn.metrics import ndcg_score
from rank_bm25 import BM25Okapi
import nltk
import string
import os
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

nltk.download('all')

# Явная установка директории для загрузки NLTK данных
nltk_data_dir = os.path.join(os.getcwd(), 'nltk_data')
os.makedirs(nltk_data_dir, exist_ok=True)
nltk.data.path.append(nltk_data_dir)

# Загрузка необходимых ресурсов NLTK с явным указанием пути
try:
    nltk.download('punkt', download_dir=nltk_data_dir, quiet=True)
    nltk.download('stopwords', download_dir=nltk_data_dir, quiet=True)
    from nltk.tokenize import word_tokenize
    from nltk.corpus import stopwords
    from nltk.stem import PorterStemmer
    stop_words = set(stopwords.words('english'))
except LookupError:
    print("Не удалось загрузить ресурсы NLTK. Используем упрощенную токенизацию.")
    
    # Упрощенная имплементация токенизации без зависимостей NLTK
    def word_tokenize(text):
        """Простая токенизация по пробелам и пунктуации"""
        # Заменяем пунктуацию на пробелы
        for punct in string.punctuation:
            text = text.replace(punct, ' ')
        # Разбиваем по пробелам и фильтруем пустые токены
        return [token for token in text.lower().split() if token]
    
    # Пустой набор стоп-слов
    stop_words = set()
    
    # Упрощенный стеммер
    class SimplePorterStemmer:
        """Очень упрощенная версия стеммера - убирает только окончания -ing, -ed, -s"""
        def stem(self, word):
            if word.endswith('ing'):
                return word[:-3]
            elif word.endswith('ed') and len(word) > 3:
                return word[:-2]
            elif word.endswith('s') and len(word) > 2:
                return word[:-1]
            return word
    
    PorterStemmer = SimplePorterStemmer

# Класс для реранкинга с использованием модели BGE Reranker
class BGEReranker:
    def __init__(self):
        try:
            print("Инициализация BGE реранкера...")
            self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            self.tokenizer = AutoTokenizer.from_pretrained("BAAI/bge-reranker-base")
            self.model = AutoModelForSequenceClassification.from_pretrained("BAAI/bge-reranker-base").to(self.device)
            self.model.eval()
            print(f"BGE реранкер успешно инициализирован (устройство: {self.device})")
        except Exception as e:
            print(f"Ошибка при инициализации BGE реранкера: {e}")
            raise

    def rerank(self, query, documents, fragment_scores, max_seq_length=512):
        """
        Переранжирует документы с использованием BGE Reranker модели.
        
        Args:
            query: Текст запроса
            documents: Список текстов документов
            fragment_scores: Исходные оценки документов (из BM25)
            max_seq_length: Максимальная длина последовательности для токенизатора
            
        Returns:
            Список кортежей (документ, исходная_оценка, новая_оценка)
        """
        if not documents:
            return []

        # Подготовка входных данных
        inputs = []
        for doc in documents:
            # Обрезаем документы, чтобы избежать слишком длинных последовательностей
            tokenized = self.tokenizer.encode_plus(
                query, 
                doc, 
                add_special_tokens=True,
                max_length=max_seq_length,
                truncation=True,
                return_tensors="pt"
            )
            inputs.append(tokenized)

        # Получение предсказаний
        scores = []
        with torch.no_grad():
            for input_dict in inputs:
                input_dict = {k: v.to(self.device) for k, v in input_dict.items()}
                output = self.model(**input_dict)
                # BGE Reranker выдает скор релевантности как один скаляр
                score = output.logits[0].item()
                scores.append(score)

        # Комбинирование результатов с исходными документами и оценками
        ranked_results = list(zip(documents, fragment_scores, scores))
        
        # Сортировка по убыванию нового скора
        ranked_results = sorted(ranked_results, key=lambda x: x[2], reverse=True)
        
        return ranked_results

class DocumentationQA_BM25:
    def __init__(self):
        self.bm25 = None
        self.doc_paragraphs = []
        self.tokenized_corpus = []
        self.md_list = [
            'https://raw.githubusercontent.com/qdrant/landing_page/master/qdrant-landing/content/documentation/concepts/collections.md',
            'https://raw.githubusercontent.com/qdrant/landing_page/master/qdrant-landing/content/documentation/concepts/explore.md',
            'https://raw.githubusercontent.com/qdrant/landing_page/master/qdrant-landing/content/documentation/concepts/filtering.md',
            'https://raw.githubusercontent.com/qdrant/landing_page/master/qdrant-landing/content/documentation/concepts/hybrid-queries.md',
            'https://raw.githubusercontent.com/qdrant/landing_page/master/qdrant-landing/content/documentation/concepts/indexing.md',
            'https://raw.githubusercontent.com/qdrant/landing_page/master/qdrant-landing/content/documentation/concepts/optimizer.md',
            'https://raw.githubusercontent.com/qdrant/landing_page/master/qdrant-landing/content/documentation/concepts/payload.md',
            'https://raw.githubusercontent.com/qdrant/landing_page/master/qdrant-landing/content/documentation/concepts/search.md',
            'https://raw.githubusercontent.com/qdrant/landing_page/master/qdrant-landing/content/documentation/concepts/points.md',
            'https://raw.githubusercontent.com/qdrant/landing_page/master/qdrant-landing/content/documentation/concepts/snapshots.md',
            'https://raw.githubusercontent.com/qdrant/landing_page/master/qdrant-landing/content/documentation/concepts/storage.md',
            'https://raw.githubusercontent.com/qdrant/landing_page/master/qdrant-landing/content/documentation/concepts/vectors.md'
        ]
        # Использование переменных из глобального контекста
        self.stop_words = stop_words
        self.stemmer = PorterStemmer()

    def preprocess_text(self, text):
        """Предобработка текста: токенизация, удаление стоп-слов и стемминг"""
        # Токенизация и приведение к нижнему регистру
        tokens = word_tokenize(text.lower())
        # Удаление пунктуации и цифр
        tokens = [token for token in tokens if token not in string.punctuation and not token.isdigit()]
        # Удаление стоп-слов
        tokens = [token for token in tokens if token not in self.stop_words]
        # Стемминг
        try:
            tokens = [self.stemmer.stem(token) for token in tokens]
        except Exception as e:
            print(f"Ошибка стемминга: {e}. Пропускаем этап стемминга.")
        return tokens

    def extract_text_from_md(self, url, max_characters=1500, new_after_n_chars=1000, overlap=0):
        """Извлечение текста из Markdown файла и разбиение на параграфы"""
        try:
            response = requests.get(url)
            response.raise_for_status()
            html_content = markdown.markdown(response.text)
            soup = BeautifulSoup(html_content, features="html.parser")
            text = soup.get_text()
        except Exception as e:
            print(f"Ошибка при получении документа {url}: {e}")
            return []

        # Разделение на смысловые элементы
        raw_paragraphs = [p.strip() for p in re.split(r'\n\s*\n', text) if p.strip()]
        
        paragraphs = []
        current_chunk = ""
        
        for p in raw_paragraphs:
            # Нормализация пробелов
            cleaned_p = re.sub(r'\s+', ' ', p).strip()
            
            # Пропуск слишком коротких фрагментов
            if len(cleaned_p.split()) < 5:
                continue
                
            # Определение, является ли текущий параграф заголовком
            is_title = len(cleaned_p.split()) < 10 and not cleaned_p.endswith(('.', '?', '!'))
            
            # Если новый параграф - заголовок или текущий чанк станет слишком большим
            if is_title or len(current_chunk) + len(cleaned_p) > new_after_n_chars:
                # Сохранение предыдущего чанка, если он не пустой
                if current_chunk:
                    paragraphs.append(current_chunk)
                    current_chunk = ""
            
            # Если параграф слишком большой, разбиваем его на части
            if len(cleaned_p) > max_characters:
                # Разбиение на предложения
                sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?|\!)\s', cleaned_p)
                
                sentence_chunk = ""
                for sentence in sentences:
                    if len(sentence_chunk) + len(sentence) > max_characters:
                        paragraphs.append(sentence_chunk)
                        # Добавление перекрытия, если задано
                        if overlap > 0:
                            words = sentence_chunk.split()
                            overlap_text = ' '.join(words[-min(len(words), overlap//5):])
                            sentence_chunk = overlap_text + " " + sentence
                        else:
                            sentence_chunk = sentence
                    else:
                        sentence_chunk = (sentence_chunk + " " + sentence).strip() if sentence_chunk else sentence
                
                if sentence_chunk:
                    paragraphs.append(sentence_chunk)
            else:
                # Добавление параграфа к текущему чанку
                current_chunk = (current_chunk + "\n\n" + cleaned_p).strip() if current_chunk else cleaned_p
                
                # Если чанк превысил максимальный размер, сохраняем его
                if len(current_chunk) > max_characters:
                    paragraphs.append(current_chunk)
                    current_chunk = ""
        
        # Добавление последнего чанка, если он не пустой
        if current_chunk:
            paragraphs.append(current_chunk)
        
        return paragraphs

    def initialize_database(self):
        """Инициализация базы данных: загрузка и предобработка документов"""
        # Обработка всех документов
        self.doc_paragraphs = []
        for url in self.md_list:
            paragraphs = self.extract_text_from_md(url)
            name = url.split('concepts/')[1].split('.md')[0]

            if name == 'collections':
                paragraphs = [p for p in paragraphs if '/ Collections' not in p]
            else:
                paragraphs = [p for p in paragraphs if f'/{name}' not in p]

            for paragraph in paragraphs:
                self.doc_paragraphs.append({
                    'name': name,
                    'text': paragraph
                })
        
        print(f"Всего загружено {len(self.doc_paragraphs)} фрагментов.")
        
        if not self.doc_paragraphs:
            raise ValueError("Не удалось загрузить ни одного документа!")
        
        # Токенизация и предобработка текстовых фрагментов для BM25
        print("Начинаем токенизацию и предобработку текста...")
        self.tokenized_corpus = [self.preprocess_text(doc['text']) for doc in self.doc_paragraphs]
        
        # Проверка на пустые токенизированные документы
        non_empty_docs = [(i, doc) for i, doc in enumerate(self.tokenized_corpus) if doc]
        if len(non_empty_docs) < len(self.tokenized_corpus):
            print(f"Внимание: {len(self.tokenized_corpus) - len(non_empty_docs)} документов не содержат токенов после предобработки.")
            
            # Отфильтровываем пустые документы
            valid_indices = [i for i, _ in non_empty_docs]
            self.tokenized_corpus = [self.tokenized_corpus[i] for i in valid_indices]
            self.doc_paragraphs = [self.doc_paragraphs[i] for i in valid_indices]
        
        # Инициализация BM25
        print("Инициализация BM25...")
        try:
            self.bm25 = BM25Okapi(self.tokenized_corpus)
            print(f"BM25 успешно инициализирован. Всего документов: {len(self.tokenized_corpus)}")
        except Exception as e:
            print(f"Ошибка при инициализации BM25: {e}")
            raise

    def search_similar_paragraphs(self, user_query, top_k=20):
        """Поиск похожих параграфов с использованием BM25"""
        if not self.bm25:
            print("Ошибка: BM25 не инициализирован!")
            return []
            
        # Предобработка запроса
        tokenized_query = self.preprocess_text(user_query)
        
        if not tokenized_query:
            print("Предупреждение: запрос не содержит значимых токенов после предобработки!")
            return []
                # Получение BM25 scores для всех документов
        try:
            scores = self.bm25.get_scores(tokenized_query)
        except Exception as e:
            print(f"Ошибка при получении оценок BM25: {e}")
            return []
        
        # Индексы документов с наивысшими оценками
        top_n = np.argsort(scores)[::-1][:top_k]
        
        # Возвращение текста, имени документа и оценки BM25
        results = []
        for i in top_n:
            if scores[i] > 0:  # Добавление только если оценка больше 0
                results.append((self.doc_paragraphs[i]['text'], self.doc_paragraphs[i]['name'], float(scores[i])))
        
        return results

# Функция для оценки релевантности с использованием API
def evaluate_relevance_with_claude(question, fragment_text, api_key):
    """Оценивает релевантность фрагмента к вопросу через API Claude"""
    url = "https://ask.chadgpt.ru/api/public/gpt-4o-mini"
    
    # Ограничиваем длину фрагмента для запроса
    max_fragment_length = 4000
    if len(fragment_text) > max_fragment_length:
        fragment_text = fragment_text[:max_fragment_length] + "..."
    
    prompt = f"""
    Задача: оценить релевантность текстового фрагмента вопросу.
    
    Вопрос: {question}
    
    Фрагмент: {fragment_text}
    
    Оцени релевантность фрагмента к вопросу по шкале от 1 до 5, где:
    1 - совершенно не релевантен
    2 - слабо релевантен
    3 - умеренно релевантен
    4 - очень релевантен
    5 - идеально релевантен
    
    Ответь только числом от 1 до 5 без пояснений.
    """
    
    # Формируем запрос согласно примеру
    request_json = {
        "message": prompt,
        "api_key": api_key
    }
    
    try:
        # Отправляем запрос и дожидаемся ответа
        response = requests.post(url=url, json=request_json)
        
        # Проверяем, отправился ли запрос
        if response.status_code != 200:
            print(f'Ошибка! Код http-ответа: {response.status_code}')
            return 1  # Возвращаем минимальную оценку в случае ошибки
        else:
            # Получаем текст ответа и преобразовываем в dict
            resp_json = response.json()
            
            # Если успешен ответ, то извлекаем результат
            if resp_json['is_success']:
                resp_msg = resp_json['response'].strip()
                # Ищем число от 1 до 5 в ответе
                import re
                score_match = re.search(r'[1-5]', resp_msg)
                if score_match:
                    relevance_score = int(score_match.group(0))
                    return relevance_score
                else:
                    print(f'Не удалось извлечь оценку из ответа: {resp_msg}')
                    return 3  # Средняя оценка по умолчанию в случае неоднозначного ответа
            else:
                error = resp_json['error_message']
                print(f'Ошибка: {error}')
                return 1  # Возвращаем минимальную оценку в случае ошибки
    except Exception as e:
        print(f'Исключение при обработке запроса: {str(e)}')
        return 1  # Возвращаем минимальную оценку в случае ошибки

def get_relevant_fragments_with_reranker(qa_system, reranker, question, initial_top_k=20, final_top_k=6):
    """
    Получение релевантных фрагментов с использованием BM25 и последующего реранкинга
    
    Args:
        qa_system: Экземпляр системы BM25
        reranker: Экземпляр реранкера
        question: Текст вопроса
        initial_top_k: Количество фрагментов, получаемых из BM25
        final_top_k: Количество фрагментов после реранкинга
    
    Returns:
        Список отсортированных по релевантности фрагментов
    """
    # Получаем исходные фрагменты с помощью BM25
    initial_fragments = qa_system.search_similar_paragraphs(question, top_k=initial_top_k)
    
    if not initial_fragments:
        print(f"Предупреждение: BM25 не нашел фрагментов для запроса '{question}'")
        return []
    
    # Разделяем фрагменты на составляющие для реранкера
    texts = [fragment[0] for fragment in initial_fragments]
    names = [fragment[1] for fragment in initial_fragments]
    scores = [fragment[2] for fragment in initial_fragments]
    
    # Применяем реранкер
    try:
        reranked_fragments = reranker.rerank(question, texts, scores)
        
        # Ограничиваем количество возвращаемых фрагментов
        reranked_fragments = reranked_fragments[:final_top_k]
        
        # Восстанавливаем формат результатов с именами документов
        result_fragments = [(text, names[texts.index(text)], orig_score, rerank_score) 
                           for text, orig_score, rerank_score in reranked_fragments]
        
        return result_fragments
    except Exception as e:
        print(f"Ошибка при реранкинге: {e}")
        # В случае ошибки возвращаем исходные результаты BM25
        return [(text, name, score, 0.0) for text, name, score in initial_fragments[:final_top_k]]

def calculate_metrics(retrieval_results, k_values=[4, 6]):
    """
    Вычисление метрик эффективности ретривала для разных значений k
    
    Args:
        retrieval_results: Результаты ретривала
        k_values: Список значений k для вычисления метрик
    
    Returns:
        Словарь метрик
    """
    metrics = {}
    
    # Инициализация метрик для всех значений k
    for k in k_values:
        metrics.update({
            f'recall@{k}': [],
            f'precision@{k}': [],
            f'mrr@{k}': [],
            f'ndcg@{k}': []
        })
    
    # Добавляем recall@1 и precision@1
    metrics['recall@1'] = []
    metrics['precision@1'] = []
    
    for result in retrieval_results:
        fragments = result['fragments']
        if not fragments:
            print(f"Предупреждение: для вопроса '{result['question']}' не найдено фрагментов")
            # Пропускаем вычисление метрик для этого запроса
            continue
            
        # Сортировка фрагментов по оценке релевантности от Claude (по убыванию)
        sorted_fragments = sorted(fragments, key=lambda x: x[3], reverse=True)
        
        # Сортировка фрагментов по скору из системы ретривала (по убыванию)
        retrieved_fragments = sorted(fragments, key=lambda x: x[2], reverse=True)
        
        # Вычисление Recall@k
        relevant_fragments = [f for f in sorted_fragments if f[3] >= 4]  # Считаем релевантными фрагменты с оценкой >= 4
        total_relevant = len(relevant_fragments)
        
        if total_relevant > 0:
            # Recall@1
            relevant_at_1 = sum(1 for f in retrieved_fragments[:1] if f[3] >= 4)
            metrics['recall@1'].append(relevant_at_1 / total_relevant)
            
            # Precision@1
            metrics['precision@1'].append(relevant_at_1 / 1 if len(retrieved_fragments) >= 1 else 0)
            
            # Для каждого значения k вычисляем метрики
            for k in k_values:
                # Recall@k
                relevant_at_k = sum(1 for f in retrieved_fragments[:min(k, len(retrieved_fragments))] if f[3] >= 4)
                metrics[f'recall@{k}'].append(relevant_at_k / total_relevant)
                
                # Precision@k
                metrics[f'precision@{k}'].append(relevant_at_k / min(k, len(retrieved_fragments)))
                
                # MRR@k (Mean Reciprocal Rank)
                first_relevant_rank = next((i + 1 for i, f in enumerate(retrieved_fragments[:min(k, len(retrieved_fragments))]) if f[3] >= 4), 0)
                if first_relevant_rank > 0:
                    metrics[f'mrr@{k}'].append(1.0 / first_relevant_rank)
                else:
                    metrics[f'mrr@{k}'].append(0.0)
                
                # nDCG@k
                if len(sorted_fragments) >= 1 and len(retrieved_fragments) >= 1:
                    # Определяем количество документов для оценки
                    k_actual = min(k, len(sorted_fragments), len(retrieved_fragments))
                    
                    # Берем только первые k_actual документов
                    true_relevance = np.array([f[3] for f in sorted_fragments[:k_actual]])
                    predicted_order_relevance = np.array([f[3] for f in retrieved_fragments[:k_actual]])
                    
                    try:
                        ndcg = ndcg_score([true_relevance], [predicted_order_relevance])
                        metrics[f'ndcg@{k}'].append(ndcg)
                    except Exception as e:
                        print(f"Ошибка при вычислении nDCG@{k}: {e}")
                        metrics[f'ndcg@{k}'].append(0.0)
                else:
                    metrics[f'ndcg@{k}'].append(0.0)
        else:
            # Если нет релевантных фрагментов
            metrics['recall@1'].append(1.0)
            metrics['precision@1'].append(0.0)
            
            for k in k_values:
                metrics[f'recall@{k}'].append(1.0)
                metrics[f'precision@{k}'].append(0.0)
                metrics[f'mrr@{k}'].append(0.0)
                metrics[f'ndcg@{k}'].append(0.0)
    
    # Вычисляем средние значения метрик
    result_metrics = {}
    for key, values in metrics.items():
        result_metrics[key] = sum(values) / len(values) if values else 0.0
    
    return result_metrics

def run_evaluation_with_reranker(api_key, test_questions, k_values=[4, 6]):
    """
    Запуск оценки системы с BM25 и реранкером на наборе тестовых вопросов
    
    Args:
        api_key: API ключ для оценки релевантности
        test_questions: Список тестовых вопросов
        k_values: Список значений k для оценки
    
    Returns:
        Метрики и результаты оценки
    """
    # Инициализация системы BM25
    qa_system = DocumentationQA_BM25()
    qa_system.initialize_database()
    
    # Инициализация реранкера
    reranker = TinyBertReranker()
    
    # Результаты для последующей оценки
    retrieval_results = []
    
    # Обработка каждого вопроса
    for question in test_questions:
        # Получение фрагментов с помощью BM25 и реранкера
        # Базовая модель отбирает 20 фрагментов, затем реранкер выбирает лучшие
        max_k = max(k_values)  # Максимальное k из запрошенных
        reranked_fragments = get_relevant_fragments_with_reranker(
            qa_system, reranker, question, initial_top_k=20, final_top_k=max_k
        )
        
        # Результаты для текущего вопроса
        result = {'question': question, 'fragments': []}
        
        # Оценка релевантности для каждого фрагмента
        for text, name, bm25_score, rerank_score in reranked_fragments:
            # Делаем задержку между запросами, чтобы не превысить лимиты API
            time.sleep(2)
            relevance_score = evaluate_relevance_with_claude(question, text, api_key)
            result['fragments'].append((text, name, bm25_score, relevance_score))
        
        retrieval_results.append(result)
    
    # Отдельные метрики для каждого значения k
    metrics_results = {}
    for k in k_values:
        # Для каждого k создаем копию результатов, но ограничиваем количество фрагментов до k
        k_results = []
        for result in retrieval_results:
            k_result = {
                'question': result['question'],
                'fragments': result['fragments'][:k] if result['fragments'] else []
            }
            k_results.append(k_result)
        
        # Вычисление метрик для текущего k
        k_metrics = calculate_metrics(k_results, [k])
        metrics_results[f'top_{k}'] = k_metrics
    
    # Объединение всех метрик
    combined_metrics = {}
    for k, metrics in metrics_results.items():
        for metric_name, value in metrics.items():
            combined_metrics[f"{k}_{metric_name}"] = value
    
    return combined_metrics, retrieval_results

def save_results_to_csv(results, filename):
    """Сохранение результатов в CSV файл"""
    rows = []
    for result in results:
        question = result['question']
        for text, name, bm25_score, relevance in result['fragments']:
            rows.append({
                'question': question,
                'document': name,
                'bm25_score': bm25_score,
                'relevance_score': relevance,
                'text': text[:200]  # Ограничиваем длину текста для CSV
            })
    
        df = pd.DataFrame(rows)
    df.to_csv(filename, index=False)
    print(f"Результаты сохранены в {filename}")

def main():
    try:
        # Загрузка API ключа
        api_key_file = 'api.txt'
        if os.path.exists(api_key_file):
            with open(api_key_file, 'r') as file:
                api_key = file.read().strip()
        else:
            print(f"Файл с API ключом {api_key_file} не найден!")
            api_key = input("Введите ваш API ключ: ")
        
        # Проверка API ключа
        if not api_key:
            raise ValueError("API ключ не может быть пустым")
        
        df = pd.read_csv('texts_with_answers.csv')
        test_questions = df.question.to_list()
        
        # Определяем значения k для оценки
        k_values = [4, 6]
        
        # Запуск оценки системы с BGE реранкером
        print("Начало оценки системы с BM25 и реранкером BGE...")
        
        # Инициализация QA системы с BM25
        qa_system = DocumentationQA_BM25()
        qa_system.initialize_database()
        
        # Инициализация BGE реранкера
        reranker = BGEReranker()
        
        # Результаты для последующей оценки
        retrieval_results = []
        
        # Обработка каждого вопроса
        for question in test_questions:
            # Получение фрагментов с помощью BM25 и реранкера
            # Базовая модель отбирает 20 фрагментов, затем реранкер выбирает лучшие
            max_k = max(k_values)  # Максимальное k из запрошенных
            reranked_fragments = get_relevant_fragments_with_reranker(
                qa_system, reranker, question, initial_top_k=20, final_top_k=max_k
            )
            
            # Результаты для текущего вопроса
            result = {'question': question, 'fragments': []}
            
            # Оценка релевантности для каждого фрагмента
            for text, name, bm25_score, rerank_score in reranked_fragments:
                # Делаем задержку между запросами, чтобы не превысить лимиты API
                time.sleep(2)
                relevance_score = evaluate_relevance_with_claude(question, text, api_key)
                result['fragments'].append((text, name, bm25_score, relevance_score))
            
            retrieval_results.append(result)
        
        # Отдельные метрики для каждого значения k
        metrics_results = {}
        for k in k_values:
            # Для каждого k создаем копию результатов, но ограничиваем количество фрагментов до k
            k_results = []
            for result in retrieval_results:
                k_result = {
                    'question': result['question'],
                    'fragments': result['fragments'][:k] if result['fragments'] else []
                }
                k_results.append(k_result)
            
            # Вычисление метрик для текущего k
            k_metrics = calculate_metrics(k_results, [k])
            metrics_results[f'top_{k}'] = k_metrics
        
        # Объединение всех метрик
        metrics = {}
        for k, k_metrics in metrics_results.items():
            for metric_name, value in k_metrics.items():
                metrics[f"{k}_{metric_name}"] = value
        
        # Вывод результатов
        print("\nРезультаты оценки системы с BGE реранкером:")
        for metric, value in metrics.items():
            print(f"{metric}: {value:.4f}")
        
        # Детальный анализ результатов
        print("\nДетальный анализ результатов:")
        for result in retrieval_results:
            question = result['question']
            print(f"\nВопрос: {question}")
            
            if not result['fragments']:
                print("Не найдено релевантных фрагментов для этого вопроса.")
                continue
                
            # Сортировка по оценке релевантности (от Claude)
            sorted_by_relevance = sorted(result['fragments'], key=lambda x: x[3], reverse=True)
            print("Топ-3 наиболее релевантных фрагмента по оценке Claude:")
            for i, (text, name, bm25_score, relevance) in enumerate(sorted_by_relevance[:min(3, len(sorted_by_relevance))]):
                print(f"{i+1}. {name} (BM25: {bm25_score:.4f}, Релевантность: {relevance})")
                print(f"   {text[:100]}...")
            
            # Сортировка по BM25
            sorted_by_bm25 = sorted(result['fragments'], key=lambda x: x[2], reverse=True)
            print("\nТоп-3 наиболее релевантных фрагмента по BM25:")
            for i, (text, name, bm25_score, relevance) in enumerate(sorted_by_bm25[:min(3, len(sorted_by_bm25))]):
                print(f"{i+1}. {name} (BM25: {bm25_score:.4f}, Релевантность: {relevance})")
                print(f"   {text[:100]}...")
        
        # Сохранение результатов в CSV для дальнейшего анализа
        save_results_to_csv(retrieval_results, "bge_reranker_results.csv")
        
        # Проведем сравнение с исходной версией без реранкера
        print("\nНачало оценки только BM25 (без реранкера) для сравнения...")
        
        # Функция для оценки BM25 без реранкера
        def run_evaluation_bm25_only(api_key, test_questions, top_k):
            """Запуск оценки только BM25 системы на наборе тестовых вопросов"""
            # Инициализация системы BM25
            qa_system = DocumentationQA_BM25()
            qa_system.initialize_database()
            
            # Результаты для последующей оценки
            retrieval_results = []
            
            # Обработка каждого вопроса
            for question in test_questions:
                # Получение фрагментов с помощью BM25
                fragments = qa_system.search_similar_paragraphs(question, top_k=top_k)
                
                # Результаты для текущего вопроса
                result = {'question': question, 'fragments': []}
                
                # Оценка релевантности для каждого фрагмента
                for text, name, score in fragments:
                    # Используем сохраненные оценки релевантности, если есть
                    found = False
                    for r in retrieval_results:
                        if r['question'] == question:
                            for t, n, _, rel_score in r['fragments']:
                                if t == text and n == name:
                                    result['fragments'].append((text, name, score, rel_score))
                                    found = True
                                    break
                        if found:
                            break
                    
                    # Если не нашли сохраненную оценку, запрашиваем новую
                    if not found:
                        time.sleep(2)
                        relevance_score = evaluate_relevance_with_claude(question, text, api_key)
                        result['fragments'].append((text, name, score, relevance_score))
                
                retrieval_results.append(result)
            
            # Вычисление метрик
            metrics_results = calculate_metrics(retrieval_results, k_values)
            
            return metrics_results, retrieval_results
        
        # Запускаем оценку BM25 отдельно для каждого значения k
        bm25_metrics = {}
        for k in k_values:
            print(f"Оценка BM25 для top_{k}...")
            k_metrics, k_results = run_evaluation_bm25_only(api_key, test_questions, k)
            
            # Добавляем префикс к метрикам
            for metric, value in k_metrics.items():
                if f"@{k}" in metric:  # Добавляем только метрики для текущего k
                    bm25_metrics[f"top_{k}_{metric}"] = value
            
            # Сохраняем результаты BM25
            save_results_to_csv(k_results, f"bm25_only_top{k}_results.csv")
        
        # Сравнение метрик
        print("\nСравнение метрик BM25 и BM25+BGE:")
        print("{:<20} {:<15} {:<15}".format("Метрика", "BM25", "BM25+BGE"))
        print("-" * 50)
        
        for k in k_values:
            for metric_name in [f"recall@{k}", f"precision@{k}", f"mrr@{k}", f"ndcg@{k}"]:
                bm25_key = f"top_{k}_{metric_name}"
                reranker_key = f"top_{k}_{metric_name}"
                
                if bm25_key in bm25_metrics and reranker_key in metrics:
                    bm25_value = bm25_metrics[bm25_key]
                    reranker_value = metrics[reranker_key]
                    
                    print("{:<20} {:<15.4f} {:<15.4f}".format(
                        bm25_key, bm25_value, reranker_value
                    ))
        
        # Анализ улучшений
        total_improvements = 0
        total_metrics = 0
        
        for k in k_values:
            for metric_name in [f"recall@{k}", f"precision@{k}", f"mrr@{k}", f"ndcg@{k}"]:
                bm25_key = f"top_{k}_{metric_name}"
                reranker_key = f"top_{k}_{metric_name}"
                
                if bm25_key in bm25_metrics and reranker_key in metrics:
                    bm25_value = bm25_metrics[bm25_key]
                    reranker_value = metrics[reranker_key]
                    
                    if reranker_value > bm25_value:
                        total_improvements += 1
                    
                    total_metrics += 1
        
        if total_metrics > 0:
            improvement_percentage = (total_improvements / total_metrics) * 100
            print(f"\nРеранкер BGE улучшил {total_improvements} из {total_metrics} метрик ({improvement_percentage:.2f}%)")
        
        return metrics, retrieval_results
    except Exception as e:
        print(f"Ошибка в функции main: {e}")
        import traceback
        traceback.print_exc()
        return None, None

if __name__ == "__main__":
    main()

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\sekho\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\sekho\AppData\Roaming\nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\sekho\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     C:\Users\sekho\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_eng is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\Users\sekho\AppData\Roaming\nltk_data...
[

Начало оценки системы с BM25 и реранкером BGE...
Всего загружено 130 фрагментов.
Начинаем токенизацию и предобработку текста...
Инициализация BM25...
BM25 успешно инициализирован. Всего документов: 130
Инициализация BGE реранкера...
BGE реранкер успешно инициализирован (устройство: cpu)

Результаты оценки системы с BGE реранкером:
top_4_recall@4: 1.0000
top_4_precision@4: 0.4979
top_4_mrr@4: 0.7833
top_4_ndcg@4: 0.8452
top_4_recall@1: 0.5035
top_4_precision@1: 0.7083
top_6_recall@6: 1.0000
top_6_precision@6: 0.4194
top_6_mrr@6: 0.7701
top_6_ndcg@6: 0.8391
top_6_recall@1: 0.4346
top_6_precision@1: 0.6833

Детальный анализ результатов:

Вопрос: What is a collection in the context of Qdrant?
Топ-3 наиболее релевантных фрагмента по оценке Claude:
1. collections (BM25: 1.0890, Релевантность: 5)
   title: Collections weight: 30 aliases: - ../collections - /concepts/collections/ - /documentation/fr...
2. vectors (BM25: 1.0080, Релевантность: 4)
   In order to use multivectors, we need to spec

KeyboardInterrupt: 

In [2]:
# Класс для реранкинга с использованием CrossEncoder модели
class CrossEncoderReranker:
    def __init__(self, model_name="cross-encoder/ms-marco-MiniLM-L-12-v2"):
        try:
            print(f"Инициализация CrossEncoder реранкера ({model_name})...")
            from sentence_transformers import CrossEncoder
            self.model = CrossEncoder(model_name)
            print(f"CrossEncoder реранкер успешно инициализирован")
        except Exception as e:
            print(f"Ошибка при инициализации CrossEncoder реранкера: {e}")
            raise

    def rerank(self, query, documents, fragment_scores, batch_size=32):
        """
        Переранжирует документы с использованием CrossEncoder модели.
        
        Args:
            query: Текст запроса
            documents: Список текстов документов
            fragment_scores: Исходные оценки документов (из BM25)
            batch_size: Размер батча для инференса
            
        Returns:
            Список кортежей (документ, исходная_оценка, новая_оценка)
        """
        if not documents:
            return []

        # Подготовка входных данных в формате для CrossEncoder
        sentence_pairs = [[query, doc] for doc in documents]
        
        # Получение предсказаний
        try:
            scores = self.model.predict(sentence_pairs, batch_size=batch_size)
        except Exception as e:
            print(f"Ошибка при получении предсказаний от CrossEncoder: {e}")
            # В случае ошибки возвращаем исходные результаты
            return list(zip(documents, fragment_scores, fragment_scores))

        # Комбинирование результатов с исходными документами и оценками
        ranked_results = list(zip(documents, fragment_scores, scores))
        
        # Сортировка по убыванию нового скора
        ranked_results = sorted(ranked_results, key=lambda x: x[2], reverse=True)
        
        return ranked_results
    
def main():
    try:
        # Загрузка API ключа
        api_key_file = 'api.txt'
        if os.path.exists(api_key_file):
            with open(api_key_file, 'r') as file:
                api_key = file.read().strip()
        else:
            print(f"Файл с API ключом {api_key_file} не найден!")
            api_key = input("Введите ваш API ключ: ")
        
        # Проверка API ключа
        if not api_key:
            raise ValueError("API ключ не может быть пустым")
        
        df = pd.read_csv('texts_with_answers.csv')
        test_questions = df.question.to_list()
        
        # Определяем значения k для оценки
        k_values = [4, 6]
        
        # Запуск оценки системы с CrossEncoder реранкером
        print("Начало оценки системы с BM25 и реранкером CrossEncoder MS Marco MiniLM...")
        
        # Инициализация QA системы с BM25
        qa_system = DocumentationQA_BM25()
        qa_system.initialize_database()
        
        # Инициализация CrossEncoder реранкера
        reranker = CrossEncoderReranker("cross-encoder/ms-marco-MiniLM-L-12-v2")
        
        # Результаты для последующей оценки
        retrieval_results = []
        
        # Обработка каждого вопроса
        for question in test_questions:
            # Получение фрагментов с помощью BM25 и реранкера
            # Базовая модель отбирает 20 фрагментов, затем реранкер выбирает лучшие
            max_k = max(k_values)  # Максимальное k из запрошенных
            reranked_fragments = get_relevant_fragments_with_reranker(
                qa_system, reranker, question, initial_top_k=20, final_top_k=max_k
            )
            
            # Результаты для текущего вопроса
            result = {'question': question, 'fragments': []}
            
            # Оценка релевантности для каждого фрагмента
            for text, name, bm25_score, rerank_score in reranked_fragments:
                # Делаем задержку между запросами, чтобы не превысить лимиты API
                time.sleep(2)
                relevance_score = evaluate_relevance_with_claude(question, text, api_key)
                result['fragments'].append((text, name, bm25_score, relevance_score))
            
            retrieval_results.append(result)
        
        # Отдельные метрики для каждого значения k
        metrics_results = {}
        for k in k_values:
            # Для каждого k создаем копию результатов, но ограничиваем количество фрагментов до k
            k_results = []
            for result in retrieval_results:
                k_result = {
                    'question': result['question'],
                    'fragments': result['fragments'][:k] if result['fragments'] else []
                }
                k_results.append(k_result)
            
            # Вычисление метрик для текущего k
            k_metrics = calculate_metrics(k_results, [k])
            metrics_results[f'top_{k}'] = k_metrics
        
        # Объединение всех метрик
        metrics = {}
        for k, k_metrics in metrics_results.items():
            for metric_name, value in k_metrics.items():
                metrics[f"{k}_{metric_name}"] = value
        
        # Вывод результатов
        print("\nРезультаты оценки системы с CrossEncoder MS Marco реранкером:")
        for metric, value in metrics.items():
            print(f"{metric}: {value:.4f}")
        
        # Детальный анализ результатов
        print("\nДетальный анализ результатов:")
        for result in retrieval_results:
            question = result['question']
            print(f"\nВопрос: {question}")
            
            if not result['fragments']:
                print("Не найдено релевантных фрагментов для этого вопроса.")
                continue
                
            # Сортировка по оценке релевантности (от Claude)
            sorted_by_relevance = sorted(result['fragments'], key=lambda x: x[3], reverse=True)
            print("Топ-3 наиболее релевантных фрагмента по оценке Claude:")
            for i, (text, name, bm25_score, relevance) in enumerate(sorted_by_relevance[:min(3, len(sorted_by_relevance))]):
                print(f"{i+1}. {name} (BM25: {bm25_score:.4f}, Релевантность: {relevance})")
                print(f"   {text[:100]}...")
            
            # Сортировка по BM25
            sorted_by_bm25 = sorted(result['fragments'], key=lambda x: x[2], reverse=True)
            print("\nТоп-3 наиболее релевантных фрагмента по BM25:")
            for i, (text, name, bm25_score, relevance) in enumerate(sorted_by_bm25[:min(3, len(sorted_by_bm25))]):
                print(f"{i+1}. {name} (BM25: {bm25_score:.4f}, Релевантность: {relevance})")
                print(f"   {text[:100]}...")
        
        # Сохранение результатов в CSV для дальнейшего анализа
        save_results_to_csv(retrieval_results, "cross_encoder_msmarco_reranker_results.csv")
        
        # Проведем сравнение с исходной версией без реранкера
        print("\nНачало оценки только BM25 (без реранкера) для сравнения...")
        
        # Функция для оценки BM25 без реранкера
        def run_evaluation_bm25_only(api_key, test_questions, top_k):
            """Запуск оценки только BM25 системы на наборе тестовых вопросов"""
            # Инициализация системы BM25
            qa_system = DocumentationQA_BM25()
            qa_system.initialize_database()
            
            # Результаты для последующей оценки
            retrieval_results = []
            
            # Обработка каждого вопроса
            for question in test_questions:
                # Получение фрагментов с помощью BM25
                fragments = qa_system.search_similar_paragraphs(question, top_k=top_k)
                
                # Результаты для текущего вопроса
                result = {'question': question, 'fragments': []}
                
                # Оценка релевантности для каждого фрагмента
                for text, name, score in fragments:
                    # Используем сохраненные оценки релевантности, если есть
                    found = False
                    for r in retrieval_results:
                        if r['question'] == question:
                            for t, n, _, rel_score in r['fragments']:
                                if t == text and n == name:
                                    result['fragments'].append((text, name, score, rel_score))
                                    found = True
                                    break
                        if found:
                            break
                    
                    # Если не нашли сохраненную оценку, запрашиваем новую
                    if not found:
                        time.sleep(2)
                        relevance_score = evaluate_relevance_with_claude(question, text, api_key)
                        result['fragments'].append((text, name, score, relevance_score))
                
                retrieval_results.append(result)
            
            # Вычисление метрик
            metrics_results = calculate_metrics(retrieval_results, k_values)
            
            return metrics_results, retrieval_results
        
        # Запускаем оценку BM25 отдельно для каждого значения k
        bm25_metrics = {}
        for k in k_values:
            print(f"Оценка BM25 для top_{k}...")
            k_metrics, k_results = run_evaluation_bm25_only(api_key, test_questions, k)
            
            # Добавляем префикс к метрикам
            for metric, value in k_metrics.items():
                if f"@{k}" in metric:  # Добавляем только метрики для текущего k
                    bm25_metrics[f"top_{k}_{metric}"] = value
            
            # Сохраняем результаты BM25
            save_results_to_csv(k_results, f"bm25_only_top{k}_results.csv")
        
        # Сравнение метрик
        print("\nСравнение метрик BM25 и BM25+CrossEncoder:")
        print("{:<20} {:<15} {:<15}".format("Метрика", "BM25", "BM25+CrossEncoder"))
        print("-" * 50)
        
        for k in k_values:
            for metric_name in [f"recall@{k}", f"precision@{k}", f"mrr@{k}", f"ndcg@{k}"]:
                bm25_key = f"top_{k}_{metric_name}"
                reranker_key = f"top_{k}_{metric_name}"
                
                if bm25_key in bm25_metrics and reranker_key in metrics:
                    bm25_value = bm25_metrics[bm25_key]
                    reranker_value = metrics[reranker_key]
                    
                    print("{:<20} {:<15.4f} {:<15.4f}".format(
                        bm25_key, bm25_value, reranker_value
                    ))
        
        # Анализ улучшений
        total_improvements = 0
        total_metrics = 0
        
        for k in k_values:
            for metric_name in [f"recall@{k}", f"precision@{k}", f"mrr@{k}", f"ndcg@{k}"]:
                bm25_key = f"top_{k}_{metric_name}"
                reranker_key = f"top_{k}_{metric_name}"
                
                if bm25_key in bm25_metrics and reranker_key in metrics:
                    bm25_value = bm25_metrics[bm25_key]
                    reranker_value = metrics[reranker_key]
                    
                    if reranker_value > bm25_value:
                        total_improvements += 1
                    
                    total_metrics += 1
        
        if total_metrics > 0:
            improvement_percentage = (total_improvements / total_metrics) * 100
            print(f"\nРеранкер MS Marco CrossEncoder улучшил {total_improvements} из {total_metrics} метрик ({improvement_percentage:.2f}%)")
        
        return metrics, retrieval_results
    except Exception as e:
        print(f"Ошибка в функции main: {e}")
        import traceback
        traceback.print_exc()
        return None, None
    
if __name__ == "__main__":
    main()

Начало оценки системы с BM25 и реранкером CrossEncoder MS Marco MiniLM...
Всего загружено 130 фрагментов.
Начинаем токенизацию и предобработку текста...
Инициализация BM25...
BM25 успешно инициализирован. Всего документов: 130
Инициализация CrossEncoder реранкера (cross-encoder/ms-marco-MiniLM-L-12-v2)...


config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

C:\Users\sekho\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\sekho\.cache\huggingface\hub\models--cross-encoder--ms-marco-MiniLM-L-12-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/732 [00:00<?, ?B/s]

CrossEncoder реранкер успешно инициализирован

Результаты оценки системы с CrossEncoder MS Marco реранкером:
top_4_recall@4: 1.0000
top_4_precision@4: 0.4458
top_4_mrr@4: 0.7576
top_4_ndcg@4: 0.8143
top_4_recall@1: 0.5590
top_4_precision@1: 0.6917
top_6_recall@6: 1.0000
top_6_precision@6: 0.3903
top_6_mrr@6: 0.7565
top_6_ndcg@6: 0.8275
top_6_recall@1: 0.4601
top_6_precision@1: 0.6750

Детальный анализ результатов:

Вопрос: What is a collection in the context of Qdrant?
Топ-3 наиболее релевантных фрагмента по оценке Claude:
1. collections (BM25: 1.0890, Релевантность: 5)
   title: Collections weight: 30 aliases: - ../collections - /concepts/collections/ - /documentation/fr...
2. collections (BM25: 1.0226, Релевантность: 4)
   Collection with sparse vectors Available as of v1.7.0 Qdrant supports sparse vectors as a first-clas...
3. indexing (BM25: 1.1448, Релевантность: 3)
   A sparse vector index in Qdrant is exact, meaning it does not use any approximation algorithms. All ...

Топ-3 на

KeyboardInterrupt: 

In [5]:
# Класс для реранкинга с использованием CrossEncoder модели
class CrossEncoderReranker:
    def __init__(self, model_name="sentence-transformers/msmarco-distilbert-base-tas-b"):
        try:
            print(f"Инициализация CrossEncoder реранкера ({model_name})...")
            from sentence_transformers import CrossEncoder
            self.model = CrossEncoder(model_name)
            print(f"CrossEncoder реранкер успешно инициализирован")
        except Exception as e:
            print(f"Ошибка при инициализации CrossEncoder реранкера: {e}")
            raise

    def rerank(self, query, documents, fragment_scores, batch_size=32):
        """
        Переранжирует документы с использованием CrossEncoder модели.
        Args:
            query: Текст запроса
            documents: Список текстов документов
            fragment_scores: Исходные оценки документов (из BM25)
            batch_size: Размер батча для инференса
        Returns:
            Список кортежей (документ, исходная_оценка, новая_оценка)
        """
        if not documents:
            return []
        # Подготовка входных данных в формате для CrossEncoder
        sentence_pairs = [[query, doc] for doc in documents]
        # Получение предсказаний
        try:
            scores = self.model.predict(sentence_pairs, batch_size=batch_size)
        except Exception as e:
            print(f"Ошибка при получении предсказаний от CrossEncoder: {e}")
            # В случае ошибки возвращаем исходные результаты
            return list(zip(documents, fragment_scores, fragment_scores))
        # Комбинирование результатов с исходными документами и оценками
        ranked_results = list(zip(documents, fragment_scores, scores))
        # Сортировка по убыванию нового скора
        ranked_results = sorted(ranked_results, key=lambda x: x[2], reverse=True)
        return ranked_results

def main():
    try:
        # Загрузка API ключа
        api_key_file = 'api.txt'
        if os.path.exists(api_key_file):
            with open(api_key_file, 'r') as file:
                api_key = file.read().strip()
        else:
            print(f"Файл с API ключом {api_key_file} не найден!")
            api_key = input("Введите ваш API ключ: ")
        # Проверка API ключа
        if not api_key:
            raise ValueError("API ключ не может быть пустым")
        df = pd.read_csv('texts_with_answers.csv')
        test_questions = df.question.to_list()
        # Определяем значения k для оценки
        k_values = [4, 6]
        # Запуск оценки системы с CrossEncoder реранкером
        print("Начало оценки системы с BM25 и реранкером sentence-transformers/msmarco-distilbert-base-tas-b...")
        # Инициализация QA системы с BM25
        qa_system = DocumentationQA_BM25()
        qa_system.initialize_database()
        # Инициализация CrossEncoder реранкера
        reranker = CrossEncoderReranker("sentence-transformers/msmarco-distilbert-base-tas-b")
        # Результаты для последующей оценки
        retrieval_results = []
        # Обработка каждого вопроса
        for question in test_questions:
            # Получение фрагментов с помощью BM25 и реранкера
            # Базовая модель отбирает 20 фрагментов, затем реранкер выбирает лучшие
            max_k = max(k_values)  # Максимальное k из запрошенных
            reranked_fragments = get_relevant_fragments_with_reranker(
                qa_system, reranker, question, initial_top_k=20, final_top_k=max_k
            )
            # Результаты для текущего вопроса
            result = {'question': question, 'fragments': []}
            # Оценка релевантности для каждого фрагмента
            for text, name, bm25_score, rerank_score in reranked_fragments:
                # Делаем задержку между запросами, чтобы не превысить лимиты API
                time.sleep(2)
                relevance_score = evaluate_relevance_with_claude(question, text, api_key)
                result['fragments'].append((text, name, bm25_score, relevance_score))
            retrieval_results.append(result)
        # Отдельные метрики для каждого значения k
        metrics_results = {}
        for k in k_values:
            # Для каждого k создаем копию результатов, но ограничиваем количество фрагментов до k
            k_results = []
            for result in retrieval_results:
                k_result = {
                    'question': result['question'],
                    'fragments': result['fragments'][:k] if result['fragments'] else []
                }
                k_results.append(k_result)
            # Вычисление метрик для текущего k
            k_metrics = calculate_metrics(k_results, [k])
            metrics_results[f'top_{k}'] = k_metrics
        # Объединение всех метрик
        metrics = {}
        for k, k_metrics in metrics_results.items():
            for metric_name, value in k_metrics.items():
                metrics[f"{k}_{metric_name}"] = value
        # Вывод результатов
        print("\nРезультаты оценки системы с sentence-transformers/msmarco-distilbert-base-tas-b реранкером:")
        for metric, value in metrics.items():
            print(f"{metric}: {value:.4f}")
        # Детальный анализ результатов
        print("\nДетальный анализ результатов:")
        for result in retrieval_results:
            question = result['question']
            print(f"\nВопрос: {question}")
            if not result['fragments']:
                print("Не найдено релевантных фрагментов для этого вопроса.")
                continue
            # Сортировка по оценке релевантности (от Claude)
            sorted_by_relevance = sorted(result['fragments'], key=lambda x: x[3], reverse=True)
            print("Топ-3 наиболее релевантных фрагмента по оценке Claude:")
            for i, (text, name, bm25_score, relevance) in enumerate(sorted_by_relevance[:min(3, len(sorted_by_relevance))]):
                print(f"{i+1}. {name} (BM25: {bm25_score:.4f}, Релевантность: {relevance})")
                print(f"   {text[:100]}...")
            # Сортировка по BM25
            sorted_by_bm25 = sorted(result['fragments'], key=lambda x: x[2], reverse=True)
            print("\nТоп-3 наиболее релевантных фрагмента по BM25:")
            for i, (text, name, bm25_score, relevance) in enumerate(sorted_by_bm25[:min(3, len(sorted_by_bm25))]):
                print(f"{i+1}. {name} (BM25: {bm25_score:.4f}, Релевантность: {relevance})")
                print(f"   {text[:100]}...")
        # Сохранение результатов в CSV для дальнейшего анализа
        save_results_to_csv(retrieval_results, "sentence_transformers_msmarco_distilbert_reranker_results.csv")
        # Проведем сравнение с исходной версией без реранкера
        print("\nНачало оценки только BM25 (без реранкера) для сравнения...")
        # Функция для оценки BM25 без реранкера
        def run_evaluation_bm25_only(api_key, test_questions, top_k):
            """Запуск оценки только BM25 системы на наборе тестовых вопросов"""
            # Инициализация системы BM25
            qa_system = DocumentationQA_BM25()
            qa_system.initialize_database()
            # Результаты для последующей оценки
            retrieval_results = []
            # Обработка каждого вопроса
            for question in test_questions:
                # Получение фрагментов с помощью BM25
                fragments = qa_system.search_similar_paragraphs(question, top_k=top_k)
                # Результаты для текущего вопроса
                result = {'question': question, 'fragments': []}
                # Оценка релевантности для каждого фрагмента
                for text, name, score in fragments:
                    # Используем сохраненные оценки релевантности, если есть
                    found = False
                    for r in retrieval_results:
                        if r['question'] == question:
                            for t, n, _, rel_score in r['fragments']:
                                if t == text and n == name:
                                    result['fragments'].append((text, name, score, rel_score))
                                    found = True
                                    break
                        if found:
                            break
                    # Если не нашли сохраненную оценку, запрашиваем новую
                    if not found:
                        time.sleep(2)
                        relevance_score = evaluate_relevance_with_claude(question, text, api_key)
                        result['fragments'].append((text, name, score, relevance_score))
                retrieval_results.append(result)
            # Вычисление метрик
            metrics_results = calculate_metrics(retrieval_results, k_values)
            return metrics_results, retrieval_results
        # Запускаем оценку BM25 отдельно для каждого значения k
        bm25_metrics = {}
        for k in k_values:
            print(f"Оценка BM25 для top_{k}...")
            k_metrics, k_results = run_evaluation_bm25_only(api_key, test_questions, k)
            # Добавляем префикс к метрикам
            for metric, value in k_metrics.items():
                if f"@{k}" in metric:  # Добавляем только метрики для текущего k
                    bm25_metrics[f"top_{k}_{metric}"] = value
            # Сохраняем результаты BM25
            save_results_to_csv(k_results, f"bm25_only_top{k}_results.csv")
        # Сравнение метрик
        print("\nСравнение метрик BM25 и BM25+sentence-transformers/msmarco-distilbert-base-tas-b:")
        print("{:<20} {:<15} {:<15}".format("Метрика", "BM25", "BM25+CrossEncoder"))
        print("-" * 50)
        for k in k_values:
            for metric_name in [f"recall@{k}", f"precision@{k}", f"mrr@{k}", f"ndcg@{k}"]:
                bm25_key = f"top_{k}_{metric_name}"
                reranker_key = f"top_{k}_{metric_name}"
                if bm25_key in bm25_metrics and reranker_key in metrics:
                    bm25_value = bm25_metrics[bm25_key]
                    reranker_value = metrics[reranker_key]
                    print("{:<20} {:<15.4f} {:<15.4f}".format(
                        bm25_key, bm25_value, reranker_value
                    ))
        # Анализ улучшений
        total_improvements = 0
        total_metrics = 0
        for k in k_values:
            for metric_name in [f"recall@{k}", f"precision@{k}", f"mrr@{k}", f"ndcg@{k}"]:
                bm25_key = f"top_{k}_{metric_name}"
                reranker_key = f"top_{k}_{metric_name}"
                if bm25_key in bm25_metrics and reranker_key in metrics:
                    bm25_value = bm25_metrics[bm25_key]
                    reranker_value = metrics[reranker_key]
                    if reranker_value > bm25_value:
                        total_improvements += 1
                    total_metrics += 1
        if total_metrics > 0:
            improvement_percentage = (total_improvements / total_metrics) * 100
            print(f"\nРеранкер sentence-transformers/msmarco-distilbert-base-tas-b улучшил {total_improvements} из {total_metrics} метрик ({improvement_percentage:.2f}%)")
        return metrics, retrieval_results
    except Exception as e:
        print(f"Ошибка в функции main: {e}")
        import traceback
        traceback.print_exc()
        return None, None

if __name__ == "__main__":
    main()

Начало оценки системы с BM25 и реранкером sentence-transformers/msmarco-distilbert-base-tas-b...
Всего загружено 130 фрагментов.
Начинаем токенизацию и предобработку текста...
Инициализация BM25...
BM25 успешно инициализирован. Всего документов: 130
Инициализация CrossEncoder реранкера (sentence-transformers/msmarco-distilbert-base-tas-b)...


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at sentence-transformers/msmarco-distilbert-base-tas-b and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


CrossEncoder реранкер успешно инициализирован

Результаты оценки системы с sentence-transformers/msmarco-distilbert-base-tas-b реранкером:
top_4_recall@4: 1.0000
top_4_precision@4: 0.2042
top_4_mrr@4: 0.3528
top_4_ndcg@4: 0.4552
top_4_recall@1: 0.6924
top_4_precision@1: 0.2667
top_6_recall@6: 1.0000
top_6_precision@6: 0.2083
top_6_mrr@6: 0.4122
top_6_ndcg@6: 0.5256
top_6_recall@1: 0.6168
top_6_precision@1: 0.3167

Детальный анализ результатов:

Вопрос: What is a collection in the context of Qdrant?
Топ-3 наиболее релевантных фрагмента по оценке Claude:
1. collections (BM25: 1.1910, Релевантность: 4)
   {{< code-snippet path="/documentation/headless/snippets/update-collection/vectors-to-disk-named/" >}...
2. indexing (BM25: 0.9407, Релевантность: 3)
   # Larger the value - more accurate the search, more time required to build index. ef_construct: 100 ...
3. collections (BM25: 1.0687, Релевантность: 3)
   Full API specification is available in schema definitions. Calls to this endpoint m